In [1]:
%pip install azure-search-documents
%pip show azure-search-documents
%pip install python-dotenv

import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,
    ComplexField,
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SemanticSettings,  
)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Name: azure-search-documents
Version: 11.4.0b3
Summary: Microsoft Azure Cognitive Search Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents
Author: Microsoft Corporation
Author-email: ascl@microsoft.com
License: MIT License
Location: c:\Users\210588\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: azure-common, azure-core, isodate
Required-by: 
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
service_name = "sotuken"
admin_key = "RvICL223LXinONIZB3GHMoQTP5MfmCUmoH4iav37B3AzSeA918fw"

index_name = "book-search"

endpoint = "https://sotuken-beta.search.windows.net/".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

In [3]:
try:
    result = admin_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

Index book-search Deleted


In [5]:
name = "book-search"
fields = [
        SimpleField(name="BookId", type=SearchFieldDataType.String, key=True),
        SearchableField(name="BookName", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="BookAuthor", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="BookGenre", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="Release_Date", type=SearchFieldDataType.DateTimeOffset, facetable=True, filterable=True, sortable=True),

        SimpleField(name="Price", type=SearchFieldDataType.Int32, facetable=True, filterable=True, sortable=True),
        SimpleField(name="Rating", type=SearchFieldDataType.Double, facetable=True, filterable=True, sortable=True),

        SearchableField(name="Text", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
        SearchableField(name="Outher_Keyword", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
]
semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="BookName"),
        prioritized_content_fields=[SemanticField(field_name="Text")],
        prioritized_keywords_fields=[SemanticField(field_name="BookGenre")]
    )
)

semantic_settings = SemanticSettings(configurations=[semantic_config])
scoring_profiles = []

In [6]:
index = SearchIndex(
    name=name,
    fields=fields,
    semantic_settings=semantic_settings,
    scoring_profiles=scoring_profiles,
)

try:
    result = admin_client.create_index(index)
    print ('Index', result.name, 'created')
except Exception as ex:
    print (ex)

Index book-search created


In [7]:
documents = [
    {
    "@search.action": "upload",
    "BookId": "92",
    "BookName": "蜘蛛の糸",
    "BookAuthor": "芥川龍之介",
    "BookGenre": "掌編小説",
    "Release_Date": "1918年7月",
    "Rating": "3.71",
    "Text": """ある日の事でございます｡御釈迦様は極楽の蓮池のふちを､独りでぶらぶら御歩きになっていらっしゃいました｡池の中に咲いている蓮の花は､みんな玉のようにまっ白で､そのまん中にある金色の蕊からは､何とも云えない好い匂が､絶間なくあたりへ溢れて居ります｡極楽は丁度朝なのでございましょう｡
 やがて御釈迦様はその池のふちに御佇みになって､水の面を蔽っている蓮の葉の間から､ふと下の容子を御覧になりました｡この極楽の蓮池の下は､丁度地獄の底に当って居りますから､水晶のような水を透き徹して､三途の河や針の山の景色が､丁度覗き眼鏡を見るように､はっきりと見えるのでございます｡
 するとその地獄の底に､ｶﾝﾀﾞﾀと云う男が一人､ほかの罪人と一しょに蠢いている姿が､御眼に止まりました｡このｶﾝﾀﾞﾀと云う男は､人を殺したり家に火をつけたり､いろいろ悪事を働いた大泥坊でございますが､それでもたった一つ､善い事を致した覚えがございます｡と申しますのは､ある時この男が深い林の中を通りますと､小さな蜘蛛が一匹､路ばたを這って行くのが見えました｡そこでｶﾝﾀﾞﾀは早速足を挙げて､踏み殺そうと致しましたが､｢いや､いや､これも小さいながら､命のあるものに違いない｡その命を無暗にとると云う事は､いくら何でも可哀そうだ｡｣と､こう急に思い返して､とうとうその蜘蛛を殺さずに助けてやったからでございます｡
 御釈迦様は地獄の容子を御覧になりながら､このｶﾝﾀﾞﾀには蜘蛛を助けた事があるのを御思い出しになりました｡そうしてそれだけの善い事をした報には､出来るなら､この男を地獄から救い出してやろうと御考えになりました｡幸い､側を見ますと､翡翠のような色をした蓮の葉の上に､極楽の蜘蛛が一匹､美しい銀色の糸をかけて居ります｡御釈迦様はその蜘蛛の糸をそっと御手に御取りになって､玉のような白蓮の間から､遥か下にある地獄の底へ､まっすぐにそれを御下しなさいました｡

二

 こちらは地獄の底の血の池で､ほかの罪人と一しょに､浮いたり沈んだりしていたｶﾝﾀﾞﾀでございます｡何しろどちらを見ても､まっ暗で､たまにそのくら暗からぼんやり浮き上っているものがあると思いますと､それは恐しい針の山の針が光るのでございますから､その心細さと云ったらございません｡その上あたりは墓の中のようにしんと静まり返って､たまに聞えるものと云っては､ただ罪人がつく微な嘆息ばかりでございます｡これはここへ落ちて来るほどの人間は､もうさまざまな地獄の責苦に疲れはてて､泣声を出す力さえなくなっているのでございましょう｡ですからさすが大泥坊のｶﾝﾀﾞﾀも､やはり血の池の血に咽びながら､まるで死にかかった蛙のように､ただもがいてばかり居りました｡
 ところがある時の事でございます｡何気なくｶﾝﾀﾞﾀが頭を挙げて､血の池の空を眺めますと､そのひっそりとした暗の中を､遠い遠い天上から､銀色の蜘蛛の糸が､まるで人目にかかるのを恐れるように､一すじ細く光りながら､するすると自分の上へ垂れて参るのではございませんか｡ｶﾝﾀﾞﾀはこれを見ると､思わず手を拍って喜びました｡この糸に縋りついて､どこまでものぼって行けば､きっと地獄からぬけ出せるのに相違ございません｡いや､うまく行くと､極楽へはいる事さえも出来ましょう｡そうすれば､もう針の山へ追い上げられる事もなくなれば､血の池に沈められる事もある筈はございません｡
 こう思いましたからｶﾝﾀﾞﾀは､早速その蜘蛛の糸を両手でしっかりとつかみながら､一生懸命に上へ上へとたぐりのぼり始めました｡元より大泥坊の事でございますから､こう云う事には昔から､慣れ切っているのでございます｡
 しかし地獄と極楽との間は､何万里となくございますから､いくら焦って見た所で､容易に上へは出られません｡ややしばらくのぼる中に､とうとうｶﾝﾀﾞﾀもくたびれて､もう一たぐりも上の方へはのぼれなくなってしまいました｡そこで仕方がございませんから､まず一休み休むつもりで､糸の中途にぶら下りながら､遥かに目の下を見下しました｡
 すると､一生懸命にのぼった甲斐があって､さっきまで自分がいた血の池は､今ではもう暗の底にいつの間にかかくれて居ります｡それからあのぼんやり光っている恐しい針の山も､足の下になってしまいました｡この分でのぼって行けば､地獄からぬけ出すのも､存外わけがないかも知れません｡ｶﾝﾀﾞﾀは両手を蜘蛛の糸にからみながら､ここへ来てから何年にも出した事のない声で､｢しめた｡しめた｡｣と笑いました｡ところがふと気がつきますと､蜘蛛の糸の下の方には､数限もない罪人たちが､自分ののぼった後をつけて､まるで蟻の行列のように､やはり上へ上へ一心によじのぼって来るではございませんか｡ｶﾝﾀﾞﾀはこれを見ると､驚いたのと恐しいのとで､しばらくはただ､莫迦のように大きな口を開いたまま､眼ばかり動かして居りました｡自分一人でさえ断れそうな､この細い蜘蛛の糸が､どうしてあれだけの人数の重みに堪える事が出来ましょう｡もし万一途中で断れたと致しましたら､折角ここへまでのぼって来たこの肝腎な自分までも､元の地獄へ逆落しに落ちてしまわなければなりません｡そんな事があったら､大変でございます｡が､そう云う中にも､罪人たちは何百となく何千となく､まっ暗な血の池の底から､うようよと這い上って､細く光っている蜘蛛の糸を､一列になりながら､せっせとのぼって参ります｡今の中にどうかしなければ､糸はまん中から二つに断れて､落ちてしまうのに違いありません｡
 そこでｶﾝﾀﾞﾀは大きな声を出して､｢こら､罪人ども｡この蜘蛛の糸は己のものだぞ｡お前たちは一体誰に尋いて､のぼって来た｡下りろ｡下りろ｡｣と喚きました｡
 その途端でございます｡今まで何ともなかった蜘蛛の糸が､急にｶﾝﾀﾞﾀのぶら下っている所から､ぷつりと音を立てて断れました｡ですからｶﾝﾀﾞﾀもたまりません｡あっと云う間もなく風を切って､独楽のようにくるくるまわりながら､見る見る中に暗の底へ､まっさかさまに落ちてしまいました｡
 後にはただ極楽の蜘蛛の糸が､きらきらと細く光りながら､月も星もない空の中途に､短く垂れているばかりでございます｡

三

 御釈迦様は極楽の蓮池のふちに立って､この一部始終をじっと見ていらっしゃいましたが､やがてｶﾝﾀﾞﾀが血の池の底へ石のように沈んでしまいますと､悲しそうな御顔をなさりながら､またぶらぶら御歩きになり始めました｡自分ばかり地獄からぬけ出そうとする､ｶﾝﾀﾞﾀの無慈悲な心が､そうしてその心相当な罰をうけて､元の地獄へ落ちてしまったのが､御釈迦様の御目から見ると､浅間しく思召されたのでございましょう｡
 しかし極楽の蓮池の蓮は､少しもそんな事には頓着致しません｡その玉のような白い花は､御釈迦様の御足のまわりに､ゆらゆら萼を動かして､そのまん中にある金色の蕊からは､何とも云えない好い匂が､絶間なくあたりへ溢れて居ります｡極楽ももう午に近くなったのでございましょう｡"""

    },
    {
    "@search.action": "upload",
    "BookId": "128",
    "BookName": "羅生門",
    "BookAuthor": "芥川龍之介",
    "BookGenre": "短編小説",
    "Release_Date": "1915年11月",
    "Rating": "3.33",
    "Text": """ある日の暮方の事である｡一人の下人が､羅生門の下で雨やみを待っていた｡
 広い門の下には､この男のほかに誰もいない｡ただ､所々丹塗の剥げた､大きな円柱に､蟋蟀が一匹とまっている｡羅生門が､朱雀大路にある以上は､この男のほかにも､雨やみをする市女笠や揉烏帽子が､もう二三人はありそうなものである｡それが､この男のほかには誰もいない｡
 何故かと云うと､この二三年､京都には､地震とか辻風とか火事とか饑饉とか云う災がつづいて起った｡そこで洛中のさびれ方は一通りではない｡旧記によると､仏像や仏具を打砕いて､その丹がついたり､金銀の箔がついたりした木を､路ばたにつみ重ねて､薪の料に売っていたと云う事である｡洛中がその始末であるから､羅生門の修理などは､元より誰も捨てて顧る者がなかった｡するとその荒れ果てたのをよい事にして､狐狸が棲む｡盗人が棲む｡とうとうしまいには､引取り手のない死人を､この門へ持って来て､棄てて行くと云う習慣さえ出来た｡そこで､日の目が見えなくなると､誰でも気味を悪るがって､この門の近所へは足ぶみをしない事になってしまったのである｡
 その代りまた鴉がどこからか､たくさん集って来た｡昼間見ると､その鴉が何羽となく輪を描いて､高い鴟尾のまわりを啼きながら､飛びまわっている｡ことに門の上の空が､夕焼けであかくなる時には､それが胡麻をまいたようにはっきり見えた｡鴉は､勿論､門の上にある死人の肉を､啄みに来るのである｡――もっとも今日は､刻限が遅いせいか､一羽も見えない｡ただ､所々､崩れかかった､そうしてその崩れ目に長い草のはえた石段の上に､鴉の糞が､点々と白くこびりついているのが見える｡下人は七段ある石段の一番上の段に､洗いざらした紺の襖の尻を据えて､右の頬に出来た､大きな面皰を気にしながら､ぼんやり､雨のふるのを眺めていた｡
 作者はさっき､｢下人が雨やみを待っていた｣と書いた｡しかし､下人は雨がやんでも､格別どうしようと云う当てはない｡ふだんなら､勿論､主人の家へ帰る可き筈である｡所がその主人からは､四五日前に暇を出された｡前にも書いたように､当時京都の町は一通りならず衰微していた｡今この下人が､永年､使われていた主人から､暇を出されたのも､実はこの衰微の小さな余波にほかならない｡だから｢下人が雨やみを待っていた｣と云うよりも｢雨にふりこめられた下人が､行き所がなくて､途方にくれていた｣と云う方が､適当である｡その上､今日の空模様も少からず､この平安朝の下人の Sentimentalisme に影響した｡申の刻下りからふり出した雨は､いまだに上るけしきがない｡そこで､下人は､何をおいても差当り明日の暮しをどうにかしようとして――云わばどうにもならない事を､どうにかしようとして､とりとめもない考えをたどりながら､さっきから朱雀大路にふる雨の音を､聞くともなく聞いていたのである｡
 雨は､羅生門をつつんで､遠くから､ざあっと云う音をあつめて来る｡夕闇は次第に空を低くして､見上げると､門の屋根が､斜につき出した甍の先に､重たくうす暗い雲を支えている｡
 どうにもならない事を､どうにかするためには､手段を選んでいる遑はない｡選んでいれば､築土の下か､道ばたの土の上で､饑死をするばかりである｡そうして､この門の上へ持って来て､犬のように棄てられてしまうばかりである｡選ばないとすれば――下人の考えは､何度も同じ道を低徊した揚句に､やっとこの局所へ逢着した｡しかしこの｢すれば｣は､いつまでたっても､結局｢すれば｣であった｡下人は､手段を選ばないという事を肯定しながらも､この｢すれば｣のかたをつけるために､当然､その後に来る可き｢盗人になるよりほかに仕方がない｣と云う事を､積極的に肯定するだけの､勇気が出ずにいたのである｡
 下人は､大きな嚔をして､それから､大儀そうに立上った｡夕冷えのする京都は､もう火桶が欲しいほどの寒さである｡風は門の柱と柱との間を､夕闇と共に遠慮なく､吹きぬける｡丹塗の柱にとまっていた蟋蟀も､もうどこかへ行ってしまった｡
 下人は､頸をちぢめながら､山吹の汗袗に重ねた､紺の襖の肩を高くして門のまわりを見まわした｡雨風の患のない､人目にかかる惧のない､一晩楽にねられそうな所があれば､そこでともかくも､夜を明かそうと思ったからである｡すると､幸い門の上の楼へ上る､幅の広い､これも丹を塗った梯子が眼についた｡上なら､人がいたにしても､どうせ死人ばかりである｡下人はそこで､腰にさげた聖柄の太刀が鞘走らないように気をつけながら､藁草履をはいた足を､その梯子の一番下の段へふみかけた｡
 それから､何分かの後である｡羅生門の楼の上へ出る､幅の広い梯子の中段に､一人の男が､猫のように身をちぢめて､息を殺しながら､上の容子を窺っていた｡楼の上からさす火の光が､かすかに､その男の右の頬をぬらしている｡短い鬚の中に､赤く膿を持った面皰のある頬である｡下人は､始めから､この上にいる者は､死人ばかりだと高を括っていた｡それが､梯子を二三段上って見ると､上では誰か火をとぼして､しかもその火をそこここと動かしているらしい｡これは､その濁った､黄いろい光が､隅々に蜘蛛の巣をかけた天井裏に､揺れながら映ったので､すぐにそれと知れたのである｡この雨の夜に､この羅生門の上で､火をともしているからは､どうせただの者ではない｡
 下人は､守宮のように足音をぬすんで､やっと急な梯子を､一番上の段まで這うようにして上りつめた｡そうして体を出来るだけ､平にしながら､頸を出来るだけ､前へ出して､恐る恐る､楼の内を覗いて見た｡
 見ると､楼の内には､噂に聞いた通り､幾つかの死骸が､無造作に棄ててあるが､火の光の及ぶ範囲が､思ったより狭いので､数は幾つともわからない｡ただ､おぼろげながら､知れるのは､その中に裸の死骸と､着物を着た死骸とがあるという事である｡勿論､中には女も男もまじっているらしい｡そうして､その死骸は皆､それが､かつて､生きていた人間だと云う事実さえ疑われるほど､土を捏ねて造った人形のように､口を開いたり手を延ばしたりして､ごろごろ床の上にころがっていた｡しかも､肩とか胸とかの高くなっている部分に､ぼんやりした火の光をうけて､低くなっている部分の影を一層暗くしながら､永久に唖の如く黙っていた｡
 下人は､それらの死骸の腐爛した臭気に思わず､鼻を掩った｡しかし､その手は､次の瞬間には､もう鼻を掩う事を忘れていた｡ある強い感情が､ほとんどことごとくこの男の嗅覚を奪ってしまったからだ｡
 下人の眼は､その時､はじめてその死骸の中に蹲っている人間を見た｡檜皮色の着物を着た､背の低い､痩せた､白髪頭の､猿のような老婆である｡その老婆は､右の手に火をともした松の木片を持って､その死骸の一つの顔を覗きこむように眺めていた｡髪の毛の長い所を見ると､多分女の死骸であろう｡
 下人は､六分の恐怖と四分の好奇心とに動かされて､暫時は呼吸をするのさえ忘れていた｡旧記の記者の語を借りれば､｢頭身の毛も太る｣ように感じたのである｡すると老婆は､松の木片を､床板の間に挿して､それから､今まで眺めていた死骸の首に両手をかけると､丁度､猿の親が猿の子の虱をとるように､その長い髪の毛を一本ずつ抜きはじめた｡髪は手に従って抜けるらしい｡
 その髪の毛が､一本ずつ抜けるのに従って､下人の心からは､恐怖が少しずつ消えて行った｡そうして､それと同時に､この老婆に対するはげしい憎悪が､少しずつ動いて来た｡――いや､この老婆に対すると云っては､語弊があるかも知れない｡むしろ､あらゆる悪に対する反感が､一分毎に強さを増して来たのである｡この時､誰かがこの下人に､さっき門の下でこの男が考えていた､饑死をするか盗人になるかと云う問題を､改めて持出したら､恐らく下人は､何の未練もなく､饑死を選んだ事であろう｡それほど､この男の悪を憎む心は､老婆の床に挿した松の木片のように､勢いよく燃え上り出していたのである｡
 下人には､勿論､何故老婆が死人の髪の毛を抜くかわからなかった｡従って､合理的には､それを善悪のいずれに片づけてよいか知らなかった｡しかし下人にとっては､この雨の夜に､この羅生門の上で､死人の髪の毛を抜くと云う事が､それだけで既に許すべからざる悪であった｡勿論､下人は､さっきまで自分が､盗人になる気でいた事なぞは､とうに忘れていたのである｡
 そこで､下人は､両足に力を入れて､いきなり､梯子から上へ飛び上った｡そうして聖柄の太刀に手をかけながら､大股に老婆の前へ歩みよった｡老婆が驚いたのは云うまでもない｡
 老婆は､一目下人を見ると､まるで弩にでも弾かれたように､飛び上った｡
｢おのれ､どこへ行く｡｣
 下人は､老婆が死骸につまずきながら､慌てふためいて逃げようとする行手を塞いで､こう罵った｡老婆は､それでも下人をつきのけて行こうとする｡下人はまた､それを行かすまいとして､押しもどす｡二人は死骸の中で､しばらく､無言のまま､つかみ合った｡しかし勝敗は､はじめからわかっている｡下人はとうとう､老婆の腕をつかんで､無理にそこへﾈじ倒した｡丁度､鶏の脚のような､骨と皮ばかりの腕である｡
｢何をしていた｡云え｡云わぬと､これだぞよ｡｣
 下人は､老婆をつき放すと､いきなり､太刀の鞘を払って､白い鋼の色をその眼の前へつきつけた｡けれども､老婆は黙っている｡両手をわなわなふるわせて､肩で息を切りながら､眼を､眼球がﾏﾌﾞﾀの外へ出そうになるほど､見開いて､唖のように執拗く黙っている｡これを見ると､下人は始めて明白にこの老婆の生死が､全然､自分の意志に支配されていると云う事を意識した｡そうしてこの意識は､今までけわしく燃えていた憎悪の心を､いつの間にか冷ましてしまった｡後に残ったのは､ただ､ある仕事をして､それが円満に成就した時の､安らかな得意と満足とがあるばかりである｡そこで､下人は､老婆を見下しながら､少し声を柔らげてこう云った｡
｢己は検非違使の庁の役人などではない｡今し方この門の下を通りかかった旅の者だ｡だからお前に縄をかけて､どうしようと云うような事はない｡ただ､今時分この門の上で､何をして居たのだか､それを己に話しさえすればいいのだ｡｣
 すると､老婆は､見開いていた眼を､一層大きくして､じっとその下人の顔を見守った｡ﾏﾌﾞﾀの赤くなった､肉食鳥のような､鋭い眼で見たのである｡それから､皺で､ほとんど､鼻と一つになった唇を､何か物でも噛んでいるように動かした｡細い喉で､尖った喉仏の動いているのが見える｡その時､その喉から､鴉の啼くような声が､喘ぎ喘ぎ､下人の耳へ伝わって来た｡
｢この髪を抜いてな､この髪を抜いてな､鬘にしようと思うたのじゃ｡｣
 下人は､老婆の答が存外､平凡なのに失望した｡そうして失望すると同時に､また前の憎悪が､冷やかな侮蔑と一しょに､心の中へはいって来た｡すると､その気色が､先方へも通じたのであろう｡老婆は､片手に､まだ死骸の頭から奪った長い抜け毛を持ったなり､蟇のつぶやくような声で､口ごもりながら､こんな事を云った｡
｢成程な､死人の髪の毛を抜くと云う事は､何ぼう悪い事かも知れぬ｡じゃが､ここにいる死人どもは､皆､そのくらいな事を､されてもいい人間ばかりだぞよ｡現在､わしが今､髪を抜いた女などはな､蛇を四寸ばかりずつに切って干したのを､干魚だと云うて､太刀帯の陣へ売りに往んだわ｡疫病にかかって死ななんだら､今でも売りに往んでいた事であろ｡それもよ､この女の売る干魚は､味がよいと云うて､太刀帯どもが､欠かさず菜料に買っていたそうな｡わしは､この女のした事が悪いとは思うていぬ｡せねば､饑死をするのじゃて､仕方がなくした事であろ｡されば､今また､わしのしていた事も悪い事とは思わぬぞよ｡これとてもやはりせねば､饑死をするじゃて､仕方がなくする事じゃわいの｡じゃて､その仕方がない事を､よく知っていたこの女は､大方わしのする事も大目に見てくれるであろ｡｣
 老婆は､大体こんな意味の事を云った｡
 下人は､太刀を鞘におさめて､その太刀の柄を左の手でおさえながら､冷然として､この話を聞いていた｡勿論､右の手では､赤く頬に膿を持った大きな面皰を気にしながら､聞いているのである｡しかし､これを聞いている中に､下人の心には､ある勇気が生まれて来た｡それは､さっき門の下で､この男には欠けていた勇気である｡そうして､またさっきこの門の上へ上って､この老婆を捕えた時の勇気とは､全然､反対な方向に動こうとする勇気である｡下人は､饑死をするか盗人になるかに､迷わなかったばかりではない｡その時のこの男の心もちから云えば､饑死などと云う事は､ほとんど､考える事さえ出来ないほど､意識の外に追い出されていた｡
｢きっと､そうか｡｣
 老婆の話が完ると､下人は嘲るような声で念を押した｡そうして､一足前へ出ると､不意に右の手を面皰から離して､老婆の襟上をつかみながら､噛みつくようにこう云った｡
｢では､己が引剥をしようと恨むまいな｡己もそうしなければ､饑死をする体なのだ｡｣
 下人は､すばやく､老婆の着物を剥ぎとった｡それから､足にしがみつこうとする老婆を､手荒く死骸の上へ蹴倒した｡梯子の口までは､僅に五歩を数えるばかりである｡下人は､剥ぎとった檜皮色の着物をわきにかかえて､またたく間に急な梯子を夜の底へかけ下りた｡
 しばらく､死んだように倒れていた老婆が､死骸の中から､その裸の体を起したのは､それから間もなくの事である｡老婆はつぶやくような､うめくような声を立てながら､まだ燃えている火の光をたよりに､梯子の口まで､這って行った｡そうして､そこから､短い白髪を倒にして､門の下を覗きこんだ｡外には､ただ､黒洞々たる夜があるばかりである｡
 下人の行方は､誰も知らない｡"""
    },
    {
    "@search.action": "upload",
    "BookId": "42",
    "BookName": "鼻",
    "BookAuthor": "芥川龍之介",
    "BookGenre": "掌編小説",
    "Release_Date": "1916年2月",
    "Rating": "3.29",
    "Text": """禅智内供の鼻と云えば､池の尾で知らない者はない｡長さは五六寸あって上唇の上から顋の下まで下っている｡形は元も先も同じように太い｡云わば細長い腸詰めのような物が､ぶらりと顔のまん中からぶら下っているのである｡
 五十歳を越えた内供は､沙弥の昔から､内道場供奉の職に陞った今日まで､内心では始終この鼻を苦に病んで来た｡勿論表面では､今でもさほど気にならないような顔をしてすましている｡これは専念に当来の浄土を渇仰すべき僧侶の身で､鼻の心配をするのが悪いと思ったからばかりではない｡それよりむしろ､自分で鼻を気にしていると云う事を､人に知られるのが嫌だったからである｡内供は日常の談話の中に､鼻と云う語が出て来るのを何よりも惧れていた｡
 内供が鼻を持てあました理由は二つある｡――一つは実際的に､鼻の長いのが不便だったからである｡第一飯を食う時にも独りでは食えない｡独りで食えば､鼻の先が鋺の中の飯へとどいてしまう｡そこで内供は弟子の一人を膳の向うへ坐らせて､飯を食う間中､広さ一寸長さ二尺ばかりの板で､鼻を持上げていて貰う事にした｡しかしこうして飯を食うと云う事は､持上げている弟子にとっても､持上げられている内供にとっても､決して容易な事ではない｡一度この弟子の代りをした中童子が､嚏をした拍子に手がふるえて､鼻を粥の中へ落した話は､当時京都まで喧伝された｡――けれどもこれは内供にとって､決して鼻を苦に病んだ重な理由ではない｡内供は実にこの鼻によって傷つけられる自尊心のために苦しんだのである｡
 池の尾の町の者は､こう云う鼻をしている禅智内供のために､内供の俗でない事を仕合せだと云った｡あの鼻では誰も妻になる女があるまいと思ったからである｡中にはまた､あの鼻だから出家したのだろうと批評する者さえあった｡しかし内供は､自分が僧であるために､幾分でもこの鼻に煩される事が少くなったと思っていない｡内供の自尊心は､妻帯と云うような結果的な事実に左右されるためには､余りにﾃﾞﾘｹｲﾄに出来ていたのである｡そこで内供は､積極的にも消極的にも､この自尊心の毀損を恢復しようと試みた｡
 第一に内供の考えたのは､この長い鼻を実際以上に短く見せる方法である｡これは人のいない時に､鏡へ向って､いろいろな角度から顔を映しながら､熱心に工夫を凝らして見た｡どうかすると､顔の位置を換えるだけでは､安心が出来なくなって､頬杖をついたり頤の先へ指をあてがったりして､根気よく鏡を覗いて見る事もあった｡しかし自分でも満足するほど､鼻が短く見えた事は､これまでにただの一度もない｡時によると､苦心すればするほど､かえって長く見えるような気さえした｡内供は､こう云う時には､鏡を箱へしまいながら､今更のようにため息をついて､不承不承にまた元の経机へ､観音経をよみに帰るのである｡
 それからまた内供は､絶えず人の鼻を気にしていた｡池の尾の寺は､僧供講説などのしばしば行われる寺である｡寺の内には､僧坊が隙なく建て続いて､湯屋では寺の僧が日毎に湯を沸かしている｡従ってここへ出入する僧俗の類も甚だ多い｡内供はこう云う人々の顔を根気よく物色した｡一人でも自分のような鼻のある人間を見つけて､安心がしたかったからである｡だから内供の眼には､紺の水干も白の帷子もはいらない｡まして柑子色の帽子や､椎鈍の法衣なぞは､見慣れているだけに､有れども無きが如くである｡内供は人を見ずに､ただ､鼻を見た｡――しかし鍵鼻はあっても､内供のような鼻は一つも見当らない｡その見当らない事が度重なるに従って､内供の心は次第にまた不快になった｡内供が人と話しながら､思わずぶらりと下っている鼻の先をつまんで見て､年甲斐もなく顔を赤らめたのは､全くこの不快に動かされての所為である｡
 最後に､内供は､内典外典の中に､自分と同じような鼻のある人物を見出して､せめても幾分の心やりにしようとさえ思った事がある｡けれども､目連や､舎利弗の鼻が長かったとは､どの経文にも書いてない｡勿論竜樹や馬鳴も､人並の鼻を備えた菩薩である｡内供は､震旦の話の序に蜀漢の劉玄徳の耳が長かったと云う事を聞いた時に､それが鼻だったら､どのくらい自分は心細くなくなるだろうと思った｡
 内供がこう云う消極的な苦心をしながらも､一方ではまた､積極的に鼻の短くなる方法を試みた事は､わざわざここに云うまでもない｡内供はこの方面でもほとんど出来るだけの事をした｡烏瓜を煎じて飲んで見た事もある｡鼠の尿を鼻へなすって見た事もある｡しかし何をどうしても､鼻は依然として､五六寸の長さをぶらりと唇の上にぶら下げているではないか｡
 所がある年の秋､内供の用を兼ねて､京へ上った弟子の僧が､知己の医者から長い鼻を短くする法を教わって来た｡その医者と云うのは､もと震旦から渡って来た男で､当時は長楽寺の供僧になっていたのである｡
 内供は､いつものように､鼻などは気にかけないと云う風をして､わざとその法もすぐにやって見ようとは云わずにいた｡そうして一方では､気軽な口調で､食事の度毎に､弟子の手数をかけるのが､心苦しいと云うような事を云った｡内心では勿論弟子の僧が､自分を説伏せて､この法を試みさせるのを待っていたのである｡弟子の僧にも､内供のこの策略がわからない筈はない｡しかしそれに対する反感よりは､内供のそう云う策略をとる心もちの方が､より強くこの弟子の僧の同情を動かしたのであろう｡弟子の僧は､内供の予期通り､口を極めて､この法を試みる事を勧め出した｡そうして､内供自身もまた､その予期通り､結局この熱心な勧告に聴従する事になった｡
 その法と云うのは､ただ､湯で鼻を茹でて､その鼻を人に踏ませると云う､極めて簡単なものであった｡
 湯は寺の湯屋で､毎日沸かしている｡そこで弟子の僧は､指も入れられないような熱い湯を､すぐに提に入れて､湯屋から汲んで来た｡しかしじかにこの提へ鼻を入れるとなると､湯気に吹かれて顔を火傷する惧がある｡そこで折敷へ穴をあけて､それを提の蓋にして､その穴から鼻を湯の中へ入れる事にした｡鼻だけはこの熱い湯の中へ浸しても､少しも熱くないのである｡しばらくすると弟子の僧が云った｡
 ――もう茹った時分でござろう｡
 内供は苦笑した｡これだけ聞いたのでは､誰も鼻の話とは気がつかないだろうと思ったからである｡鼻は熱湯に蒸されて､蚤の食ったようにむず痒い｡
 弟子の僧は､内供が折敷の穴から鼻をぬくと､そのまだ湯気の立っている鼻を､両足に力を入れながら､踏みはじめた｡内供は横になって､鼻を床板の上へのばしながら､弟子の僧の足が上下に動くのを眼の前に見ているのである｡弟子の僧は､時々気の毒そうな顔をして､内供の禿げ頭を見下しながら､こんな事を云った｡
 ――痛うはござらぬかな｡医師は責めて踏めと申したで｡じゃが､痛うはござらぬかな｡
 内供は首を振って､痛くないと云う意味を示そうとした｡所が鼻を踏まれているので思うように首が動かない｡そこで､上眼を使って､弟子の僧の足に皹のきれているのを眺めながら､腹を立てたような声で､
――痛うはないて｡
 と答えた｡実際鼻はむず痒い所を踏まれるので､痛いよりもかえって気もちのいいくらいだったのである｡
 しばらく踏んでいると､やがて､粟粒のようなものが､鼻へ出来はじめた｡云わば毛をむしった小鳥をそっくり丸炙にしたような形である｡弟子の僧はこれを見ると､足を止めて独り言のようにこう云った｡
 ――これを鑷子でぬけと申す事でござった｡
 内供は､不足らしく頬をふくらせて､黙って弟子の僧のするなりに任せて置いた｡勿論弟子の僧の親切がわからない訳ではない｡それは分っても､自分の鼻をまるで物品のように取扱うのが､不愉快に思われたからである｡内供は､信用しない医者の手術をうける患者のような顔をして､不承不承に弟子の僧が､鼻の毛穴から鑷子で脂をとるのを眺めていた｡脂は､鳥の羽の茎のような形をして､四分ばかりの長さにぬけるのである｡
 やがてこれが一通りすむと､弟子の僧は､ほっと一息ついたような顔をして､
 ――もう一度､これを茹でればようござる｡
 と云った｡
 内供はやはり､八の字をよせたまま不服らしい顔をして､弟子の僧の云うなりになっていた｡
 さて二度目に茹でた鼻を出して見ると､成程､いつになく短くなっている｡これではあたりまえの鍵鼻と大した変りはない｡内供はその短くなった鼻を撫でながら､弟子の僧の出してくれる鏡を､極りが悪るそうにおずおず覗いて見た｡
 鼻は――あの顋の下まで下っていた鼻は､ほとんど嘘のように萎縮して､今は僅に上唇の上で意気地なく残喘を保っている｡所々まだらに赤くなっているのは､恐らく踏まれた時の痕であろう｡こうなれば､もう誰も哂うものはないにちがいない｡――鏡の中にある内供の顔は､鏡の外にある内供の顔を見て､満足そうに眼をしばたたいた｡
 しかし､その日はまだ一日､鼻がまた長くなりはしないかと云う不安があった｡そこで内供は誦経する時にも､食事をする時にも､暇さえあれば手を出して､そっと鼻の先にさわって見た｡が､鼻は行儀よく唇の上に納まっているだけで､格別それより下へぶら下って来る景色もない｡それから一晩寝てあくる日早く眼がさめると内供はまず､第一に､自分の鼻を撫でて見た｡鼻は依然として短い｡内供はそこで､幾年にもなく､法華経書写の功を積んだ時のような､のびのびした気分になった｡
 所が二三日たつ中に､内供は意外な事実を発見した｡それは折から､用事があって､池の尾の寺を訪れた侍が､前よりも一層可笑しそうな顔をして､話も碌々せずに､じろじろ内供の鼻ばかり眺めていた事である｡それのみならず､かつて､内供の鼻を粥の中へ落した事のある中童子なぞは､講堂の外で内供と行きちがった時に､始めは､下を向いて可笑しさをこらえていたが､とうとうこらえ兼ねたと見えて､一度にふっと吹き出してしまった｡用を云いつかった下法師たちが､面と向っている間だけは､慎んで聞いていても､内供が後さえ向けば､すぐにくすくす笑い出したのは､一度や二度の事ではない｡
 内供ははじめ､これを自分の顔がわりがしたせいだと解釈した｡しかしどうもこの解釈だけでは十分に説明がつかないようである｡――勿論､中童子や下法師が哂う原因は､そこにあるのにちがいない｡けれども同じ哂うにしても､鼻の長かった昔とは､哂うのにどことなく容子がちがう｡見慣れた長い鼻より､見慣れない短い鼻の方が滑稽に見えると云えば､それまでである｡が､そこにはまだ何かあるらしい｡
 ――前にはあのようにつけつけとは哂わなんだて｡
 内供は､誦しかけた経文をやめて､禿げ頭を傾けながら､時々こう呟く事があった｡愛すべき内供は､そう云う時になると､必ずぼんやり､傍にかけた普賢の画像を眺めながら､鼻の長かった四五日前の事を憶い出して､｢今はむげにいやしくなりさがれる人の､さかえたる昔をしのぶがごとく｣ふさぎこんでしまうのである｡――内供には､遺憾ながらこの問に答を与える明が欠けていた｡
 ――人間の心には互に矛盾した二つの感情がある｡勿論､誰でも他人の不幸に同情しない者はない｡所がその人がその不幸を､どうにかして切りぬける事が出来ると､今度はこっちで何となく物足りないような心もちがする｡少し誇張して云えば､もう一度その人を､同じ不幸に陥れて見たいような気にさえなる｡そうしていつの間にか､消極的ではあるが､ある敵意をその人に対して抱くような事になる｡――内供が､理由を知らないながらも､何となく不快に思ったのは､池の尾の僧俗の態度に､この傍観者の利己主義をそれとなく感づいたからにほかならない｡
 そこで内供は日毎に機嫌が悪くなった｡二言目には､誰でも意地悪く叱りつける｡しまいには鼻の療治をしたあの弟子の僧でさえ､｢内供は法慳貪の罪を受けられるぞ｣と陰口をきくほどになった｡殊に内供を怒らせたのは､例の悪戯な中童子である｡ある日､けたたましく犬の吠える声がするので､内供が何気なく外へ出て見ると､中童子は､二尺ばかりの木の片をふりまわして､毛の長い､痩せた尨犬を逐いまわしている｡それもただ､逐いまわしているのではない｡｢鼻を打たれまい｡それ､鼻を打たれまい｣と囃しながら､逐いまわしているのである｡内供は､中童子の手からその木の片をひったくって､したたかその顔を打った｡木の片は以前の鼻持上げの木だったのである｡
 内供はなまじいに､鼻の短くなったのが､かえって恨めしくなった｡
 するとある夜の事である｡日が暮れてから急に風が出たと見えて､塔の風鐸の鳴る音が､うるさいほど枕に通って来た｡その上､寒さもめっきり加わったので､老年の内供は寝つこうとしても寝つかれない｡そこで床の中でまじまじしていると､ふと鼻がいつになく､むず痒いのに気がついた｡手をあてて見ると少し水気が来たようにむくんでいる｡どうやらそこだけ､熱さえもあるらしい｡
 ――無理に短うしたで､病が起ったのかも知れぬ｡
 内供は､仏前に香花を供えるような恭しい手つきで､鼻を抑えながら､こう呟いた｡
 翌朝､内供がいつものように早く眼をさまして見ると､寺内の銀杏や橡が一晩の中に葉を落したので､庭は黄金を敷いたように明るい｡塔の屋根には霜が下りているせいであろう｡まだうすい朝日に､九輪がまばゆく光っている｡禅智内供は､蔀を上げた縁に立って､深く息をすいこんだ｡
 ほとんど､忘れようとしていたある感覚が､再び内供に帰って来たのはこの時である｡
 内供は慌てて鼻へ手をやった｡手にさわるものは､昨夜の短い鼻ではない｡上唇の上から顋の下まで､五六寸あまりもぶら下っている､昔の長い鼻である｡内供は鼻が一夜の中に､また元の通り長くなったのを知った｡そうしてそれと同時に､鼻が短くなった時と同じような､はればれした心もちが､どこからともなく帰って来るのを感じた｡
 ――こうなれば､もう誰も哂うものはないにちがいない｡
 内供は心の中でこう自分に囁いた｡長い鼻をあけ方の秋風にぶらつかせながら｡"""
    },
    
    {
    "@search.action": "upload",
    "BookId": "55",
    "BookName": "芋粥",
    "BookAuthor": "芥川龍之介",
    "BookGenre": "短編小説",
    "Release_Date": "1916年9月",
    "Rating": "3.38",
    "Text": """元慶の末か､仁和の始にあつた話であらう｡どちらにしても時代はさして､この話に大事な役を､勤めてゐない｡読者は唯､平安朝と云ふ､遠い昔が背景になつてゐると云ふ事を､知つてさへゐてくれれば､よいのである｡――その頃､摂政藤原基経に仕へてゐる侍の中に､某と云ふ五位があつた｡
 これも､某と書かずに､何の誰と､ちやんと姓名を明にしたいのであるが､生憎旧記には､それが伝はつてゐない｡恐らくは､実際､伝はる資格がない程､平凡な男だつたのであらう｡一体旧記の著者などと云ふ者は､平凡な人間や話に､余り興味を持たなかつたらしい｡この点で､彼等と､日本の自然派の作家とは､大分ちがふ｡王朝時代の小説家は､存外､閑人でない｡――兎に角､摂政藤原基経に仕へてゐる侍の中に､某と云ふ五位があつた｡これが､この話の主人公である｡
 五位は､風采の甚揚らない男であつた｡第一背が低い｡それから赤鼻で､眼尻が下つてゐる｡口髭は勿論薄い｡頬が､こけてゐるから､頤が､人並はづれて､細く見える｡唇は――一々､数へ立ててゐれば､際限はない｡我五位の外貌はそれ程､非凡に､だらしなく､出来上つてゐたのである｡
 この男が､何時､どうして､基経に仕へるやうになつたのか､それは誰も知つてゐない｡が､余程以前から､同じやうな色の褪めた水干に､同じやうな萎々した烏帽子をかけて､同じやうな役目を､飽きずに､毎日､繰返してゐる事だけは､確である｡その結果であらう､今では､誰が見ても､この男に若い時があつたとは思はれない｡(五位は四十を越してゐた｡)その代り､生れた時から､あの通り寒むさうな赤鼻と､形ばかりの口髭とを､朱雀大路の衢風に､吹かせてゐたと云ふ気がする｡上は主人の基経から､下は牛飼の童児まで､無意識ながら､悉さう信じて疑ふ者がない｡
 かう云ふ風采を具へた男が､周囲から受ける待遇は､恐らく書くまでもないことであらう｡侍所にゐる連中は､五位に対して､殆ど蠅程の注意も払はない｡有位無位､併せて二十人に近い下役さへ､彼の出入りには､不思議な位､冷淡を極めてゐる｡五位が何か云ひつけても､決して彼等同志の雑談をやめた事はない｡彼等にとつては､空気の存在が見えないやうに､五位の存在も､眼を遮らないのであらう｡下役でさへさうだとすれば､別当とか､侍所の司とか云ふ上役たちが頭から彼を相手にしないのは､寧ろ自然の数である｡彼等は､五位に対すると､殆ど､子供らしい無意味な悪意を､冷然とした表情の後に隠して､何を云ふのでも､手真似だけで用を足した｡人間に､言語があるのは､偶然ではない｡従つて､彼等も手真似では用を弁じない事が､時々ある｡が､彼等は､それを全然五位の悟性に､欠陥があるからだと､思つてゐるらしい｡そこで彼等は用が足りないと､この男の歪んだ揉烏帽子の先から､切れかかつた藁草履の尻まで､万遍なく見上げたり､見下したりして､それから､鼻で哂ひながら､急に後を向いてしまふ｡それでも､五位は､腹を立てた事がない｡彼は､一切の不正を､不正として感じない程､意気地のない､臆病な人間だつたのである｡
 所が､同僚の侍たちになると､進んで､彼を飜弄しようとした｡年かさの同僚が､彼れの振はない風采を材料にして､古い洒落を聞かせようとする如く､年下の同僚も､亦それを機会にして､所謂興言利口の練習をしようとしたからである｡彼等は､この五位の面前で､その鼻と口髭と､烏帽子と水干とを､品隲して飽きる事を知らなかつた｡そればかりではない｡彼が五六年前に別れたうけ唇の女房と､その女房と関係があつたと云ふ酒のみの法師とも､屡彼等の話題になつた｡その上､どうかすると､彼等は甚､性質の悪い悪戯さへする｡それを今一々､列記する事は出来ない｡が､彼の篠枝の酒を飲んで､後へ尿を入れて置いたと云ふ事を書けば､その外は凡､想像される事だらうと思ふ｡
 しかし､五位はこれらの揶揄に対して､全然無感覚であつた｡少くもわき眼には､無感覚であるらしく思はれた｡彼は何を云はれても､顔の色さへ変へた事がない｡黙つて例の薄い口髭を撫でながら､するだけの事をしてすましてゐる｡唯､同僚の悪戯が､嵩じすぎて､髷に紙切れをつけたり､太刀の鞘に草履を結びつけたりすると､彼は笑ふのか､泣くのか､わからないやうな笑顔をして､｢いけぬのう､お身たちは｡｣と云ふ｡その顔を見､その声を聞いた者は､誰でも一時或いぢらしさに打たれてしまふ｡(彼等にいぢめられるのは､一人､この赤鼻の五位だけではない､彼等の知らない誰かが――多数の誰かが､彼の顔と声とを借りて､彼等の無情を責めてゐる｡)――さう云ふ気が､朧げながら､彼等の心に､一瞬の間､しみこんで来るからである｡唯その時の心もちを､何時までも持続ける者は甚少い｡その少い中の一人に､或無位の侍があつた｡これは丹波の国から来た男で､まだ柔かい口髭が､やつと鼻の下に､生えかかつた位の青年である｡勿論､この男も始めは皆と一しよに､何の理由もなく､赤鼻の五位を軽蔑した｡所が､或日何かの折に､｢いけぬのう､お身たちは｣と云ふ声を聞いてからは､どうしても､それが頭を離れない｡それ以来､この男の眼にだけは､五位が全く別人として､映るやうになつた｡栄養の不足した､血色の悪い､間のぬけた五位の顔にも､世間の迫害にべそを掻いた､｢人間｣が覗いてゐるからである｡この無位の侍には､五位の事を考へる度に､世の中のすべてが急に本来の下等さを露すやうに思はれた｡さうしてそれと同時に霜げた赤鼻と数へる程の口髭とが何となく一味の慰安を自分の心に伝へてくれるやうに思はれた｡……
 しかし､それは､唯この男一人に､限つた事である｡かう云ふ例外を除けば､五位は､依然として周囲の軽蔑の中に､犬のやうな生活を続けて行かなければならなかつた｡第一彼には着物らしい着物が一つもない｡青鈍の水干と､同じ色の指貫とが一つづつあるのが､今ではそれが上白んで､藍とも紺とも､つかないやうな色に､なつてゐる｡水干はそれでも､肩が少し落ちて､丸組の緒や菊綴の色が怪しくなつてゐるだけだが､指貫になると､裾のあたりのいたみ方が一通りでない｡その指貫の中から､下の袴もはかない､細い足が出てゐるのを見ると､口の悪い同僚でなくとも､痩公卿の車を牽いてゐる､痩牛の歩みを見るやうな､みすぼらしい心もちがする｡それに佩いてゐる太刀も､頗る覚束ない物で､柄の金具も如何はしければ､黒鞘の塗も剥げかかつてゐる｡これが例の赤鼻で､だらしなく草履をひきずりながら､唯でさへ猫背なのを､一層寒空の下に背ぐくまつて､もの欲しさうに､左右を眺め眺め､きざみ足に歩くのだから､通りがかりの物売りまで莫迦にするのも､無理はない｡現に､かう云ふ事さへあつた｡……
 或る日､五位が三条坊門を神泉苑の方へ行く所で､子供が六七人､路ばたに集つて､何かしてゐるのを見た事がある｡｢こまつぶり｣でも､廻してゐるのかと思つて､後ろから覗いて見ると､何処かから迷つて来た､尨犬の首へ繩をつけて､打つたり殴いたりしてゐるのであつた｡臆病な五位は､これまで何かに同情を寄せる事があつても､あたりへ気を兼ねて､まだ一度もそれを行為に現はしたことがない｡が､この時だけは相手が子供だと云ふので､幾分か勇気が出た｡そこで出来るだけ､笑顔をつくりながら､年かさらしい子供の肩を叩いて､｢もう､堪忍してやりなされ｡犬も打たれれば､痛いでのう｣と声をかけた｡すると､その子供はふりかへりながら､上眼を使つて､蔑すむやうに､ぢろぢろ五位の姿を見た｡云はば侍所の別当が用の通じない時に､この男を見るやうな顔をして､見たのである｡｢いらぬ世話はやかれたうもない｡｣その子供は一足下りながら､高慢な唇を反らせて､かう云つた｡｢何ぢや､この鼻赤めが｡｣五位はこの語が自分の顔を打つたやうに感じた｡が､それは悪態をつかれて､腹が立つたからでは毛頭ない｡云はなくともいい事を云つて､恥をかいた自分が､情なくなつたからである｡彼は､きまりが悪いのを苦しい笑顔に隠しながら､黙つて､又､神泉苑の方へ歩き出した｡後では､子供が､六七人､肩を寄せて､｢べつかつかう｣をしたり､舌を出したりしてゐる｡勿論彼はそんな事を知らない｡知つてゐたにしても､それが､この意気地のない五位にとつて､何であらう｡……
 では､この話の主人公は､唯､軽蔑される為にのみ生れて来た人間で､別に何の希望も持つてゐないかと云ふと､さうでもない｡五位は五六年前から芋粥と云ふ物に､異常な執着を持つてゐる｡芋粥とは山の芋を中に切込んで､それを甘葛の汁で煮た､粥の事を云ふのである｡当時はこれが､無上の佳味として､上は万乗の君の食膳にさへ､上せられた｡従つて､吾五位の如き人間の口へは､年に一度､臨時の客の折にしか､はいらない｡その時でさへ､飲めるのは僅に喉を沾すに足る程の少量である｡そこで芋粥を飽きる程飲んで見たいと云ふ事が､久しい前から､彼の唯一の欲望になつてゐた｡勿論､彼は､それを誰にも話した事がない｡いや彼自身さへそれが､彼の一生を貫いてゐる欲望だとは､明白に意識しなかつた事であらう｡が事実は彼がその為に､生きてゐると云つても､差支ない程であつた｡――人間は､時として､充されるか充されないか､わからない欲望の為に､一生を捧げてしまふ｡その愚を哂ふ者は､畢竟､人生に対する路傍の人に過ぎない｡
 しかし､五位が夢想してゐた､｢芋粥に飽かむ｣事は､存外容易に事実となつて現れた｡その始終を書かうと云ふのが､芋粥の話の目的なのである｡
       ―――――――――――――――――
 或年の正月二日､基経の第に､所謂臨時の客があつた時の事である｡(臨時の客は二宮の大饗と同日に摂政関白家が､大臣以下の上達部を招いて催す饗宴で､大饗と別に変りがない｡)五位も､外の侍たちにまじつて､その残肴の相伴をした｡当時はまだ､取食みの習慣がなくて､残肴は､その家の侍が一堂に集まつて､食ふ事になつてゐたからである｡尤も､大饗に等しいと云つても昔の事だから､品数の多い割りに碌な物はない､餅､伏菟､蒸鮑､干鳥､宇治の氷魚､近江の鮒､鯛の楚割､鮭の内子､焼蛸､大海老､大柑子､小柑子､橘､串柿などの類である｡唯､その中に､例の芋粥があつた｡五位は毎年､この芋粥を楽しみにしてゐる｡が､何時も人数が多いので､自分が飲めるのは､いくらもない｡それが今年は､特に､少かつた｡さうして気のせゐか､何時もより､余程味が好い｡そこで､彼は飲んでしまつた後の椀をしげしげと眺めながら､うすい口髭についてゐる滴を､掌で拭いて誰に云ふともなく､｢何時になつたら､これに飽ける事かのう｣と､かう云つた｡
｢大夫殿は､芋粥に飽かれた事がないさうな｡｣
 五位の語が完らない中に､誰かが､嘲笑つた｡錆のある､鷹揚な､武人らしい声である｡五位は､猫背の首を挙げて､臆病らしく､その人の方を見た｡声の主は､その頃同じ基経の恪勤になつてゐた､民部卿時長の子藤原利仁である｡肩幅の広い､身長の群を抜いた逞しい大男で､これは､ﾕﾃﾞｸﾞﾘを噛みながら､黒酒の杯を重ねてゐた｡もう大分酔がまはつてゐるらしい｡
｢お気の毒な事ぢやの｡｣利仁は､五位が顔を挙げたのを見ると､軽蔑と憐憫とを一つにしたやうな声で､語を継いだ｡｢お望みなら､利仁がお飽かせ申さう｡｣
 始終､いぢめられてゐる犬は､たまに肉を貰つても容易によりつかない｡五位は､例の笑ふのか､泣くのか､わからないやうな笑顔をして､利仁の顔と､空の椀とを等分に見比べてゐた｡
｢おいやかな｡｣
｢……｣
｢どうぢや｡｣
｢……｣
 五位は､その中に､衆人の視線が､自分の上に､集まつてゐるのを感じ出した｡答へ方一つで､又､一同の嘲弄を､受けなければならない｡或は､どう答へても､結局､莫迦にされさうな気さへする｡彼は躊躇した｡もし､その時に､相手が､少し面倒臭そうな声で､｢おいやなら､たつてとは申すまい｣と云はなかつたなら､五位は､何時までも､椀と利仁とを､見比べてゐた事であらう｡
 彼は､それを聞くと､慌しく答へた｡
｢いや……忝うござる｡｣
 この問答を聞いてゐた者は､皆､一時に､失笑した｡｢いや……忝うござる｡｣――かう云つて､五位の答を､真似る者さへある｡所謂､橙黄橘紅を盛つた窪坏や高坏の上に多くの揉烏帽子や立烏帽子が､笑声と共に一しきり､波のやうに動いた｡中でも､最､大きな声で､機嫌よく､笑つたのは､利仁自身である｡
｢では､その中に､御誘ひ申さう｡｣さう云ひながら､彼は､ちよいと顔をしかめた｡こみ上げて来る笑と今飲んだ酒とが､喉で一つになつたからである｡｢……しかと､よろしいな｡｣
｢忝うござる｡｣
 五位は赤くなつて､吃りながら､又､前の答を繰返した｡一同が今度も､笑つたのは､云ふまでもない｡それが云はせたさに､わざわざ念を押した当の利仁に至つては､前よりも一層可笑しさうに広い肩をゆすつて､哄笑した｡この朔北の野人は､生活の方法を二つしか心得てゐない｡一つは酒を飲む事で､他の一つは笑ふ事である｡
 しかし幸に談話の中心は､程なく､この二人を離れてしまつた｡これは事によると､外の連中が､たとひ嘲弄にしろ､一同の注意をこの赤鼻の五位に集中させるのが､不快だつたからかも知れない｡兎に角､談柄はそれからそれへと移つて､酒も肴も残少になつた時分には､某と云ふ侍学生が､行縢の片皮へ､両足を入れて馬に乗らうとした話が､一座の興味を集めてゐた｡が､五位だけは､まるで外の話が聞えないらしい｡恐らく芋粥の二字が､彼のすべての思量を支配してゐるからであらう｡前に雉子の炙いたのがあつても､箸をつけない｡黒酒の杯があつても､口を触れない｡彼は､唯､両手を膝の上に置いて､見合ひをする娘のやうに霜に犯されかかつた鬢の辺まで､初心らしく上気しながら､何時までも空になつた黒塗の椀を見つめて､多愛もなく､微笑してゐるのである｡……
       ―――――――――――――――――
 それから､四五日たつた日の午前､加茂川の河原に沿つて､粟田口へ通ふ街道を､静に馬を進めてゆく二人の男があつた｡一人は濃い縹の狩衣に同じ色の袴をして､打出の太刀を佩いた｢鬚黒く鬢ぐきよき｣男である｡もう一人は､みすぼらしい青鈍の水干に､薄綿の衣を二つばかり重ねて着た､四十恰好の侍で､これは､帯のむすび方のだらしのない容子と云ひ､赤鼻でしかも穴のあたりが､洟にぬれてゐる容子と云ひ､身のまはり万端のみすぼらしい事夥しい｡尤も､馬は二人とも､前のは月毛､後のは蘆毛の三歳駒で､道をゆく物売りや侍も､振向いて見る程の駿足である｡その後から又二人､馬の歩みに遅れまいとして随いて行くのは､調度掛と舎人とに相違ない｡――これが､利仁と五位との一行である事は､わざわざ､ここに断るまでもない話であらう｡
 冬とは云ひながら､物静に晴れた日で､白けた河原の石の間､潺湲たる水の辺に立枯れてゐる蓬の葉を､ゆする程の風もない｡川に臨んだ背の低い柳は､葉のない枝に飴の如く滑かな日の光りをうけて､梢にゐる鶺鴒の尾を動かすのさへ､鮮かに､それと､影を街道に落してゐる｡東山の暗い緑の上に､霜に焦げた天鵞絨のやうな肩を､丸々と出してゐるのは､大方､比叡の山であらう｡二人はその中に鞍の螺鈿を､まばゆく日にきらめかせながら鞭をも加へず悠々と､粟田口を指して行くのである｡
｢どこでござるかな､手前をつれて行つて､やらうと仰せられるのは｡｣五位が馴れない手に手綱をかいくりながら､云つた｡
｢すぐ､そこぢや｡お案じになる程遠くはない｡｣
｢すると､粟田口辺でござるかな｡｣
｢まづ､さう思はれたがよろしからう｡｣
 利仁は今朝五位を誘ふのに､東山の近くに湯の湧いてゐる所があるから､そこへ行かうと云つて出て来たのである｡赤鼻の五位は､それを真にうけた｡久しく湯にはいらないので､体中がこの間からむづ痒い｡芋粥の馳走になつた上に､入湯が出来れば､願つてもない仕合せである｡かう思つて､予め利仁が牽かせて来た､蘆毛の馬に跨つた｡所が､轡を並べて此処まで来て見ると､どうも利仁はこの近所へ来るつもりではないらしい｡現に､さうかうしてゐる中に､粟田口は通りすぎた｡
｢粟田口では､ござらぬのう｡｣
｢いかにも､もそつと､あなたでな｡｣
 利仁は､微笑を含みながら､わざと､五位の顔を見ないやうにして､静に馬を歩ませてゐる｡両側の人家は､次第に稀になつて､今は､広々とした冬田の上に､餌をあさる鴉が見えるばかり､山の陰に消残つて､雪の色も仄に青く煙つてゐる｡晴れながら､とげとげしい櫨の梢が､眼に痛く空を刺してゐるのさへ､何となく肌寒い｡
｢では､山科辺ででもござるかな｡｣
｢山科は､これぢや｡もそつと､さきでござるよ｡｣
 成程､さう云ふ中に､山科も通りすぎた｡それ所ではない｡何かとする中に､関山も後にして､彼是､午少しすぎた時分には､とうとう三井寺の前へ来た｡三井寺には､利仁の懇意にしてゐる僧がある｡二人はその僧を訪ねて､午餐の馳走になつた｡それがすむと､又､馬に乗つて､途を急ぐ｡行手は今まで来た路に比べると遙に人煙が少ない｡殊に当時は盗賊が四方に横行した､物騒な時代である｡――五位は猫背を一層低くしながら､利仁の顔を見上げるやうにして訊ねた｡
｢まだ､さきでござるのう｡｣
 利仁は微笑した｡悪戯をして､それを見つけられさうになつた子供が､年長者に向つてするやうな微笑である｡鼻の先へよせた皺と､眼尻にたたへた筋肉のたるみとが､笑つてしまはうか､しまふまいかとためらつてゐるらしい｡さうして､とうとう､かう云つた｡
｢実はな､敦賀まで､お連れ申さうと思うたのぢや｡｣笑ひながら､利仁は鞭を挙げて遠くの空を指さした｡その鞭の下には､ﾃｷﾚｷとして､午後の日を受けた近江の湖が光つてゐる｡
 五位は､狼狽した｡
｢敦賀と申すと､あの越前の敦賀でござるかな｡あの越前の――｣
 利仁が､敦賀の人､藤原有仁の女婿になつてから､多くは敦賀に住んでゐると云ふ事も､日頃から聞いてゐない事はない｡が､その敦賀まで自分をつれて行く気だらうとは､今の今まで思はなかつた｡第一､幾多の山河を隔ててゐる越前の国へ､この通り､僅二人の伴人をつれただけで､どうして無事に行かれよう｡ましてこの頃は､往来の旅人が､盗賊の為に殺されたと云ふ噂さへ､諸方にある｡――五位は歎願するやうに､利仁の顔を見た｡
｢それは又､滅相な､東山ぢやと心得れば､山科｡山科ぢやと心得れば､三井寺｡揚句が越前の敦賀とは､一体どうしたと云ふ事でござる｡始めから､さう仰せられうなら､下人共なりと､召つれようものを｡――敦賀とは､滅相な｡｣
 五位は､殆どべそを掻かないばかりになつて､呟いた｡もし｢芋粥に飽かむ｣事が､彼の勇気を鼓舞しなかつたとしたら､彼は恐らく､そこから別れて､京都へ独り帰つて来た事であらう｡
｢利仁が一人居るのは､千人ともお思ひなされ｡路次の心配は､御無用ぢや｡｣
 五位の狼狽するのを見ると､利仁は､少し眉を顰めながら､嘲笑つた｡さうして調度掛を呼寄せて､持たせて来たﾂﾎﾞﾔﾅｸﾞﾋを背に負ふと､やはり､その手から､黒漆の真弓をうけ取つて､それを鞍上に横へながら､先に立つて､馬を進めた｡かうなる以上､意気地のない五位は､利仁の意志に盲従するより外に仕方がない｡それで､彼は心細さうに､荒涼とした周囲の原野を眺めながら､うろ覚えの観音経を口の中に念じ念じ､例の赤鼻を鞍の前輪にすりつけるやうにして､覚束ない馬の歩みを､不相変とぼとぼと進めて行つた｡
 馬蹄の反響する野は､茫々たる黄茅に蔽はれて､その所々にある行潦も､つめたく､青空を映したまま､この冬の午後を､何時かそれなり凍つてしまふかと疑はれる｡その涯には､一帯の山脈が､日に背いてゐるせゐか､かがやく可き残雪の光もなく､紫がかつた暗い色を､長々となすつてゐるが､それさへ蕭条たる幾叢の枯薄に遮られて､二人の従者の眼には､はいらない事が多い｡――すると､利仁が､突然､五位の方をふりむいて､声をかけた｡
｢あれに､よい使者が参つた｡敦賀への言づけを申さう｡｣
 五位は利仁の云ふ意味が､よくわからないので､怖々ながら､その弓で指さす方を､眺めて見た｡元より人の姿が見えるやうな所ではない｡唯､野葡萄か何かの蔓が､灌木の一むらにからみついてゐる中を､一疋の狐が､暖かな毛の色を､傾きかけた日に曝しながら､のそりのそり歩いて行く｡――と思ふ中に､狐は､慌ただしく身を跳らせて､一散に､どこともなく走り出した｡利仁が急に､鞭を鳴らせて､その方へ馬を飛ばし始めたからである｡五位も､われを忘れて､利仁の後を､逐つた｡従者も勿論､遅れてはゐられない｡しばらくは､石を蹴る馬蹄の音が､戞々として､曠野の静けさを破つてゐたが､やがて利仁が､馬を止めたのを見ると､何時､捕へたのか､もう狐の後足を掴んで､倒に､鞍の側へ､ぶら下げてゐる｡狐が､走れなくなるまで､追ひつめた所で､それを馬の下に敷いて､手取りにしたものであらう｡五位は､うすい髭にたまる汗を､慌しく拭きながら､漸､その傍へ馬を乗りつけた｡
｢これ､狐､よう聞けよ｡｣利仁は､狐を高く眼の前へつるし上げながら､わざと物々しい声を出してかう云つた｡｢其方､今夜の中に､敦賀の利仁が館へ参つて､かう申せ｡『利仁は､唯今俄に客人を具して下らうとする所ぢや｡明日､巳時頃､高島の辺まで､男たちを迎ひに遣はし､それに､鞍置馬二疋､牽かせて参れ｡』よいか忘れるなよ｡｣
 云ひ畢ると共に､利仁は､一ふり振つて狐を､遠くの叢の中へ､抛り出した｡
｢いや､走るわ｡走るわ｡｣
 やつと､追ひついた二人の従者は､逃げてゆく狐の行方を眺めながら､手を拍つて囃し立てた｡落葉のやうな色をしたその獣の背は､夕日の中を､まつしぐらに､木の根石くれの嫌ひなく､何処までも､走つて行く｡それが一行の立つてゐる所から､手にとるやうによく見えた｡狐を追つてゐる中に､何時か彼等は､曠野が緩い斜面を作つて､水の涸れた川床と一つになる､その丁度上の所へ､出てゐたからである｡
｢広量の御使でござるのう｡｣
 五位は､ﾅｲｲｳﾞな尊敬と讃嘆とを洩らしながら､この狐さへ頤使する野育ちの武人の顔を､今更のやうに､仰いで見た｡自分と利仁との間に､どれ程の懸隔があるか､そんな事は､考へる暇がない｡唯､利仁の意志に､支配される範囲が広いだけに､その意志の中に包容される自分の意志も､それだけ自由が利くやうになつた事を､心強く感じるだけである｡――阿諛は､恐らく､かう云ふ時に､最自然に生れて来るものであらう｡読者は､今後､赤鼻の五位の態度に､幇間のやうな何物かを見出しても､それだけで妄にこの男の人格を､疑ふ可きではない｡
 抛り出された狐は､なぞへの斜面を､転げるやうにして､駈け下りると､水の無い河床の石の間を､器用に､ぴよいぴよい､飛び越えて､今度は､向うの斜面へ､勢よく､すぢかひに駈け上つた｡駈け上りながら､ふりかへつて見ると､自分を手捕りにした侍の一行は､まだ遠い傾斜の上に馬を並べて立つてゐる｡それが皆､指を揃へた程に､小さく見えた｡殊に入日を浴びた､月毛と蘆毛とが､霜を含んだ空気の中に､描いたよりもくつきりと､浮き上つてゐる｡
 狐は､頭をめぐらすと､又枯薄の中を､風のやうに走り出した｡
       ―――――――――――――――――
 一行は､予定通り翌日の巳時ばかりに､高島の辺へ来た｡此処は琵琶湖に臨んだ､ささやかな部落で､昨日に似ず､どんよりと曇つた空の下に､幾戸の藁屋が､疎にちらばつてゐるばかり､岸に生えた松の樹の間には､灰色の漣※(｢さんずい+猗｣､第3水準1-87-6)をよせる湖の水面が､磨くのを忘れた鏡のやうに､さむざむと開けてゐる｡――此処まで来ると利仁が､五位を顧みて云つた｡
｢あれを御覧じろ｡男どもが､迎ひに参つたげでござる｡｣
 見ると､成程､二疋の鞍置馬を牽いた､二三十人の男たちが､馬に跨がつたのもあり徒歩のもあり､皆水干の袖を寒風に翻へして､湖の岸､松の間を､一行の方へ急いで来る｡やがてこれが､間近くなつたと思ふと､馬に乗つてゐた連中は､慌ただしく鞍を下り､徒歩の連中は､路傍に蹲踞して､いづれも恭々しく､利仁の来るのを､待ちうけた｡
｢やはり､あの狐が､使者を勤めたと見えますのう｡｣
｢生得､変化ある獣ぢやて､あの位の用を勤めるのは､何でもござらぬ｡｣
 五位と利仁とが､こんな話をしてゐる中に､一行は､郎等たちの待つてゐる所へ来た｡｢大儀ぢや｡｣と､利仁が声をかける｡蹲踞してゐた連中が､忙しく立つて､二人の馬の口を取る｡急に､すべてが陽気になつた｡
｢夜前､稀有な事が､ございましてな｡｣
 二人が､馬から下りて､敷皮の上へ､腰を下すか下さない中に､檜皮色の水干を着た､白髪の郎等が､利仁の前へ来て､かう云つた｡｢何ぢや｡｣利仁は､郎等たちの持つて来た篠枝や破籠を､五位にも勧めながら､鷹揚に問ひかけた｡
｢さればでございまする｡夜前､戌時ばかりに､奥方が俄に､人心地をお失ひなされましてな｡『おのれは､阪本の狐ぢや｡今日､殿の仰せられた事を､言伝てせうほどに､近う寄つて､よう聞きやれ｡』と､かう仰有るのでございまする｡さて､一同がお前に参りますると､奥方の仰せられまするには､『殿は唯今俄に客人を具して､下られようとする所ぢや｡明日巳時頃､高島の辺まで､男どもを迎ひに遺はし､それに鞍置馬二疋牽かせて参れ｡』と､かう御意遊ばすのでございまする｡｣
｢それは､又､稀有な事でござるのう｡｣五位は利仁の顔と､郎等の顔とを､仔細らしく見比べながら､両方に満足を与へるやうな､相槌を打つた｡
｢それも唯､仰せられるのではございませぬ｡さも､恐ろしさうに､わなわなとお震へになりましてな､『遅れまいぞ｡遅れれば､おのれが､殿の御勘当をうけねばならぬ｡』と､しつきりなしに､お泣きになるのでございまする｡｣
｢して､それから､如何した｡｣
｢それから､多愛なく､お休みになりましてな｡手前共の出て参りまする時にも､まだ､お眼覚にはならぬやうで､ございました｡｣
｢如何でござるな｡｣郎等の話を聞き完ると､利仁は五位を見て､得意らしく云つた｡｢利仁には､獣も使はれ申すわ｡｣
｢何とも驚き入る外は､ござらぬのう｡｣五位は､赤鼻を掻きながら､ちよいと､頭を下げて､それから､わざとらしく､呆れたやうに､口を開いて見せた｡口髭には､今飲んだ酒が､滴になつて､くつついてゐる｡
       ―――――――――――――――――
その日の夜の事である｡五位は､利仁の館の一間に､切燈台の灯を眺めるともなく､眺めながら､寝つかれない長の夜をまぢまぢして､明してゐた｡すると､夕方､此処へ着くまでに､利仁や利仁の従者と､談笑しながら､越えて来た松山､小川､枯野､或は､草､木の葉､石､野火の煙のにほひ､――さう云ふものが､一つづつ､五位の心に､浮んで来た｡殊に､雀色時の靄の中を､やつと､この館へ辿りついて､長櫃に起してある､炭火の赤い焔を見た時の､ほつとした心もち､――それも､今かうして､寝てゐると､遠い昔にあつた事としか､思はれない｡五位は綿の四五寸もはいつた､黄いろい直垂の下に､楽々と､足をのばしながら､ぼんやり､われとわが寝姿を見廻した｡
 直垂の下に利仁が貸してくれた､練色の衣の綿厚なのを､二枚まで重ねて､着こんでゐる｡それだけでも､どうかすると､汗が出かねない程､暖かい｡そこへ､夕飯の時に一杯やつた､酒の酔が手伝つてゐる｡枕元の蔀一つ隔てた向うは､霜の冴えた広庭だが､それも､かう陶然としてゐれば､少しも苦にならない｡万事が､京都の自分の曹司にゐた時と比べれば､雲泥の相違である｡が､それにも係はらず､我五位の心には､何となく釣合のとれない不安があつた｡第一､時間のたつて行くのが､待遠い｡しかもそれと同時に､夜の明けると云ふ事が､――芋粥を食ふ時になると云ふ事が､さう早く､来てはならないやうな心もちがする｡さうして又､この矛盾した二つの感情が､互に剋し合ふ後には､境遇の急激な変化から来る､落着かない気分が､今日の天気のやうに､うすら寒く控へてゐる｡それが､皆､邪魔になつて､折角の暖かさも､容易に､眠りを誘ひさうもない｡
 すると､外の広庭で､誰か大きな声を出してゐるのが､耳にはいつた｡声がらでは､どうも､今日､途中まで迎へに出た､白髪の郎等が何か告れてゐるらしい｡その乾からびた声が､霜に響くせゐか､凛々として凩のやうに､一語づつ五位の骨に､応へるやうな気さへする｡
｢この辺の下人､承はれ｡殿の御意遊ばさるるには､明朝､卯時までに､切口三寸､長さ五尺の山の芋を､老若各､一筋づつ､持つて参る様にとある｡忘れまいぞ､卯時までにぢや｡｣
 それが､二三度､繰返されたかと思ふと､やがて､人のけはひが止んで､あたりは忽ち元のやうに､静な冬の夜になつた｡その静な中に､切燈台の油が鳴る｡赤い真綿のやうな火が､ゆらゆらする｡五位は欠伸を一つ､噛みつぶして､又､とりとめのない､思量に耽り出した｡――山の芋と云ふからには､勿論芋粥にする気で､持つて来させるのに相違ない｡さう思ふと､一時､外に注意を集中したおかげで忘れてゐた､さつきの不安が､何時の間にか､心に帰つて来る｡殊に､前よりも､一層強くなつたのは､あまり早く芋粥にありつきたくないと云ふ心もちで､それが意地悪く､思量の中心を離れない｡どうもかう容易に｢芋粥に飽かむ｣事が､事実となつて現れては､折角今まで､何年となく､辛抱して待つてゐたのが､如何にも､無駄な骨折のやうに､見えてしまふ｡出来る事なら､突然何か故障が起つて一旦､芋粥が飲めなくなつてから､又､その故障がなくなつて､今度は､やつとこれにありつけると云ふやうな､そんな手続きに､万事を運ばせたい｡――こんな考へが､｢こまつぶり｣のやうに､ぐるぐる一つ所を廻つてゐる中に､何時か､五位は､旅の疲れで､ぐつすり､熟睡してしまつた｡
 翌朝､眼がさめると､直に､昨夜の山の芋の一件が､気になるので､五位は､何よりも先に部屋の蔀をあげて見た｡すると､知らない中に､寝すごして､もう卯時をすぎてゐたのであらう｡広庭へ敷いた､四五枚の長筵の上には､丸太のやうな物が､凡そ､二三千本､斜につき出した､檜皮葺の軒先へつかへる程､山のやうに､積んである｡見るとそれが､悉く､切口三寸､長さ五尺の途方もなく大きい､山の芋であつた｡
 五位は､寝起きの眼をこすりながら､殆ど周章に近い驚愕に襲はれて､呆然と､周囲を見廻した｡広庭の所々には､新しく打つたらしい杭の上に五斛納釜を五つ六つ､かけ連ねて､白い布の襖を着た若い下司女が､何十人となく､そのまはりに動いてゐる｡火を焚きつけるもの､灰を掻くもの､或は､新しい白木の桶に､｢あまづらみせん｣を汲んで釜の中へ入れるもの､皆芋粥をつくる準備で､眼のまはる程忙しい｡釜の下から上る煙と､釜の中から湧く湯気とが､まだ消え残つてゐる明方の靄と一つになつて､広庭一面､はつきり物も見定められない程､灰色のものが罩めた中で､赤いのは､烈々と燃え上る釜の下の焔ばかり､眼に見るもの､耳に聞くもの悉く､戦場か火事場へでも行つたやうな騒ぎである｡五位は､今更のやうに､この巨大な山の芋が､この巨大な五斛納釜の中で､芋粥になる事を考へた｡さうして､自分が､その芋粥を食ふ為に京都から､わざわざ､越前の敦賀まで旅をして来た事を考へた｡考へれば考へる程､何一つ､情無くならないものはない｡我五位の同情すべき食慾は､実に､此時もう､一半を減却してしまつたのである｡
 それから､一時間の後､五位は利仁や舅の有仁と共に､朝飯の膳に向つた｡前にあるのは､銀の提の一斗ばかりはいるのに､なみなみと海の如くたたへた､恐るべき芋粥である｡五位はさつき､あの軒まで積上げた山の芋を､何十人かの若い男が､薄刃を器用に動かしながら､片端から削るやうに､勢よく切るのを見た｡それからそれを､あの下司女たちが､右往左往に馳せちがつて､一つのこらず､五斛納釜へすくつては入れ､すくつては入れするのを見た｡最後に､その山の芋が､一つも長筵の上に見えなくなつた時に､芋のにほひと､甘葛のにほひとを含んだ､幾道かの湯気の柱が､蓬々然として､釜の中から､晴れた朝の空へ､舞上つて行くのを見た｡これを､目のあたりに見た彼が､今､提に入れた芋粥に対した時､まだ､口をつけない中から､既に､満腹を感じたのは､恐らく､無理もない次第であらう｡――五位は､提を前にして､間の悪さうに､額の汗を拭いた｡
｢芋粥に飽かれた事が､ござらぬげな｡どうぞ､遠慮なく召上つて下され｡｣
 舅の有仁は､童児たちに云ひつけて､更に幾つかの銀の提を膳の上に並べさせた｡中にはどれも芋粥が､溢れんばかりにはいつてゐる｡五位は眼をつぶつて､唯でさへ赤い鼻を､一層赤くしながら､提に半分ばかりの芋粥を大きな土器にすくつて､いやいやながら飲み干した｡
｢父も､さう申すぢやて｡平に､遠慮は御無用ぢや｡｣
 利仁も側から､新な提をすすめて､意地悪く笑ひながらこんな事を云ふ｡弱つたのは五位である｡遠慮のない所を云へば､始めから芋粥は､一椀も吸ひたくない｡それを今､我慢して､やつと､提に半分だけ平げた｡これ以上､飲めば､喉を越さない中にもどしてしまふ､さうかと云つて､飲まなければ､利仁や有仁の厚意を無にするのも､同じである｡そこで､彼は又眼をつぶつて､残りの半分を三分の一程飲み干した｡もう後は一口も吸ひやうがない｡
｢何とも､忝うござつた｡もう十分頂戴致したて｡――いやはや､何とも忝うござつた｡｣
 五位は､しどろもどろになつて､かう云つた｡余程弱つたと見えて､口髭にも､鼻の先にも､冬とは思はれない程､汗が玉になつて､垂れてゐる｡
｢これは又､御少食ぢや｡客人は､遠慮をされると見えたぞ｡それそれその方ども､何を致して居る｡｣
 童児たちは､有仁の語につれて､新な提の中から､芋粥を､土器に汲まうとする｡五位は､両手を蠅でも逐ふやうに動かして､平に､辞退の意を示した｡
｢いや､もう､十分でござる｡……失礼ながら､十分でござる｡｣
 もし､此時､利仁が､突然､向うの家の軒を指して､｢あれを御覧じろ｣と云はなかつたなら､有仁は猶､五位に､芋粥をすすめて､止まなかつたかも知れない｡が､幸ひにして､利仁の声は､一同の注意を､その軒の方へ持つて行つた｡檜皮葺の軒には､丁度､朝日がさしてゐる｡さうして､そのまばゆい光に､光沢のいい毛皮を洗はせながら､一疋の獣が､おとなしく､坐つてゐる｡見るとそれは一昨日､利仁が枯野の路で手捕りにした､あの阪本の野狐であつた｡
｢狐も､芋粥が欲しさに､見参したさうな｡男ども､しやつにも､物を食はせてつかはせ｡｣
 利仁の命令は､言下に行はれた｡軒からとび下りた狐は､直に広庭で芋粥の馳走に､与つたのである｡
 五位は､芋粥を飲んでゐる狐を眺めながら､此処へ来ない前の彼自身を､なつかしく､心の中でふり返つた｡それは､多くの侍たちに愚弄されてゐる彼である｡京童にさへ｢何ぢや｡この鼻赤めが｣と､罵られてゐる彼である｡色のさめた水干に､指貫をつけて､飼主のない尨犬のやうに､朱雀大路をうろついて歩く､憐む可き､孤独な彼である｡しかし､同時に又､芋粥に飽きたいと云ふ慾望を､唯一人大事に守つてゐた､幸福な彼である｡――彼は､この上芋粥を飲まずにすむと云ふ安心と共に､満面の汗が次第に､鼻の先から､乾いてゆくのを感じた｡晴れてはゐても､敦賀の朝は､身にしみるやうに､風が寒い｡五位は慌てて､鼻をおさへると同時に銀の提に向つて大きな嚔をした｡"""
    },
{
"@search.action": "upload",
"BookId": "462",
"BookName": "さぶ",
"BookAuthor": "山本周五郎",
"BookGenre": "長編小説",
"Release_Date": "1963年1月",
"Rating": "3.00",
"Text": """小雨が靄のようにけぶる夕方､両国橋を西から東へ､さぶが泣きながら渡っていた｡
双子縞の着物に､小倉の細い角帯､色の褪せた黒の前掛をしめ､頭から濡れていた｡雨と涙とでぐしょぐしょになった顔を､ときどき手の甲でこするため､眼のまわりや頬が黒く斑になっている｡ずんぐりした躯つきに､顔もまるく､頭が尖っていた｡――彼が橋を渡りきったとき､うしろから栄二が追って来た｡こっちは痩せたすばしっこそうな躯つきで､おもながな顔の濃い眉と､小さなひき緊った唇が､いかにも賢そうな､そしてきかぬ気の強い性質をあらわしているようにみえた｡
栄二は追いつくとともに､さぶの前へ立ち塞がった｡さぶは俯向いたまま､栄二をよけて通りぬけようとし､栄二はさぶの肩をつかんだ｡
｢よせったら､さぶ｣と栄二が云った､｢いいから帰ろう｣
さぶは手の甲で眼を拭き､咽びあげた｡
｢帰るんだ｣と栄二が云った､｢聞えねえのか｣
｢いやだ､おら葛西へ帰る｣とさぶが云った､｢おかみさんに出ていけって云われたんだ､もう三度めなんだ｣
｢あるきな｣と云って栄二は左のほうへ顎をしゃくった､｢人が見るから｣
二人の少年は橋のたもとを左へ曲った｡雨は同じような調子で､殆んど音もなくけぶっていた｡
｢おらほんとに知らなかったんだ｣とさぶが云った､｢ゆうべ粉袋を戸納へしまってたときに､勝手で使うから一つ出しておけって､おかみさんに云われた､だから一つだけ残しといたんだ､そしたらその袋が出しっ放しになってて､おかみさんは使ったあとでしまっとけって､その袋を返したのに､おれがしまい忘れたっていうんだ｣
｢癖だよ､癖じゃねえか｣
｢粉が湿気をくっちゃった､へまばかりする小僧だって｣さぶは立停って､手の甲で眼のまわりをこすりながら泣いた､｢――おら､返してもらわなかった､そんな覚えはほんとにねえんだ､ほんとに知らなかったんだ｣
｢癖だってば､おかみさんはなんとも思っちゃあいねえよ｣
｢だめだ､おら､だめだ､ほんとにとんまで､ぐずで､――自分でも知ってた､とても続けられやしねえ､もうたくさんだ｣さぶは喉を詰らせた､｢おら､思うんだが､いっそ葛西へ帰って､百姓をするほうがましだって｣
広い河岸通りの､右が武家屋敷､左が大川で､もう少しゆくと横網になる｡折助とも人足ともわからない中年の､ふうていのよくない男が二人､穴のある傘をさして､なにかくち早に話しながら､通りすぎていった｡その男たちの､半纏の下から出ている裸の脛が､栄二にはひどく寒そうにみえた｡さぶはあるきだしながら､小舟町の｢芳古堂｣へ奉公に来てから三年間の､休む暇もなくあびせられた小言と嘲笑と平手打ちのことを語った｡それは訴えの強さではなく､赤児のなが泣きのような､弱よわしく平板なひびきを持っていた｡大川の水がときたま､思いだしたように石垣を叩き､低い呟きの音をたてた｡
｢奉公が辛いのはどこだっておんなしこった､おかみさんの口の悪いのは癖だし｣と栄二はつかえつかえ云った､｢それにおめえ､女なんてもともと､――車だ｣
栄二がさぶの腕に触り､二人は立停って川のほうへよけた｡からの荷車を曳いた男がうしろから来て､二人を追いぬいていった｡
｢腕に職を付けるのは辛えさ｣と栄二は続けた､｢考えてみな､葛西へ帰ったって､朝から晩まで笑ってくらせやしねえだろう､それとも百姓はごしょう楽か｣
｢葛西のうちなら｣とさぶが云った､｢出ていけなんて云われることだけはありゃあしねえ｣
｢ほんとにそうか｣
さぶは返辞をしなかった｡栄二も返辞を期待していなかった｡さぶは葛西にある実家のことを考えてみた｡腰の曲った喘息持ちの祖父､気の弱い父と､男まさりで手の早い母､朝から母と喧嘩の絶えない口やかましい兄嫁､三人いる弟妹と､呑んだくれの兄と､五人もいる甥や姪たち｡うす暗く煤だらけな､古くて狭くて､ぜんたいが片方へ傾いている家や､五反歩そこそこの痩せた田畑など｡さぶは途方にくれ､しゃくりあげながら､またあるきだした｡
｢おめえにゃあ田舎がある｣いっしょにあるきながら栄二が云った､｢どんなうちにしろ帰るところがあるからいい､だがおらあ親きょうだいも身寄りもねえ独りぼっちだ､今年の春､おらあ店を追ん出されるようなことをしちまった､追ん出されるか自分でおん出るか､どっちか一つという､とんでもねえことをしちまったんだ｣
さぶはそろそろと振り向いて､栄二の顔を見た｡好奇心からではなく､戸惑ったような眼つきであった｡栄二はふきげんな､怒ってでもいるような口ぶりで､自分が去年から幾たびか帳場の銭をぬすみ､それを主婦のお由にみつかったのだ､と告白した｡
｢わこく橋の側の堀っぷちに鰻の蒲焼の屋台が出る｣と栄二は続けた､｢おらあ蒲焼の匂いを嗅ぐとがまんができなくなるんだ｣
通りがかりにその匂いを嗅ぐと､喰べるまでは胃の腑がおさまらない｡気持もおちつかず､することが手につかない｡まるで病気のようになってしまい､ときには手足がふるえだすことさえあった｡帳場の銭箱から銭をつかみ出したのはそういうときで､去年の秋から十二､三たび盗みだしたろうか､食いたい一心で悪いことをしたとは思わなかった｡それがこの二月､主婦のお由にその部屋へ呼ばれた｡
｢おかみさんは小言は云わなかった｣と栄二は泥でも噛むように､顔をしかめながら云った､｢――去年の八月五日と､昨日､おまえが帳場でやったことをあたしは見たよ､もうあんなことはおよし､欲しかったらあたしがあげるから､あたしのところへそう云っておいで､って､――それっきりだった｣
お由は二度だけしか見なかったのだろうか､それともすっかり知っていて､わざと知らないふうをよそおったのか､いずれにもせよ､栄二は死ぬほど恥じ､もう店にはいられないと思った｡自分をぬすっとだなどとは考えもしなかったが､銭箱から銭をつかみだした自分の姿が､あさましくて恥ずかしくて､そのまま店にいる気になれなかったのだ｡
｢だが､店をとびだしてどこへゆく｣と栄二は続けた､｢おらあ八つの年､大鋸[#ﾙﾋﾞの｢おおのこ｣はﾏﾏ]町で夏火事にあい､両親と妹に焼け死なれた､おれ一人は白魚河岸へ釣りにいっていて助かったが､ほかに身寄りは一軒もなかった､おやじは伊勢から出て来たと云ってたが､伊勢のどこだかおらあ覚えちゃあいねえし､覚えていたって頼ってゆけるもんじゃあねえ､おらあそのときくれえ自分にうちのねえことが悲しかったこたあなかった｣
｢知らなかった､おら､ちっとも知らなかった｣とさぶが呟いた､｢――それで栄ちゃんは､がまんしたんだね｣
｢銭も二度とはぬすまなかった｣
二人は横網の河岸まで来てい､さぶが立停って､地面をみつめ､濡れて重くなった草履の先で､地面を左右にこすった｡
｢おら､思うんだが｣と彼は心のきまらない口ぶりで云った､｢――小さいじぶんおふくろにぶたれたことがある､弟のやつがいたずらをして､それをおれがしたもんだと思ってぶった､おら､泣きながらおれのしたことじゃあねえって云って､それから､弟のしたことだとわかったとき､おふくろは平気な顔で云った､それじゃあおまえはこれまでに､ぶたれるようなことは一度もしなかったっていうのかい､ってさ｣
｢女なんてそんなもんだ｣と栄二が云った､｢撫でた手でつねるし､つねった手で撫でるようなことをする､そしてどっちもすぐに忘れちまうんだ､――少しはおちついたか､さぶ､もうここいらで帰ってもいいだろう｣
さぶは不決断にううと云った｡
｢ありがと｣とさぶはよく聞きとれない声で云った､｢ごめんよ､栄ちゃん｣
｢こんどは黙ってとび出さねえでくれよ｣と栄二は云った､｢これからはなんでもおれに相談してくれ､力になるからな｣
さぶはゆっくりと頷いた｡
二人は引返した｡そして両国橋まで戻ると､うしろから十二､三になる少女が追って来て､せいせいと喘ぎながら声をかけた｡
｢この傘をさしなさいよ｣と少女はから傘を二人のほうへ差出した､｢姉さんのとこへ持ってゆくんだから､方角ちがいじゃだめだけれど､さあ､さしなさいよ｣
栄二は少女を見た｡自分でさしている傘は穴があいていた｡着ているのも継ぎのあたった青梅縞の古袷で､帯はよれよれだし､はいている下駄はちびたおとな物で､それもすっかり鼻緒がのびているから､泥のはねた足指をまむしにしていた｡
｢いらねえよ｣と栄二が云った､｢おれたちは小舟町へ帰るんだ､さっさといっちまいな｣
｢あら､ちょうどいいじゃない｣と少女はうれしそうに笑った､｢あたしは堀江町よ､堀江町のすみよしっていうお店に姉さんが勤めているの､だからあたし､あんたたちのうちまで送っていってもいいのよ｣
｢うるせえな｣と栄二が云った､｢傘なんかいらねえって云ってるじゃねえか｣
｢だって二人ともびしょ濡れじゃないの､ねえ､これをさしてらっしゃいよ｣
｢さぶ｣と栄二が云った､｢駆けようぜ｣
二人は小雨の中を走りだした｡
｢ばかねえ｣と少女がどなった､｢いいから二人とも濡れてらっしゃい､いくじなし｣
栄二とさぶとは､そのとき同じ十五歳であった｡少女のことは二人ともすぐに忘れてしまった｡
一の二
二人が二十歳になった年の二月十五日｡生れて初めて､いっしょに外へ酒を飲みに出た｡酒が初めてなのではない､それまでにも祝い日などに､店で酒の出るときは盃に二つか三つは舐めたことがあった｡けれども外へ出て､手銭で飲むようなことはなかった｡半分は恐ろしくもあったが､主人の芳兵衛に禁じられていたからである｡躯のかたまらないうちに酒を入れると骨がやわになる､はたちになるまでは飲むな､というのが口癖であった｡
芳古堂は表具と経師とで､格も高く､手堅いので知られていた｡先代からのきまったとくい先と､当代知名な五､六人の書家や絵師｡老舗の骨董屋とか武家､大店などに限り､安い仕事はいっさい断わるというふうであった｡したがって､八人いる職人たちの躾もきびしく､みな子飼いから育てられ､読み書きはもちろん､生け花､茶の湯までひととおり習わされ､書画のよしあしなども､小さいころから実物について教えられた｡――いま店にいるのは八人､職人がしらが和助といって二十九､次が多市で二十七､それから重七､五郎､栄二とさぶが二十で､下に十七歳の伝六と十五歳の半次がいた｡そのほかに店から出て､かよいで来たり､独立して店を持っている者が十三人おり､芳古堂の仕事がたて混んだり､特別な注文があったりすると､その十三人の中から適当な者が手伝いに呼ばれた｡
店のしきたりがそういうふうなので､職人たちの日常も規則ただしく､毎月十五日と一日のほかは､夜遊びに出ることも禁じられていたし､二十歳からは夜食に一本の酒が付くけれども､それ以上は一滴も飲ませなかった｡念には及ばないだろうが､こういう生活をきちんと守る者ばかりはいない｡仕事は夕方の五時限り､どんなに仕事が溜っていても､五時になればやめてしまい､あと片づけをして銭湯にゆき､夕食をしたあと､九時に寝るのがきまりになっていた｡寝るまでの時間は､本を読むとか習字をするとか､碁や将棋をさすとか､自分たちの好きなことをしていいのだが､中には店を抜け出して､酒を飲んだり女遊びをしたりする者がなくはなかった｡――そういうことも主人の芳兵衛は知っていて､たいていの場合はなにも云わないが､ぬけ遊びがたび重なると､しぜん仕事にひびくので､そのとき初めて小言を云い､それでも行状が改まらないと暇を出してしまう｡五年のうちに二人くらいはそういう職人がいて､これらは芳古堂にいた､と云うことも許されないのであった｡
栄二とさぶは心がときめいていた｡
｢はたちになったなんて､へんな心持だな｣とさぶがまだるっこい調子で云った､｢おら､思うんだが､十六で月代を剃ったときよりもへんな心持だ｣
｢そうだな｣と栄二が云った｡
二人は千筋の手織り木綿の袷に双子縞の羽折､小倉の角帯をしめ､麻裏草履をはいていた｡ちょうど黄昏どきで､人の往来の多い小舟町の通りを東のほうへ､かくべつ目的もなくあるいていた｡とにかく両国広小路へでもいってみようか､と思っているようであった｡
｢おめえはいいな､栄ちゃん｣とさぶが云った､｢おめえはもう屏風にかかれる､襖の下張りならいちにんめえだ､ところがおらときたら､いまだに糊の仕込みだ｣
｢それも仕事だぜ｣
｢おら､思うんだが､水の中で袋を揉みながら､ときどき自分がやりきれなくなるよ､はたちにもなってこのざまかって｣
｢それも仕事だよ､さぶ｣と栄二が云った､｢表具や経師は糊の出来のよしあしが仕事の仕上りをきめるんだぜ､おめえわかっていねえのか｣
｢そりゃあそうだが｣
｢わかってたらぐちを云うなよ｣と栄二は云った､｢糊の仕込みで日本一になれば､それはそれで立派な職人なんだ､おめえ日本一の糊作りになれよ｣
｢そりゃあそうなんだが｣
しかし芳古堂の職人となれば､表具とか屏風､屋敷襖なども覚えたい｡そう云いたかったのだが､さぶは口には出せなかった｡
｢ちょっと｣と栄二が云って立停った｡
堀江町と新材木町のあいだに堀がある｡その堀端に五､六軒､小料理屋がとびとびにあって､その端の一軒で｢すみよし｣と紺地に白く､仮名で染め抜いた半のれんを､軒先に掛けている女がいた｡小柄な躯もほっそりしているし､襷をかけてあらわになった二の腕も､裾を端折った黄八丈の着物の下に覗いている白い脛も､ほっそりと柔軟そうにみえた｡
｢なんだい､栄ちゃん｣
｢すみよし｣と栄二は口の中で呟いた､｢聞いたことがあるようだな｣
｢柳橋の料理屋だよ､すみよし､とくい先じゃあねえか｣
｢そうじゃあねえ､柳橋じゃあねえ､どこかよそで聞いたことがあるんだ｣
のれんを掛け終った女は､足許の盛り塩をよけて､家の中へはいった｡栄二は記憶を呼びおこそうとするように､眼を細めて暫く考えていたが､どうしても思いだすことができないようすで､そっと舌打ちをすると､まあいいや､はいってみようと云い､さぶを促してそっちへあるいていった｡
店へはいると､四十がらみの男が､灯を入れたはちけんを天床へ吊りあげているところだった｡三間に五間くらいの土間に､飯台が二た側､おのおの左右に作り付けの腰掛が据えられ､蒲で編んだ円座が二尺ほどの間隔をとって置いてある｡客が多くてもぎっしり詰めず､ゆとりをおいて飲めるように按配してあるらしい｡右手に竹で格子を組んだ板場､つきあたりにまたのれんが掛けてあり､これは浅黄に紺で｢すみよし｣と書いてあった｡
｢早かったかな｣店へはいった栄二は､はちけんを吊りあげている男にきいた､｢まだ始めないのかい｣
｢いらっしゃい｣と男はいさましく答えた､｢どうぞ｣
そして奥のほうへ向って､お客だよと､大きな声でどなった｡栄二はさぶの肩を押し､飯台の一つを選んで､その奥の端のほうへ腰を掛けた｡すぐに女が二人､自分の髪へ手を触れながら出て来て､あいそを云いながら注文をきいた｡さっきのれんを掛けていた女ではなく､一人は十八､九､一人は二十二､三で､どちらも小太りで､白粉と香油をつよく匂わせていた｡栄二は酢の物とうま煮で酒を二本と云い､云いながら赤くなった｡
｢あたしあんたのこと知ってるわ｣と年嵩のほうの女がさぶに云った､｢あんた小舟町の芳古堂のしとでしょ｣
さぶは戸惑ったように栄二を見た｡女の一人は注文をとおしにゆき､年嵩のその女は腰をかけた｡
｢そうじゃねえさ｣とさぶが云い､慌てて云い直した､｢いや､本当はそうだよ､今日は親方やおかみさんに許されて来たんだ､こっちは栄ちゃん､おらあさぶっていうんだが､二人とも今年でちょうど｣
｢よせよ｣と栄二が云った､｢よけいなことを饒舌るなよ｣
｢あら､いいじゃないの｣と女が云った､｢さぶちゃんに栄ちゃんね､あたしおかめ――仇名じゃなくって本名なのよ､どうぞよろしく｣
さぶが笑いだし､栄二が睨んだ｡
｢おれたちは二人っきりで飲みてえんだ｣と栄二が女に云った､｢酌はいいから､二人っきりにしてくんねえか｣
｢そんなら奥になさいな｣と女は気に障ったようすもなく云った､｢もうすぐにここはたて混むから､ゆっくり話なんかできやしないわ､狭いけれども向うなら静かよ､そうなすったらどう｣
｢うん｣と云って栄二がふところへ手を入れた､｢おれたち､あんまり持ってねえんだ｣
女は笑いながら､そんな心配はいらないと云い､二人を立たせて､奥へ案内した｡のれんをくぐったところに､四帖半くらいの小座敷が三部屋並んでいる｡右側は隣りの塀で､塀隠しに竹が植えてあるが､まばらな葉がみな茶色にちぢれて根じめのつもりだろう､ところどころに据えてある苔付きの石の､苔もまたかさかさに乾いていた｡
｢ここがいいわ｣女は二人を端の四帖半にとおして云った､｢いま行燈を持って来るわね｣
小座敷ながら半間の床に掛物があり､隣りとの襖を隠すように､二枚折りの小屏風を立て､四角な桐の火鉢には火がおこっていた｡自分で本名だというのだから慥かだろう､まもなくおかめが火のはいった行燈を持って来､続いてもう一人が蝶足の膳を二つ持って来た｡
｢こんなことして､大丈夫かな｣さぶが心ぼそそうに囁いた､｢もしも勘定がたりなかったらどうする｣
｢うるせえな｣栄二は胸のときめきを隠しながら云った､｢向うは店の名を知ってるんだ､そうでなくったって､まさか首を取ろうとは云やあしねえだろう､そうわさわさするなよ｣
やがておかめが､注文の酒肴をはこんで来､二人の膳へ分けて置くと､では用があったら手を鳴らしてくれと云って去った｡
｢めえめえでやろうぜ｣と栄二が云った､｢めんどくせえから盃のやりとりはなし､酒も手酌でやるとしよう､いいな｣
｢いいけれども｣とさぶは膳をじっと見まもりながら云った､｢なんだか少し､おっかねえような心持だ｣
｢なにがおっかないの｣と云って､三人めの女が障子をあけ､顔だけ出してにこっと笑いかけた､｢あらいやだ､河岸の親方かと思っちゃったわ､ごめんなさい｣
表でのれんを掛けていた女であった｡きりっと緊ったほそおもてで､笑ったとき唇のあいだから八重歯が覗いた｡栄二はふいと､冷淡にそっぽを向いた｡
一の三
｢陰気じゃない｣とその女は云った､｢誰もいないの｣
｢いいんだ｣と栄二がそっぽを向いたままで云った､｢酌はいらないんだ｣
｢お通夜みたい｣と女が云った､｢それとも人に聞かせられないような悪い相談でもするの｣
栄二が振り向いて｢うるせえな｣と云った｡女はちょっと笑いかけたが､栄二の眼を見ると顔をひきしめ､ごめんなさい､と低い声で云って障子を閉めた｡女が笑ったとき､また八重歯が覗き､それが栄二の眼に残った｡
三月一日には栄二は休めなかった｡日本橋本町の綿文という大きな両替商で､客座敷の襖を張り替えるのに､紙合せと下見のため､兄弟子の多市といっしょにでかけたのである｡綿文も芳古堂にとっては古いとくいで､一年に一度は襖の張替えをする｡栄二は十三の年から毎年､多市や重七たちの下廻りでゆき､家族や召使たちとも顔馴染になっていた｡――主人の徳兵衛は肥えていて躯が大きく､いつでも酒臭い息をして､店へは殆んど出ず骨董いじりや俳諧に凝っていた｡妻女はおみよといって､躯も細くて小柄だし､顔もちまちまとして､大店の主婦というより､横丁あたりのおかみさんのような感じであった｡男の子はなく､おきみ､おそのという娘が二人､としは二つ違いで､どちらも評判のきりょうよしだが､姉は父親に似てゆったりした躯つきだし､性質ものんびりしていた｡妹は痩せがたで顔も細く､ませた口をきくし､することがすばしこかった｡
綿文は角店で､土蔵が二戸前｡二階造りの店とはべつに､中庭を隔てて､平屋建ての住居があった｡住居のほうは横に門があり､正面が玄関｡右へ火除け用の厚い土塀に沿って廻ると､向うに屋根を掛けた釣瓶井戸があり､その手前の左側に勝手口があった｡炊事場ではなく､家族や内客や､諸商人､職人などの出入りするところで､客の多い家だから下足番を兼ねた小僧が一人､あがり端の六帖で小粒金や小判のはいった麻袋を､板の上で叩いていた｡袋をあげては板へ落す､という単調でまのぬけた動作であるが､そうやっていると､微量の金が麻袋に付くので､一定の期間をおいて袋を焼けば､溜った金屑がとれるのだという｡役人にみつかっては悪いから店ではやらないのだろうが､両替商ならどこでもすることだそうで､栄二はそれを聞いたとき､こんな大店のくせにけちなことをするものだと､ひどく軽蔑したものであった｡
見本の紙の包みを持って､多市と二人､座敷へとおされると､十五､六になる中働きのおすえが茶と菓子を持って来た｡――栄二は一昨年と去年はこの家へ来なかったが､三年まえまでは毎年来ていて､二人の娘たちとも親しかったし､おすえともよく知っていた｡
｢暫くね､栄さん｣とおすえは多市に挨拶してから栄二を見た､｢ずいぶん大きくなったじゃありませんか､あたし初めはちょっと見違えちゃったわ｣
｢よせよ｣と云って多市が笑った､｢可哀そうにこれでもはたちになったんだぜ｣
｢ごめんなさい｣おすえは赤くなった､｢あたし立派になったっていうつもりだったのに､つい口がすべっちゃったんです｣
栄二も赤くなったが､おすえのほうへは眼も向けなかった｡
｢おすえちゃんは幾つになった｣
｢十六です｣とおすえは多市に答えた､｢躯が小さいから十二､三にしきゃみえないって､よくみんなにからかわれるんですよ､恥ずかしい｣
廊下に人が立ち､こっちを覗きこんだ｡この家の姉娘であるが､そこへまた一人､妹娘が通りかかって､姉のうしろから首だけ出して覗き､あら栄ちゃんだわと云った｡姉はそこを動かなかったが､妹のおそのは座敷へとびこんで来､栄二の前にばたんと坐って､大きな眼でじっと彼をみつめた｡おすえは会釈をして出てゆき､栄二が眼の隅でそのうしろ姿をちらっと見た｡
｢まあおどろいた､あんた栄ちゃんじゃないの｣とおそのが顔を輝かせながら云った､｢大きくなったわね､びっくりしちゃったわ｣
多市が唇だけで笑った､｢いまそれを云われたばかりなんですよ｣
｢栄ちゃん｣とおそのは多市には構わず､栄二の眼をみつめたままで云った､｢あたし誰だかわかって｣
｢おそのさんですよ｣と栄二は答えた､｢何年も会わなかったわけじゃない､たった二年ここへ来なかっただけじゃありませんか｣
｢あたしも大きくなったでしょ｣
｢こんちは｣と栄二は廊下にいるおきみに呼びかけた､｢暫くでした｣
おきみはおっとりと頷いて､いらっしゃいと､ゆっくり云った｡
一の四
そこへ主人の徳兵衛がはいって来､多市が紙見本をひろげた｡徳兵衛は相変らず酒臭い息をしていた｡
｢ちょっと来てよ､栄ちゃん｣とおそのが云った､｢あんたに見せたいものがあるのよ｣
｢そのちゃん｣と廊下でおきみが云った｡
｢ねえ､お父っさんいいでしょ｣とおそのが父親に鼻声で云った､｢栄ちゃんにちょっと見せたいものがあるのよ､あっちへ来てもらってもいいわね｣
姉がまた｢そのちゃんったら｣とたしなめ､徳兵衛は無関心に手を振って､｢うるさい､好きなようにしろ｣と云った｡栄二は救いを求めるように多市を見たが､多市は笑いもせずに､顎をしゃくって､ゆけよ､という表情をした｡
｢さあ､栄ちゃん｣とおそのは彼の手を取った､｢ねえ､早く｣
あのころのままだな､と栄二は思った｡兄弟子たちとここへ仕事に来ると､必ずこの姉妹につかまって遊び相手をさせられた｡襖を張り替えるのは毎年十二月だから､かるたとか追い羽根とか､お手玉､おはじきなど､たあいのないものだし､女の子の遊び相手をするなど屈辱さえ感じたものだ｡けれども大事なとくい先のことではあり､兄弟子たちがそうしろと云うので､断わるわけにはいかず､しぶしぶつきあっているうちに､どの遊びでも彼がいちばん上手になり､勝気なおそのはくやしがってよく泣いたものであった｡
伴れてゆかれたのは姉妹の部屋で､箪笥がそれぞれ二棹ずつに､人形などを置いた飾り棚､琴､三味線､茶道具を入れる茶箪笥､朱塗の衣桁など､みな娘の居間らしい華やかな色と､香料の匂いがあふれていた｡
｢あたし十六になったでしょ｣とおそのが自分の箪笥の前に膝をつきながら云った､｢――それでね､また友禅の振袖を作ってもらったのよ､きれいよう｣
そして抽出の一つをあけ､中からその品を取り出して､さも大切そうに両手で持って栄二に渡した｡
｢ひろげてみてよ｣とおそのが云った､｢四季の千草って云う柄なの､京の田丸屋で染めさしたのよ｣
｢あたしのは裾模様よ｣と姉のおきみが二人の脇から云った､｢あたしのも見せてあげるわ｣
｢あとで｣とおそのがきめつけた､｢姉さんてばいつでもあたしのまねばかりするのね､邪魔しないでよ｣
栄二は着物をひろげてみて､きれいだなと云った｡これだけ大きな資産家の娘なら､京染めの友禅ぐらいなんでもないだろうのに､わざわざ人を呼んで見せるところに､この姉妹の気取らない､下町っ子らしい開放的な性質がよくあらわれていた｡
妹に決めつけられた姉のおきみは､きげんを悪くするでもなく､そろそろと自分の箪笥をあけた｡その裾模様というのを出すつもりであろう､けれどもおそのはそれより早く､こんどは帯を見せてあげる､と云いながら､下の抽出をあけたとたん､悲鳴をあげてとびあがり､両手で栄二に抱きついた｡
｢こわい｣とおそのは栄二にしがみつきながら叫んだ､｢鼠よ､鼠がいるのよ｣
おきみも吃驚してうしろへさがり､栄二はおそのの腕を放そうとした｡だがしがみついたおそのの力はおどろくほど強く､すぐにはふり放すことができなかった｡
｢放さなくっちゃだめだ｣と栄二が云った､｢それじゃあ鼠が追えやしないよ｣
｢いや､こわい｣とおそのはもっと腕に力をいれた､｢あたし息が止りそうよ｣
｢鼠を追っぱらうんだから｣栄二はようやく躯を自由にし､おそのを押しやった､｢さあおきみさんもどいてください｣
そして､その抽出の中を覗いてみたが､鼠のいるようすはなかった｡手を入れて､重ねてある帯の片方をつぎつぎとあげ､底までさぐってみたけれども､鼠どころか虫一疋もみあたらなかった｡栄二は抽出の中を元のようにして立ちあがり､おそのの顔をにらんだ｡おきみは両手で胸を抱え､怯えたような表情で栄二を見あげていた｡
｢ほんとよ､嘘つかない｣とおそのは栄二の眼を眩しそうに避けて云った､｢帯を取ろうとしたらすぐそこにしゃがんでいて､あたしのこと食いつこうとしたのよ｣
栄二がなにか云おうとしたとき､廊下から名を呼ばれ､振り返るとおすえがいた｡
｢多市さんが呼んでいます｣とおすえはこっちを見ずに云った､｢尺取りをするから来て下さいって｣
栄二はそれに頷いてから､おそのに向って帯のぎっしり詰っている抽出の中を指さしてみせた｡そこにはどんな小さな鼠でもしゃがんでいる余地などはない､ということを示したのであろう｡おそのはひょいと肩をすくめて云った｡
｢でもいたのよ､ほんと､こんなふうにしゃがんで､あたしのこと食いつこうと思って､こんなように歯を剥きだしていたのよ｣
おそのはそんなような恰好をしてみせたが､栄二はなにも云わずに出ていった｡
一の五
用が済んだので､栄二は先に勝手口から出た｡見本紙や尺取りの帳面を入れた包みを持って､格子戸を出るとおすえの姿が見えた｡井戸のところに立っていて､待っていたようにこっちへ走って来､栄二の眼をみつめながら頬笑んだ｡みつめる眼にはいっしょうけんめいな､思い詰めたような光があり､その頬笑みはまるで泣きべそでもかくように歪んでいた｡
｢さっきのこと､堪忍して下さいね｣
｢なにをさ｣と栄二がきき返した｡
｢大きくなったって云ったこと｣とおすえは眼をそらさずに云った､｢あたしほんとに､立派になったって云いたかったんです｣
｢いいよ､そんなこと｣栄二は包みを持ち直しながら云った､｢怒ってなんかいやしねえよ｣
｢ほんとね｣と囁きかけながら､おすえは涙をこぼした､｢よかったわ｣
｢なんだ､あんなこと､つまらねえ｣
｢あたし初めて栄さんに会ったとき十三だったけれど､栄さんのこと怒りっぽいこわい人だなって､思ったのを覚えてるわ｣
栄二はなにか云いそうにして赤くなり､それから怒ったような口ぶりで云った､｢――おれだっておめえのこと覚えてたよ｣
おすえは｢ありがと｣と囁き声で云い､くるっと向き直って小走りに去った｡栄二はそっちを見なかった｡顔は赤くなったままで､深い呼吸をするために､胸が見えるほど波打っていた｡
｢栄二｣と呼ぶ声がした､｢ちょっと｣
格子戸の中から多市が覗いていた｡栄二は悪いことでもみつけられたように､どぎまぎしながらそっちへいった｡
｢先に帰ってくれ｣と多市は云った､｢旦那の酒のお相手だ､うんざりだがしようがねえ､親方にそう云っといてくれ｣
上り端の六帖では､小僧がまだ麻袋を重たそうに板へ打ちつけていた｡栄二は多市に頷いて､そこから去った｡
｢大きくなった､か｣道をあるきながら､栄二はそっと呟いて微笑した､｢てめえだって大きくなったくせに､――躯つきも顔もあのころとちっとも変っちゃいねえ､十三のときもいまのまんまだった｣
女は十三になるともう､顔も躯も娘になってしまうのかな､おかしなもんだなと思って､栄二はまた微笑した｡
小舟町へ帰ると､休みだから店は閉めてあり､栄二は横の木戸からはいった｡すると､裏の狭い空地でさぶが糊の仕込みをしていた｡裾を端折り､襷を掛け､五升樽くらいの桶に向って､小さな腰掛にかけたまま､桶の中へ両手を入れて揉み出しをしていた｡小麦粉をよく水で練りあげ､袋に入れ揉むと白い水が出る､それを沈澱させたうえ､壺に入れて日蔭の土に壺の半分を埋めて貯える｡表具とか屏風の裏打ち用には､そうして作った糊しか使わないし､壺の中でねかせる期間も二年から三年はかかった｡
｢さぶ､どうしたんだ｣栄二は近よってゆきながら呼びかけた､｢休みじゃあねえか､なにをしてるんだ､おまけにこんな裏なんかへ出たりしてさ､え｣
さぶは答えもせず､振り向きもしなかった｡栄二は彼の横顔が濡れているのを認めた｡
｢どうしたんだ｣栄二は声を低めた､｢なにかあったのか｣
｢なんでもねえ｣さぶは首を振った､｢なんでもねえんだよ｣
｢泣いてるじゃねえか｣
｢泣いてやしねえ｣と云って､さぶは腕で眼のまわりをこすった､｢捏ねをやってるときに､粉が眼にへえったんだ｣
栄二はなお､さぶの横顔をみつめていたが､さぶは振り向こうとはしなかった｡
｢二人でどこかへいこうと思って､いそいで帰って来たんだが｣と栄二が云った､｢それを始めちゃったとするとだめだな｣
揉み出しを始めれば､壺へ仕込むまで手は放せない｡栄二はさぶと飲みにでも出て､おすえのことが話したかった｡なにを話したいかは自分でもわからない..."""
},

    
    {
    "@search.action": "upload",
    "BookId": "57625",
    "BookName": "五瓣の椿",
    "BookAuthor": "山本周五郎",
    "BookGenre": "長編小説",
    "Release_Date": "1959年1月",
    "Rating": "3.00",
    "Text": """天保五年正月二日に､本所の亀戸天神に近い白河端というところで､中村仏庵という奇人が病死した｡年は八十四歳であった｡彼は大工と畳職の棟梁であるが､書をよくし､雲介舎弥太夫と号していた｡それは､箱根へ湯治にいったとき､駕籠舁から息杖を買って帰り､その杖に諸家から題詩を貰って彫りつけ柱に掛けて自慢していた｡それで雲介舎などとなのったらしいが､そのまえ､本所の小梅に住んでいたとき､役者の岩井紫若がその土地を買った｡紫若はそこへ家を建てるので､追立てられた彼ははらだちまぎれに､その住み古した家の壁へ左のような狂歌を書いて立退いた｡
雲介が住みあらしたる家なれば
河原乞食や跡にきぬらん
 奇人ではあったけれども､あまり世間からは好かれていなかったようだ｡この仏庵の住居から一丁ほど南に寄って､やはり白河端に｢むさし屋喜兵衛｣の寮があった｡むさし屋は日本橋本石町三丁目の薬種屋で､隣りに油屋も兼業してい､老舗としても資産家としても､市中にひろくその名を知られていた｡
 仏庵が死んでから四日めに当る､同じ正月の六日の夜半､その｢むさし屋｣の寮が自火で焼け､焼け跡から三人の死躰が出た｡兼業の油屋の品が置いてあったものか､建物の中は油が火を引いたもようで､柱まできれいに焼け落ち､三人の死躰も殆んど男女の区別がつかなかった｡――寮にはおまさという中年の下女と､五助という下男がいた｡五助はかよいで､夕方には自宅へ帰るが､おまさはもちろん住込みであった｡しかしおまさも､その日は本所業平にある弟の家へ帰ってい､明くる朝､寮へ戻って来て初めて､その出来事を知った｡
 町方の訊問に対して､おまさは次のように答えた｡
｢この寮にはいつもおかみさんがいました､それが十二月の二十九日に､ええ､仏庵さんの亡くなったのが二日ですから､日に間違いはありません｣とおまさは云った､｢二十九日の朝おでかけになって､あとずっと留守番をしていました､すると六日の午すぎに､日本橋のお店から旦那を戸板にのせて､おしのさんがいらしったんです､ええ､おしのさんはむさし屋の一人娘で､年はたしか明けて十八だったでしょう､おきれいで静かで､おっとりしたいい方でした｣
 主人の喜兵衛は養子だった｡年は四十五歳､三年まえに癆がいで倒れたが､家付きの妻おそのは病気に感染するのを怖れて､看病は娘のおしのに任せ､自分は寮のほうへ移った｡それ以来ずっと別居生活が続いてい､店へは寄りつきもしなかった｡
｢旦那は戸板のまま奥の間へ運ばれ､若い衆たちはすぐに帰ってゆきました｣とおまさは云った､｢おしのさんは奥の間で旦那の側に付きっきり､あたしが御用はないかと訊きにいっても､ただ静かにしていておくれ､お父つぁんの病気が重いから､と仰しゃるばかりでした｣
 そのうちに香の匂いがして来､あんまり強く匂うので､どうかしたかと思ってみにゆくと､病人が躯が臭いと云うので､香をたいているのだから心配には及ばない､とおしのは答えた｡やがて夕方になると､おしのは自分で勝手へ来て､ゆきひらで米をとぎ､｢火鉢で粥を煮るのだ｣と告げ､また､自分の夕食はなんでもよい､と云って奥へ去った｡――それからおまさは天神橋の前まで買物にゆき､魚や野菜を買って来て､飯を炊いたり煮物をしたりしていた｡このあいだに､七日間も留守だった主婦のおそのが帰っていたらしい､おまさはまったく知らなかったが､娘のおしのが来て､母が帰っているから酒の支度をするように､と命じた｡
｢ちょうどお酒がきれていましたので｣とおまさは続けた､｢酒屋へ注文にゆくと云いましたら､それではついでに酒の肴もと仰しゃいました､おまえおっ母さんの好みを知っているだろうから､いいようにしておくれと仰しゃるので､あたしは酒屋と仕出し屋へいって来ました｣
 酒が来､肴が届いて､膳立てをしていると､またおしのが来て云った｡
 ――あとはあたしがしよう､こんな時刻になったけれど､おまえには暇をあげるから､業平のうちへいって泊っておいで｡
 おまさは弟の家に子供が預けてあった｡そのまえの年にやくざな亭主と別れ､七つになる男の子を弟に預けて､自分は下女奉公に出ていたのである｡今年はまだ正月にも帰っていない､というのは､主婦のおそのが連日のように客をするため､暇をもらうことができなかったのである｡
｢ではお願いしますと云って､あたしはすぐに支度をし､途中で土産物を買って業平へ帰ったのです｣おまさはこう云った､｢ええ､ほかに人はいませんでしたから､焼け跡から出たのは旦那とおかみさんと､おしのさんの死骸でしょう､旦那やおかみさんはともかく､おしのさんは本当にお気の毒だと思います｣
 喜兵衛は重態だったが､どうしておそのを気の毒とは思わないのか､町方の者がそう訊くと､おまさはわけは話せないと答えた｡
｢仮にも奉公していた人ですから､あたしの口から詳しいことは云えません､けれど｣とおまさは冷やかな調子で云った､｢おかみさんがあんな死にかたをしたのは､天罰だと思います｣
 店のほうもしらべられたが､結局はっきりした事情は不明のまま､死躰は親子三人のものときまり､五日めに本石町の家で三人の葬式がおこなわれた｡そしてそのあと､親類や縁者たちが集まって相談し､｢むさし屋｣ほどの老舗を潰すのは惜しいということになって､分家に当る亀屋伊兵衛の二男で､伊四郎という者が養子にはいり､喜兵衛の跡を継ぐことになった｡
第一話
一
 喜兵衛の病状が悪化したのは､十二月二十七日の夜のことであった｡
 暮六つに店を閉めてから､夕食を済ませたあと､番頭と二人の手代を相手に､帳合をするのが店のきまりで､そのときは歳末が近づいていたため､地方との取引先の分もあり､十一時過ぎになって､ようやく一と区切りついた｡
 おしのはいつものとおり､女中二人に指図して､九時に茶菓を出してから､父と自分の夜具の並べて敷かれた部屋へいって寝る支度をした｡火鉢の火かげんをみて灰をかけ､煎薬の土瓶を仕掛けた｡その脇の盆には､湯呑茶碗と布で掩いをした金盥､金盥には水がはいってい､たたんだ手拭が五枚重ねてある｡おしのはそれらをよくしらべ､また､父の寝衣が三枚出してあることも慥かめた｡父はひどい盗汗をかくし､毎夜幾たびか咳の発作を起こすため､それだけの支度はどうしても必要なのであった｡
 おしのが着替えをしようとしていると､番頭の嘉助が､声をかけて､はいって来た｡彼は三十七になり､四丁目の裏に家のあるかよい勤めで､妻とのあいだに子供が二人あった｡九時半になると帰るのがきまりで､いつもは襖の外から挨拶だけしてゆくのだが､その夜は声をかけて､部屋の中へはいって来た｡
｢どうしたの｣おしのは訝しげに訊いた｡
｢旦那のことなんですが｣と嘉助は囁くように云った､｢どうもいつもよりぐあいがお悪いようですから､もうおやすみになるように仰しゃって下さいませんか｣
｢どんなぐあいなの｣
｢ひどい熱なんでしょう､お顔が赤いし息苦しそうで､激しい咳こみが二度もございました｣と嘉助は云った､｢おやすみになるようにといくらおすすめしても､おまえこそもう帰れと仰しゃるばかりで｣
 おしのは頷いた､｢いいわ､あたしがいってみるから､嘉助さんはもう帰ってちょうだい｣
｢今夜はもう少し残ることにします｣
｢いいえ､却ってお父つぁんが気をもむから､いつもどおりにしたほうがいいわ｣
 嘉助はよほど残りたそうであったが､おしのは帰るように云い､自分はすぐに店へいってみた｡手代の忠三と徳次郎が､父と机をはさんで坐り､帳合をしていた｡忠三は二十一､徳次郎は二十三歳､年が明けると暖簾を分けて店を出すことになっている｡ほかに小僧をいれて店の者が七人いるが､みんな店の次の部屋で､読み書きや算盤の稽古をしていた｡――おしのは父のようすを見た｡喜兵衛は四十五歳という年よりずっと老けてみえる｡躯もすっかり痩せているし､顔は頬骨から顎の骨まであらわになり､おちくぼんだ眼や､抉ったようなこめかみや､油けのない灰色の髪などは､そのまま死相を示すように感じられた｡
 ――いいえ､そんなことはない｡
 父はまえからあんなふうだった､とおしのはかぶりを振った｡十二の年で奉公に来､二十五で婿養子に直った｡それから二十年､ものみ遊山にもゆかず､寄席や芝居を見るでもなく､もちろん酒や煙草の味も知らない｡ただ｢むさし屋｣のため､しょうばい第一と勤めとおして来た｡老舗の名は聞えていたが確実に資産を積んだのも､隣りにあった紙問屋を買って､油屋の店をひらいたのも､みな喜兵衛のはたらきであった｡
 ――五､六年まえから疲れが出はじめた｡
 あたしが十一､二のころからだ､とおしのは思った｡おしのは幼ないじぶんから､父と母とのあいだに寝かされていた｡それは母の主張であったが､十一､二になってから､夜半に眼がさめたときなど､暗くしてある行燈の光で見ると､父の寝顔はすさまじいほど憔悴してい､死んでいるのではないか､とさえ思ったことが幾たびもあった｡
 ――あのころからときどきこんなふうに見えたことがある｡
 これは死相などというものではない､おしのはそう思いながら､さりげなく父のほうへ近よってゆき､あまえた動作で､帳場格子へよりかかった｡
｢どうした｣と喜兵衛が眼をあげた､｢まだ寝なかったのか｣
｢今夜はなんだか淋しいのよ｣おしのは小さく肩をゆすった､｢お父つぁんが寝てくれなければ眠れそうもないの｣
 喜兵衛はそっと笑った､｢有難いが､私は大丈夫だよ｣
｢あたし本当に淋しいのよ｣
｢わかってる｣と喜兵衛はまた帳面に向かって筆を取った､｢私は大丈夫だよ､自分の躯のことはよく知っているからね､もういって寝なくちゃあだめだよ｣
 そして徳次郎と読み合せを続けた｡
 おしのは諦めた｡父がそう云いだしたらきかないことは､よくわかっていたからである｡部屋へ戻って､寝衣に着替え､もういちど火鉢の埋み火をみてから､夜具の中へはいり､読みかけの｢松代物語｣というよみ本をひろげた｡信濃のくにの松代という城下町に生れた姉妹が､じつの母親を捜すために諸国をまわりながら､いろいろと辛い苦しいめにあったり､悲しいおもいをしたりする話で､もう二度も読み､筋は暗記しているくらいだが､繰返し読んでもおしのは飽きなかった｡――だがその夜は､いくらも､よまないうちに眠ってしまったらしい｡呼び起こされて眼をさますと､徳次郎がまっ蒼な顔をして､｢旦那が｣と云いながら､店のほうを指さしていた｡徳次郎の顔の相が変っているのを見て､おしのははっきり眼がさめ､起きあがって半纒をひっかけた｡
 父は座蒲団を並べた上に寝ていたが､喀血をしたのだろう､口のまわり､着物の衿､そして畳の上まで血が飛び散ってい､それを店の者たちが拭き取っているところだった｡
｢騒ぐな｣と喜兵衛は眼をつむったまま云った､｢薬はいらない､ぬるま湯に塩を入れて来てくれ､おしのを起こすな｣
 喀血は初めてではないが､そんなに多量に吐いたことはなかった｡おしのは足のほうから震えだし､息が詰まりそうになった｡声が出ないので､徳次郎に眼で頷き､茶の間へいって塩のぬるま湯を作った｡喀血したらそうするようにと､まえに医者から聞いていたのである｡おしのは自分をおちつけるために､水を一杯ゆっくりと飲み､それからまた店へ出ていった｡喜兵衛は眼をあいて､はいって来るおしのをみつめた｡
｢塩湯よ｣とおしのは微笑しながら坐った､｢濃すぎたら薄くしますわ｣
｢起きなくってもいいのに｣
｢口をきいてはだめ｣とおしのが云った､｢そうっと頭だけあげてね､徳どんと忠どん､うしろからそっと支えてちょうだい｣
 二人がうしろから抱き起こすようにし､おしのは父に塩湯を飲ませた｡暫くは動かせないので､汚れた着物もそのまま､枕をさせ掻巻を掛けてから､湯で手拭を絞って､胸元から口のまわりを拭いた｡
｢胸がおちつかない｣と喜兵衛が云った､｢手拭で冷やして当ててくれ｣
｢ごめんなさい｣とおしのはすぐに立った､｢それをすっかり忘れてたわ｣
｢おまえたちはいい｣喜兵衛は店の者に向かって云った､｢もうおそいから､片づけるのは明日にしてみんな寝てくれ｣
｢横山さんを呼んで来ましょう｣
｢いや､それも明日でいい､いま呼んだところで､医者にもどうしようもない､じっとしているほかに手はないんだから｣
二
 朝になって横山参得を呼んだ｡呉服橋に住む医者で､喜兵衛が癆がいになって以来のかかりつけだったが､診察をして帰るとき､送りに出たおしのにそっと首を振ってみせた｡
｢むずかしいことになった｣と参得は囁いた､｢こんどはあとをよほど大事にしないと､取返しのつかぬことになるかもしれない｣
 おしのは睡をのんだ｡
｢店に置いてはだめだな｣と参得は首を捻りながら云った､｢あのとおりの性分だから､少しおちつくとすぐに動きだすだろう､こんどは寮のほうへ移して､しょうばいのことなど忘れてゆっくり養生をさせなければいけない､それも一日でも早いほうがいい｣
｢はい｣とおしのは頷いて云った､｢なにか特に効くお薬はないものでしょうか｣
｢ないだろうな､この病気ばかりは特に効のある薬というのはなさそうだ｣参得はまたそっと首を振った､｢空気のきれいなところで､暢気にして精の付く喰べ物をたべる､そうして病勢の衰えるのを待つ､というよりほかにいまのところ手はないと思う｣
｢わかりました｣おしのはまた頷いた､｢できるだけ早くそうするように致します｣
 医者が帰るとすぐに､おしのは亀戸の寮へ使いをやった｡母に来てもらいたい､ということづてを命じ､駕籠でゆかせたのであるが､おたみというその若い女中は､帰って来ると､喜兵衛に向かって､｢おかみさんは躯のぐあいが悪くて来られないそうです｣という返辞を伝えた｡おしのがちょっと座を外していたあいだのことで､珍らしく喜兵衛は怒った｡
｢どうして迎えなどやったんだ｣喜兵衛は尖った声で云った､｢迎えをやっても来ないことはわかっているじゃないか｣
｢ごめんなさい｣おしのは眼で詫びながら答えた､｢どうしても相談しなければならないことがあるんです､あたしちょっといって来てもいいでしょうか｣
｢相談とはどういうことだ｣
 おしのは勇をふるうという気持で､医者の云ったことを父に告げた｡喜兵衛は終るまで聞かずに､激しくおしのを遮った｡
｢私は寮などへゆかない､そんなことをしなくとも大丈夫だ｣
｢だってお父つぁん｣
｢いや､大丈夫だと云ったら大丈夫だ｣と喜兵衛は云った､｢この病気のことなら医者よりも私のほうがよくわかる､私は若いころから躯のことには気をつけているし､この病気にかかってからは､病気に波のあることや､その波がしらを逃げるこつも覚えた､いや本当だ｣喜兵衛はこみあげてくる咳をこらえるために､やや暫く息を休めた､｢人間には寿命というものがある､養生不養生によって､生れつきの寿命を損うこともあるが､自分の躯の調子によく気をつけていれば､どうしたら寿命を保つかということはわかるものだ､それにこの暮に迫って､店をあけるなどということはできない､大丈夫だから心配はしないでおくれ｣
 その日は琴の稽古納めがあった｡午ごろになると喜兵衛は､もうおちついたから着替えをしていっておいで､とおしのに云った｡
｢あとで起きたりなさらなければ｣とおしのが云った､｢でも､お父つぁんは､あたしがいなくなるときっと起きてしまうでしょ｣
｢まだそんなことを云うのか｣と喜兵衛は枕の上で首を振った､｢あまりうるさくしないでくれ､病気の起こっているときはこらえ性がなくなるんだから｣
｢ごめんなさい｣おしのは明るい調子で云った､｢じゃああたしいって来ます｣
｢私は眠るからゆっくりしておいで｣
｢はい｣おしのは元気に頷いた｡
 二人の女中､特にお孝と､手代の徳次郎によく頼んでおいておしのはざっと身支度を済ませ､家を出てから辻駕籠をひろった｡むろん稽古納めにゆくのではない､まっすぐに亀戸の寮へ走らせた｡途中で雨が降りだし､気温も低くなり､寮へ着いたときには､あたりはもういちめんの雪景色であった｡
 冬になってから二度めであるが､その季節にしては珍らしく､形も量も多い牡丹雪で､門から寮の戸口さえ見えないくらいであった｡
 出て来たのはおまさであったが､おしのを見るなりひどく狼狽し､ちょっとお待ち下さい､と云って小走りに奥へ去った｡奥のほうで三味線の音と､唄の声が聞えてい､おしのは構わずにあがって､そこで足袋をぬいだ｡おまさは戻って来ると､あちらはいまちらかっていますからと云い､客用の八帖へとおした｡
｢いますぐに火を取ります｣おまさはそわそわと敷物や火鉢を直しながら云った､｢急にいやなものが降りだして､さぞ途中お寒かったでしょう､いまおかみさんもおいでになりますから｣
｢いいのよ､せかないでも｣
｢風邪ぎみでくさくさするからって｣おまさは眩しそうな顔つきで云った､｢ちょっといまなにしていらしったんですよ､いま火を取りますから｣
 三味線と唄の声はもう聞えなかった｡おまさが火鉢に火を入れ､茶と菓子をはこんで来ると､まもなく母のおそのがあらわれた｡年は三十五歳､大柄ではあるが躯の線が美しく､胸や腰などは娘のようにすんなりしている｡色のやや浅黒いほそおもての顔に､憂いを含んだような切れ長の細い眼と､やはり薄くて小さな唇許が､娘のおしのでさえ惚れぼれするほどの､際立った魅力をもっていた｡
｢途中で降られたんでしょ｣おそのは火鉢へかぶさるように坐りながら､うるんだ眼つきでじっと娘をみつめた､｢寒いからなにか温かいものでも取りましょう､あんたきれいになるばかりだわね｣
｢おっ母さんぐあいが悪いって｣とおしのが云った､｢誰かお客があるようだけれど､寝ていたんじゃないんですか｣
｢お客なんかいやしなくってよ､寝ていたんだけれど頭がくしゃくしゃするもんだから､いま床の上でちょっといたずらをしていたところなの､お午になにを御馳走しようか｣
｢おっ母さん､おたみから話を聞いたでしょう｣とおしのは云った､｢お父つぁんの病気､こんどはひどく重いのよ､これまでにないほどたくさん血を吐いたし｣
｢やめてやめて｣おそのはきれいな長い指の手を振りながらいたずらっ子のように眉をしかめた､｢そんな話を聞くと胸がむかむかするじゃないの､横山さんていう医者が付いてるんだもの､病人のことは医者に任せておけばいいでしょ｣
｢その横山先生が仰しゃるのよ｣おしのはがまん強く､子供をなだめるような調子で云った､｢こんどはよっぽど大事にしなければいけない､店に置いてはだめだから､寮のほうへ移してゆっくり養生をさせるようにって｣
｢いやよ､いいえだめよ､そんなこと｣
三
｢おっ母さん｣とおしのが云った｡
｢そんなことだめよ｣おそのは脇のほうを向き､すぐにまた向き直って､娘の眼をみつめながら云った､｢あんただってわかってるでしょ､そんなこと云ったって当人がきやあしないわ､たとえ医者に死ぬとおどかされても､あの人は店をはなれるような人じゃないことよ､そうでしょ｣
 おしのは母の顔を見まもった｡
｢そらね､あんたの眼を見ればわかるわ｣とおそのは眼を細めて微笑した､｢あんたあの人が承知しないものだから､あたしにすすめさせるつもりで呼びに来たんでしょ｣
｢お父つぁんは｣とおしのが云った､｢おっ母さんの云うことならきっときいてよ｣
｢そんなことをあたしが嬉しがるとでも思ってるの｣
｢お願いよ､おっ母さん｣おしのは母の手を握りながら云った､｢こんどは本当に危なそうなの､こんどだけでいいからうちへ来て､寮で養生するように云ってちょうだい､ねえ､一生のお願いよ､おっ母さん｣
 おそのは娘の手をやさしく撫で､あやすような口ぶりで云った､｢おちついて､おちついて､そんな大げさなこと云わないでよ､あんたっておっ母さんには薄情なくせに､あの人のこととなるとすぐにのぼせあがるのね､たまにはあたしのことを考えてくれてもいいじゃないの｣
｢ええ､そうね｣おしのはぎこちない動作で頷いた｡云いたいことを抑えつけるような､硬い頷きかたで､それから云った､｢おっ母さんのお世話をしなくってほんとに悪いと思うわ､お父つぁんさえおちついたら､こんどこそどんなにでもおっ母さんのお気にいるようにしてよ､だからあたしのお願いもきいてちょうだい｣
｢いいわ､あんたには負けた｣おそのは娘の手を軽く叩き､それを放しながら云った､｢あしたかあさって､そうね､あさっての夕方ごろゆくことにしましょ｣
｢嬉しい､ほんとね｣
｢断わっておくけれど､あんたの考えているとおりになるかどうかは､あたしにも保証できないことよ｣
｢ほんとに来て下さるわね｣
｢ゆくって云ったらゆくわよ｣とおそのが云った､｢お午になにを喰べる､いっそ鳥鍋でも取ろうか｣
 おしのは母といっしょに午飯を喰べた｡
 その座敷に炬燵を入れ､久しぶりに差向いで､話しながら喰べたのであるが､奥の部屋に誰かいるように感じられ､それが気になっておちつかなかった｡人がいるというような､物音やけはいは少しもないし､おまさや母のそぶりにも変ったところはなかった｡ことによると､初めにはいって来たときの､おまさの狼狽したようすや､爪弾きの三味線と低い唄の声を聞いたのが耳に残っているためであろうか｡しんとした奥の部屋に､誰かが息をひそめて､こちらのようすをうかがっている､というふうな感じがどうしても頭から去らなかった｡
｢じゃあきっとね｣おしのは帰るときにもういちど念を押した､｢あさっての夕方､待ってるわよ｣
｢あんたもいま云ったこと忘れないで｣とおそのは云った､｢肌の手入れとお化粧､もう少し髪の流行に気をつけること､よくって｣
 呼びにやった駕籠が来､おまさのさしかける傘で雪をよけながら､おしのは拾い足で門の外へ出た｡
｢肌の手入れ｣駕籠が動きだしてから､おしのは訝しそうに呟き､｢そうか｣と自分に頷いた､｢ごはんをたべながら､ずっとその話をしていたのね｣
 髪化粧､帯､着物､母にはそういうことしか興味はない｡昔からそうだった､とおしのは思った｡眼をつむって母のことを回想すると､化粧をするところや､反物を選んでいるところや､着飾ってでかける姿しかうかんでこない｡父に抱かれて寝たり､子守唄をうたってもらった覚えがあるし､麻疹のときや疱瘡のときはもちろん､風邪をひいたぐらいのときでも､父は側をはなれずに看病してくれた｡しかし母に面倒をみてもらったようなことは少なくとも自分の記憶には残っていない､とおしのは思った｡母は自分に化粧をしてくれ､着飾らせて､芝居や寄席や､遊山や見物には伴れていってくれた｡芝居茶屋でひいきの役者たちを呼び､自分と子役を並んで坐らせ､興がって酒宴をしたことも幾たびかあるし､母がなかまたちと一中節をさらうのだと云って､料理茶屋へ集まり､おさらいなどするようすもなく､男女の芸人や役者などを呼んで遊ぶ､などということも珍らしくはなかった｡
 ――お父つぁんに済まない｡
 そんなときには気が咎め､こんど母にさそわれても決して遊びには出まいと誓うが､華やかでたのしい座敷を思うと､ついその誓いも忘れて､十か十一くらいの年まではそんなことが続いた｡
 ――おっ母さんは平気だった｡
 こんなことはないしょだよ､と云ったこともないし､口止めをするようなこともなかった｡十一ぐらいから､自分は母の誘惑を拒むようになり､しだいに母と疎くなった｡母はそんなことには気もつかないようすで､それからずっと好きなようにくらして来た｡あたしがいっしょでないことが､却って自由なようにさえみえたものだ､とおしのは思った｡
｢あんなに気やすく受け合って来るかしら｣おしのは駕籠に揺られながら呟いた､｢場合が場合だから来てくれるだろうけれど､もし来てくれなかったら､――こんどこそあたし云ってやるわ｣
 こんどこそ云うだけのことを云ってやる､とおしのは思った｡
 夜になると雪は雨になり､明くる朝はきれいに晴れていた｡喜兵衛の病勢は変らず､食欲のないのが心配だったが､咳も少なくなり､熱もさがってゆくようであった｡おそのは来なかった｡約束の三十日は午後から待っていたが､夜になっても使いさえよこさなかった｡
 ――やっぱりそうだったのね｡
 おしのはくやしさのあまり身がふるえた｡すぐ自分で迎えにゆきたいと思ったが､年末の商家は多忙で､女中までが店の手伝いに追われ､おしの以外に父の世話をする手がないため､亀戸までいって来るような暇はまったくなかった｡
 大晦日の晩､十二時過ぎに､喜兵衛がまた喀血した｡十時に店を閉め､番頭と手代たちが帳合にかかった｡それが済んでから､三人が帳簿を持って来､喜兵衛の枕許で説明をした｡喜兵衛は夜具の上に坐り､いちいち帳簿をつき合わせながら聞いていた｡十二時ちょっとまえに年越し蕎麦が来､番頭たち三人が喰べた｡おしのは箸を付けただけで､父に薬湯をのませようとしたが､｢もうすぐに終るから｣と云って､喜兵衛は算盤を置こうとしなかった｡そうしてすっかり終ったとき､喜兵衛は手洗いにゆき､その戻りに廊下で喀血して倒れた｡
 血の量はそれほど多くはなかったが､このまえのときがひどかったし､躰力がまだ恢復していなかったため､喀血したあと失神し､夜が明けるまで昏睡状態が続いた｡
四
 おしのは夜明けを待って､亀戸へ使いをやった｡医者の横山参得は首を振った｡
｢どんな名医でもだめだ｣と参得は病人の枕許で云った､｢もう人間の手には負えない､明日までもむずかしいと思う｣
 おしのは番頭たちと相談をし､例年どおり元旦を祝うことにした｡どんなことがあっても､父は必ず店のしきたりを守った｡おしのはその父の性分を盾に取り､番頭たちの反対を押し切って､いつものように祝儀の支度をさせた｡――亀戸へ使いにやったのは､年上の女中のお孝であったが駕籠でいったにしても帰りが早く､おしのを廊下へ呼んで､｢おかみさんは留守でした｣と囁いた｡
｢留守って､どういうこと｣
｢江ﾉ島へいらしったんですって｣
 おしのの口がゆっくりとあいた｡
｢二十九日の朝｣とお孝は続けた､｢石町の伊勢久のおかみさん､通り二丁目の吉井屋のおかみさん､そのほかお二人ごいっしょに､往き帰りとも七日のつもりで､江ﾉ島の弁天様へおまいりにいらしったんだそうです｣
｢おっ母さんが､二十九日に｣おしのはぼんやりと訊き返した､｢――江ﾉ島へだって｣
｢おまささんがそう云ってました｣
｢二十九日｣とおしのはまた訊き返した､｢それ本当のことだろうね｣
 だがお孝の返辞を聞こうともせず､おしのはふらふらと､よろめきながら自分の部屋へはいり､畳の上へ崩れるように坐った｡二十九日はおしのが訪ねた翌日に当る､そのとき母はあさっての夕方にゆくと約束をした｡父がこれまでとは違って､本当に重態だということをはっきりと云ったし､それが誇張でないことはわかった筈である｡
｢おっ母さんは騙した｣おしのは歯と歯のあいだで呟いた､｢おっ母さんはあたしを騙したのよ､あのときもう江ﾉ島へゆくことはきまっていたんだわ｣
 二十七日の夜から､火のけを入れなかったその部屋は､壁までしみとおるほど冷えていた｡しかしおしのはその寒さにも気づかず､おたみが呼びに来るまで､独りでじっと坐っていた｡おたみは襖の外から呼びかけ､返辞がないので襖をあけた｡
｢いらしったんですか｣とおたみが云った､｢旦那がお呼びです｣
 おしのはゆっくりと女中を見た｡
｢旦那が眼をおさましになったんです｣とおたみは云った､｢そしておしのさんはどこにいるかって｣
 おしのは夢からさめでもしたように､ああと声をあげて立ちあがった｡
 喜兵衛の枕許には手代の忠三がいて､喜兵衛がなにか云うのを､のり出すような恰好で聞いていたが､おしのと入れ替りに店のほうへ出ていった｡喜兵衛はおしのを見ると､安心したように頷き､そして眼をつむった｡おしのは坐って､気分はどうかと訊いた｡喜兵衛は暫く経ってから､力のない低い声で｢大丈夫だ｣と云った｡
｢心配しなくてもいい､大丈夫だ｣と喜兵衛は云った､壁の向うから聞えて来るような声であった､｢同じことを云うようだが､私は大丈夫だ､こんなことで死にはしない､参得さんにはわからないんだ｣
｢ごしょうよ､そんなに話をしないで｣
｢これだけ云っておきたいんだ｣と喜兵衛は眼をつむったままで云った､｢私は夜なかに倒れてからのことを､みんな知っている､気を失ったのはほんの僅かなまで､それからちょっと眠ったときのほかは､なにもかも知っている､もうどんな名医でもだめだと､参得さんの云うのも聞いていたんだ｣
 そして喜兵衛は眼をあき､娘を見て唇に微笑をうかべた｡
｢わかるだろう｣と喜兵衛は云った､｢私はまだ死にはしない､ゆうべ吐いたのは､残っていた悪い血だ､病気の根になっていた悪い血が､ゆうべできれいに出てしまった､医者にはわからないが､私にはそれがよくわかるんだ､きょうほど､――｣と云って喜兵衛はまた眼をつむった､｢この病気になってから､きょうほど気持の軽くなったことはない､胸の中も洗ったようにさっぱりしたよ｣
｢よかったわ､よかったわお父つぁん｣とおしのが云った､｢でも話はもうそのくらいにして､少し眠って下さいな｣
｢ああ眠ろう｣喜兵衛は云った､｢私は眠るから､おまえもつまらない心配はしないでおくれ､――みんなが来たようだね｣
 おしのは振り返った｡すぐに襖があいて､番頭､手代をはじめ店の者がぜんぶはいって来､襖際に並んで坐った｡
｢私がそう云ったんだ｣と喜兵衛が云った､｢おまえが挨拶をしておくれ｣
 おしのは向き直った｡
｢明けましておめでとうございます｣と番頭の嘉助が両手を突いて低頭した｡他の者も一斉に低頭し､嘉助が続けた､｢昨年ちゅうはお世話になりました､今年はお店もますます御繁昌､私共もまたどうぞよろしくお願い申します｣
 そして他の者が｢おめでとうございます｣と声をそろえて云った｡おしのは歯をくいしばった｡
｢はい｣とおしのは俯向いて答えた､｢おめでとう｣それからようやくのことで続けた､｢どうぞ今年も､よろしくお頼み申します｣
 店の者たちは逃げるように去った｡
 ――おめでとう｡
 おしのはその姿勢のまま､心の中でそう繰返した｡
 ――明けまして､おめでとう｡
 おしのは叫びだしたいような衝動におそわれ､それをこらえるために力限り手を握り緊めた｡喜兵衛は太息をつき､これで元日らしくなった､と云った｡私は眠るから､おまえは雑煮を祝って､私の代りに年賀の客に会っておくれ､悪いけれど親類の方たちにも失礼するから､と喜兵衛は云った｡
 三日の夜半――､およそ午前一時ごろに､おしのは父..."""
    },



    {
    "@search.action": "upload",
    "BookId": "57574",
    "BookName": "青かべ物語",
    "BookAuthor": "山本周五郎",
    "BookGenre": "長編小説",
    "Release_Date": "1960年1月",
    "Text": """浦粕町は根戸川のもっとも下流にある漁師町で､貝と海苔と釣場とで知られていた｡町はさして大きくはないが､貝の罐詰工場と､貝殻を焼いて石灰を作る工場と､冬から春にかけて無数にできる海苔干し場と､そして､魚釣りに来る客のための釣舟屋と､ごったくやといわれる小料理屋の多いのが､他の町とは違った性格をみせていた｡
 町は孤立していた｡北は田畑､東は海､西は根戸川､そして南には｢沖の百万坪｣と呼ばれる広大な荒地がひろがり､その先もまた海になっていた｡交通は乗合ﾊﾞｽと蒸気船とあるが､多くは蒸気船を利用し､｢通船｣と呼ばれる二つの船会社が運航していて､片方の船は船躰を白く塗り､片方は青く塗ってあった｡これらの発着するところを｢蒸気河岸｣と呼び､隣りあっている両桟橋の前にそれぞれの切符売り場があった｡
 西の根戸川と東の海を通じる掘割が､この町を貫流していた｡蒸気河岸とこの堀に沿って､釣舟屋が並び､洋食屋､ごったくや､地方銀行の出張所､三等郵便局､巡査駐在所､消防署――と云っても旧式な手押しﾎﾟﾝﾌﾟのはいっている車庫だけであったが､――そして町役場などがあり､その裏には貧しい漁夫や､貝を採るための長い柄の付いた竹籠を作る者や､その日によって雇われ先の変る､つまり舟を漕ぐことも知らず､力仕事のほかには能のない人たちの長屋､土地の言葉で云うと｢ぶっくれ小屋｣なるものが､ごちゃごちゃと詰めあっていた｡
 町の中心部は｢堀南｣と呼ばれ､｢四丁目｣といわれる洋食屋や､｢浦粕亭｣という寄席や､諸雑貨洋品店､理髪店､銭湯､｢山口屋｣という本当の意味の料理屋――これはもっぱら町の旦那方用であるが､そのほか他の田舎町によくみられる旅籠宿や小商いの店などが軒を列ねていた｡その南側の裏に､やはり｢ごったくや｣の一画があり､たった一軒の芝居小屋と､ときたま仮設劇場のかかる空地がある､というぐあいであった｡
 これらのことをどんなに詳しく記したところで､浦粕町の全貌を尽すわけにはいかない｡私も決してそんなつもりはないので､ただこの小さな物語の篇中に出てくる人たちや､出来事の背景になっているものだけを､いちおう予備知識として紹介したにすぎないのである｡
 はじめに｢沖の百万坪｣と呼ばれる空地が､この町の南側にひろがっていると書いた｡私は目測する能力がないので､正確にはなんともいえないが､そこは慥かにその名にふさわしい広さをもっていた｡畑といくらかの田もあるが､大部分は芦や雑草の繁った荒地と､沼や池や湿地などで占められ､そのあいだを根戸川から引いた用水堀が､｢一ついりから｢四ついり｣まで､荒地に縦横の水路を通じていた｡――この水路や沼や池には､鮒､鯉､鮠､鯰などがよく繁殖するため､陸釣りを好む人たちの取って置きの場所のようであった｡また､沼や池や芦の茂みの中には､獺とか鼬などが棲んでいて､よく人をおどろかしたり､なにごとでもすぐに信ずるような､昔ふうの住民を｢隙さえあれば化かそうと思っている｣ということであった｡
 この町ではときたま､太陽が二つ､東と西の地平線上にあらわれることがある｡そういうときはすぐにそっぽを向かなければ危ない｡おかしなことがあるものだ､などと云って二つの太陽を見ると｢うみどんぼ野郎｣になってしまう｡そうしてそのときにはすぐ脇のほうで､獺か鼬の笑っている声が聞えるということである｡特に鼬はたちの悪いいたずら好きで､人が道を歩いていると､ひょいと向うへとびだして来て､立ちあがって､交通整理でもするように､右手をあげて右をさし示したり､左手で左のほうをさしたりする｡そうしたら必ず反対のほうにゆかなければならない｡うっかりしてそちらへゆけば､きまって池か堀か､わるくすると根戸川へ落ちこんでしまう､といわれていた｡
 百万坪から眺めると､浦粕町がどんなに小さく心ぼそげであるか､ということがよくわかる｡それは荒れた平野の一部にひらべったく密集した､一とかたまりの､廃滅しかかっている部落といった感じで､貝の罐詰工場の煙突からたち昇る煙と､石灰工場の建物ぜんたいを包んで､絶えず舞いあがっている雪白の煙のほかには､動くものも見えず物音も聞えず､そこに人が生活しているとは信じがたいように思えるくらいであった｡
 私はその町の人たちから｢蒸気河岸の先生｣と呼ばれ､あしかけ三年あまり独りで住んでいた｡
｢青べか｣を買った話
 芳爺さんに初めて会ったのは｢東｣の海水小屋であった｡冬のことで､海水小屋は取り払われ､半分朽ちた葭簾の屋根と､板を打ちつけた腰掛が一部だけ残っていた｡町を西から東へ貫流する掘割が､東の海へ出る川口のところで､土地の人たちはそのあたり一帯を漠然と｢東｣と呼んでいた｡
 私は海を眺めていた｡腰掛は釘がゆるんでいるので､足を突っ張ってうまく支えていないと､すぐさま潰れてしまいそうであった｡干潮で､遠浅の海は醜い底肌を曝し､堀の水は細く､土色に濁っていた｡急に腰掛がぐらっと揺れたので､私は吃驚して､突っ張っている足に力を入れながら振り返った｡すると一人の老人が､すぐうしろに腰を掛けて､私などは眼にもはいらないといったような顔つきで､古風な莨入を腰から抜くところであった｡私は支える足に気をくばりながら､また海のほうへ眼を戻した｡
｢ずっとめえに､ここへなにかぶっ建てようと思ったっけだが｣と老人が大きな声で云った､百ﾒｰﾄﾙも先にいる人に話しかけるような声であった､｢なんかぶっ建ってくれべえと思ったっけだがねえよ｣
 私は黙っていた｡私は老人しか見なかったが､それではもう一人伴れでもいるのか､と思ったのである｡しかし答える声はなく､老人はやかましい音をさせて煙管をはたき､次のﾀﾊﾞｺを吸いつけた｡煙管はつまっていて､喘息患者の喉のように､ぐずぐずとやにの鳴る音が聞えた｡
｢ずっとめえのこった､おつゆのおっかあがまだ綿屋へ嫁にいかねえころのこった｣と老人は大きな声で云った､そしてやや暫く黙っていてから､また煙管をはたき､三服めを吸いつけて､喚きたてた､｢なんにもおっ建たなかっただよ｣
 私はやはり黙っていた｡
 二度めには百万坪で会った｡季節は春で､強い風が吹いていた｡私は｢二ついり｣の堀に沿った道を､沖の弁天社のほうへ歩いていた｡なんのふぜいもない､だだっ広いだけのその荒地のほぼ中ほどに､無人の､小さな､毀れかかったような古い社が､ひねこびた六七本の松に囲まれて建っている｡いつのころかたいへん流行った弁天で､特に各地の花柳界の女性たちが参詣に列を作ったそうである｡どういう霊験があったのか土地の者は知らない､ただひところばかげて流行り､夥しい参詣者の絶えなかったことと､当時その境内が別世界のように賑わったということだけは､子供たちでさえよく知っていた｡
 潮の匂いのする強い風に吹かれながら､沖の弁天のほうへ歩いていたとき､うしろからいきなり大きな声で呼びかけられ､私はとびあがりそうに驚いて振り返った｡あの老人がすぐうしろにいた｡継ぎはぎだらけの､洗い晒しためくら縞の半纏に､綿入の股引をはき､鼠色になった手拭で頬かぶりをしている｡それはこの土地の漁師たちに共通の常着であるが､もう綿入の股引をはく季節ではなかった｡
｢おめえ舟買わねえか｣と老人は私と並んで歩きながら喚いた､｢ﾀﾊﾞｺを忘れて来ちまっただが､おめえさん持ってねえだかい｣
 私はﾀﾊﾞｺを渡し､ﾏｯﾁを渡した｡老人はﾀﾊﾞｺを一本抜いて口に咥え､風をよけながら巧みに火をつけると､ﾀﾊﾞｺとﾏｯﾁの箱をふところへしまった｡
｢いい舟があんだが｣と老人は二百ﾒｰﾄﾙも向うにあるひねこびた松ﾉ木にでも話しかけるような､大きな声でどなりたてた､｢いい舟で値段も安いもんだが､買わねえかね｣
 私が答えると､老人は初めからその答えを予期していたように､なんの反応もあらわさず､吸っていたﾀﾊﾞｺを地面でもみ消し､残りを耳に挾んでから､手洟をかんだ｡
｢おめえ｣暫く歩いたのち､老人がひとなみな声で云った､｢この浦粕へなにょうしに来ただい｣
 私は考えてから答えた｡
｢ふうん｣と老人は首を振り､ついで例の高ごえで喚いた､｢おんだらにゃあよくわかんねえだが､職はあるだかい｣
 私が答えると､老人はちょっと考えた｡
｢つまり失業者だな｣と老人は喚いた､｢嫁を貰う気はねえだかい｣
 私は黙っていた｡別れるときﾏｯﾁだけ返してもらったが､急に耳の遠くなった老人は､二度も三度も私の云うことを訊き返し､そのため私は自分がひどい吝嗇漢になったような､恥ずかしさを感じた｡
 三度めは根戸川亭で会った｡それは蒸気河岸にある洋食屋で､土間が食堂､奥に座敷があって､夜になると蒸気船(通船といわれていた)の船員や漁師たちが､しばしば盛大に酔って騒いだ｡或る日の午ごろ､私が食堂のがたがたする椅子に掛け､一本のﾋﾞｰﾙでｶﾂ･ﾗｲｽを喰べていると､老人が私の卓子へ来て差向いの椅子に掛けた｡
 いまでもそうであるが､外で食事をするときには､私はなにか読みながらでないとおちつけない癖がある｡そのときも私は青巻という本を読んでいて､老人がそこへ腰掛けたものだから､いっそう熱心に読むふりをし､そうして本から少しも眼を放さないままで､ﾄﾝｶﾂを噛んだりﾋﾞｰﾙを啜ったりしていた｡
 女が座敷のところへ来て､｢芳さんなんにするだえ｣と呼びかけた｡
｢うう｣と老人が答えた､｢おっかあがいねえからめし食うべえと思って来ただが､うう､なんにすべえか考げえてるだ｣
｢うちじゃあ考げえるほどごたいそうなものは出来ねえよ｣
 すると老人が私を見ながら､――そこへ腰掛けたときからずっと､老人が私をみつめ続けていることを私は知っていた､――で､老人は私の顔を見ながら､例のずばぬけた高ごえで喚きたてた｡
｢ﾋﾞｰﾙをｺｯﾌﾟに一杯くんねえかね｣
｢ﾋﾞｰﾙを一杯だって｣と女が云った､｢おらそんなこと聞いたこともねえ､酎のまちげえじゃねえのかえ｣
 東京へゆけばﾋﾞｰﾙの一杯売りをやっている､と老人が云った｡それはﾋﾞﾔﾎｰﾙというものだ､と女が云った｡いや､ﾄﾝｶﾂやｶﾚｰﾗｲｽが出来るから洋食屋と違いはない､と老人が云った｡一杯売りをするのは生ﾋﾞｰﾙといって､樽で来るから一杯ずつでも売れるが､壜詰はあけてしまえばあとがかんのんさまだから一杯だけ売るわけにはいかないのだ､と女が云った｡あとがかんのんさまになってもしょうばいは損して得取れということがある､と老人が喚きたてた｡
 私は縛りあげられ､罠にはまったことを知った｡まだ三分の一ほど残っているﾋﾞｰﾙ壜を､老人のほうへ置き直しながら､私は云わなければならないことを云った｡
｢そうかね｣と云うより早く老人は女に向って喚きたてた､｢ｺｯﾌﾟ｣
 それから私を見て｢ﾀﾊﾞｺの持合せはねえかね｣
 私が答えると､老人は｢なに､いま欲しかねえだよ｣と云った｡
 釣舟宿の｢千本｣の三男の長から､私は老人のことを聞いた｡その土地の出来事について､籠屋のおたまと｢千本｣の長とが､つねにぬかりなく情報を呉れるのである｡おたまも長も小学校の三年生であった｡――老人の名は芳､夫婦っきりで､三本松の裏に住み､｢大蝶｣の倉庫番をしている､ということであった｡｢大蝶｣はその町でいちばん大きく貝の罐詰工場を経営してい､漁師たちの採る貝を沖で買い取るために､大蝶丸という船を持っていた｡
 私の問いに答えて､長はつよく首を振った｡
｢ううん､そんなこたねえだよ｣と長は云った､｢工場はやかましかんべ､だからみんなえっけえ声になっちまうだ｣
 えっけえとはもちろん大きなという意味である｡長はなお｢芳爺さまはそら耳を使う｣と云ったが､それはもう私の知っていることであった｡
 それからのちもときどき道で会ったが､老人は挨拶もしないし､私を見ても棒杭か石ころでも見るような眼つきしかしなかった｡頬かぶりをとった老人の顔は､痩せていて小さく､太陽と潮風にやけた頭は禿げていて､灰色の髪の毛がほんの少し後頭部にあり､頬や顎にはまばらな無精髭が､古くなったﾌﾞﾗｼのように､一本ずつ数えられるほどまばらに､きらきらと銀色に光っていた｡眼には非人間的な鈍い冷たい光があり､殆んど唇が無いようにみえる薄い唇には､いつも人を小ばかにしたような､狡猾な微笑が刻みつけられていた｡
 尤もこれは芳爺さんに限らず､その土地の一部の人たちに共通した顔だちであった｡かれらは季節ごとに来る遊覧客､――魚釣り､汐干狩り､海水浴など､遊びに来る都会の客たちから｢うまくせしめる｣習慣がついているので､その冷たく鈍い眼や､狡猾そうな口つきの裏には､いつでも朴訥な表情をつくり､あいそ笑いをする用意ができているのであった｡――四月の末か五月のはじめころ､たぶん五月のはじめころであったろう､私は三本松のところで老人に捉まった｡
 三本松といっても､樹齢の古い松ﾉ木が一本しかない｡ずっと昔は三本あったそうであるが､私の聞いた限りでは､それを自分の眼で見たという者はなかった｡――堀の岸に横這いのかたちで枝を伸ばしている｡その松ﾉ木の脇に､水から揚げて久しいべか舟が伏せてあった｡ずいぶんまえからそこにあり､私は通りかかるたびにそれを見た｡べか舟というのは一人乗りの平底舟で､多く貝や海苔採りに使われ､笹の葉のような軽快なかたちをしてい､小さいながら中央に帆桁もあって､小さな三角帆を張ることができた｡しかし､そこに伏せてあったのは胴がふくれていてかたちが悪く､外側が青いﾍﾟﾝｷで塗ってあり､見るからに鈍重で不恰好だった｡
｢あのぶっくれ舟か｣と長が或るとき鼻柱へ皺をよらせ､さも軽蔑に耐えないというように云った､｢青べかってえだよ｣
 この誇り高い小学三年生は､見る気にもなれないという顔つきでそっぽを向いた｡
 それは慥かにぶっくれ舟であった｡伏せてある平底の板は乾いてはしゃぎ､一とところあいている穴から､去年の枯れ草がひょろひょろと伸びていた｡水から揚げられた古い舟ほど､哀れに頼りなげなものはない｡それは老衰して役に立たなくなった馬が､飼主にも忘れられ､厩の裏でひとりしょんぼり首を垂れているような感じにみえる｡――その日も私は道傍に佇んで､人間も同じようなものだ､などというのは俗すぎるな､というようなことを思いながら､暫くﾀﾊﾞｺをふかしていた｡
 そこへ老人が来て話しかけた｡私は気づかなかったが､老人は私のようすを見ていたらしい｡おそらく､私がその舟にすっかり惚れこんだものと思ったのであろう､にこやかな､とりいるような笑顔をつくり､｢この舟を買わねえかね｣とあいそのいい声で喚いた｡
 私は答えることができなかった｡
｢先生はこの土地のことを詳しく見てえって云ってたんべが｣と老人が喚いた､｢そんなら岡の上べえ歩きまわってもしょあんめえじゃ､根戸川のまわりだの百万坪のいりだの､堀もそうだし､沖へも出てみるがいいだ､それにはこの舟さえあれば用が足りるだよ｣
 まあ見てくれと云って､老人は伏せてある青べかをひき起こした｡それは極めてすばやく､声をかける隙もない動作だった｡
｢ほれ見せえま｣と老人は云った､｢まっさらとは云えねえが､造ってからまだ七年にしかなんねえ､大事にしろばまだ十五年や二十年はたっぷり使えるだ｣
 私は自分の考えを述べようとした｡
｢値段もまけるだよ｣と､老人は喚きたてた､｢蒸気河岸の先生のこったからよ､思いきって五までまけるだ､たった五だ｣
 私が答えると､老人は片手を出した｡
｢ﾀﾊﾞｺ｣と老人は云った｡
 私はﾀﾊﾞｺとﾏｯﾁを渡した｡
｢じゃあ､なんだ｣と老人はﾀﾊﾞｺを一本抜いて火をつけ､ﾀﾊﾞｺの箱はふところへ入れ､ﾏｯﾁだけを返しながら喚いた､｢先生のこったから思いきって四にすべえ､四だ｣
 私が答えると､老人はﾀﾊﾞｺを地面でもみ消し､残りを耳にはさみながら喚きたてた｡私は長の顔や､軽蔑しきった口ぶりを思いだしたが､同時に､自分が老人に縛りあげられ､ぬけ出すことのできない罠にかかったことを悟った｡｢見せえま｣と老人は喚き続けた､｢揚げっ放しにしといたからちっとばかはしゃいでるだが､まだこんなにしっかりしてるだ｣
 老人は舟べりや舳先を､大事そうに撫でたり叩いたりした｡私はそれを眺めながら､老人が舟をひき起こすときのすばやい動作には二つの意図があった､ということに気づいた｡一つは私を捉えること､他の一つは去年の枯れ草が覗いていた舟底の穴を私から隠そうとしたのだ､ということである｡――もう一つ､これを書いては人が信じなくなるだろうと思って､書かないことにするつもりであるが､老人が舳先を掴んでゆすぶったとき､舳先の尖ったところが折れてしまった｡すると老人は自分の手にある折れた舳先の､折れたところへ唾をつけて､元の部分と合わせ､そこを片手で押えたまま､いっそう高ごえになって喚きたてるのであった｡事実はこのとおりだったのだが､これを文字にすると､おそらく人は筆者が調子づいてふざけていると思うにちがいない｡｢事実を書く｣ということがいかに困難なしごとであるかは､こんな些細な点でも思い知らされるのである｡
｢よし､そんなら三と五十にすべえ｣と老人は云った､｢これ以上は鐚一文負からねえだ､三と五十､これで話はきまっただ｣
 私はちょっと質問した｡
｢そんなこたあ屁でもねえさ｣と老人は云った､｢いかずちの船大工に頼めばすぐ繕ってくれるだ､いいとも､おらが持ってって頼んでやるだよ｣
｢それから｣と老人はいそいで付け加えた､｢こういう売り買いには､買い手のほうでなにか物を付けるのがしきたりになってるだ､豚肉の百匁でもいいし､夏なら西瓜の三つくれえかな､うう､おめえよく舶来のﾀﾊﾞｺを吸ってるようだが｣
 私は豚肉を届けると答えた｡
 こうして私は｢青べか｣の持ち主になった｡どんなに小さく､そしてぶっくれ舟であるにもせよ､一ぱいの舟の所有者になったのだが､私はうれしくもなかったし､誇りがましい気持にもなれなかった｡長をはじめとする少年たちの軽侮の眼や､嘲笑の声を考えるだけで､むしろ急に肩身のせまくなったような鬱陶しい､沈んだ気分にとらわれたのであった｡
｢いいさ､あんな舟｣と私は帰る道で自分に云った､｢乗らなければいいんだ｣
 私は明くる日､老人のところへ舟の代金と､豚肉を百匁だけ届け､なお青べかについて､二三のことを頼んだ｡老人はこころよく受け合い､そのとおりにすると約束した｡
蜜柑の木
 助なあこ(あにいというほどの意味)はお兼に恋をした｡助なあこは大蝶丸の水夫であり､お兼は｢大蝶｣の罐詰工場へ貝を剥きにかよう雇い女で､亭主があった｡
 この土地で恋といえば､沖の百万坪にある海苔漉き小屋へいって寝ることであった｡そんなてまをかける暇がなければ､裏の空地の枯れ芦の中でもいいし､夏なら根戸川の堤でも､妙見堂の境内でも､消防のﾎﾟﾝﾌﾟ小屋でも用は足りた｡実際のところ､海苔漉き小屋まで寝にゆくのは､よほど二人がのぼせあがっているか､ゆきすぎた声を抑えることのできない女との場合､――土地の人たちのあいだで､そういう癖のある五人の女性の名が公然と話題になっていたが､――などで､かれらの意見によれば､｢そんなにてま暇をかけるほど珍しいことでもあんめえじゃあ｣というのが常識であった｡
 助なあこはそうではなかった｡彼は中学生が女学生を恋するように､純粋に､初心に恋していた｡大蝶丸で沖へ貝を積みにいっているあいだ､彼の胸はつねにお兼を想うことで痛み､その眼にはお兼の姿､――工場の古びた建物の前で､大勢の女や老婆たちと並んで､巧みに貝を剥いている姿が､絶えずあらわれたり消えたりするのであった｡
 大蝶丸の水夫は三人で､船長の荒木さんはべつに家庭を持っていたが､ｴﾝｼﾞさんの正山さんと水夫たちは､工場の中にある小屋に住んでいた｡助なあこは自分の恋を秘し隠しにし､誰にも気どられないように､最高の抑制を保ち続けていたが､或る夜半､ねごとにお兼の名を呼んだのを､隣りに寝ていた二人の水夫に聞かれて､せっかくの努力がむだになってしまった｡
｢ゆんべが初めてじゃねえぞ｣と水夫の一人が云った､｢おんだらあ何遍も聞いているだ､なあ｣
｢おうよ｣と他の水夫が云った､｢名めえをはっきり云ったなあ､ゆんべが初めてだっけ｡ずっとめえから何遍も好きだあ好きだってねごとう云ってたっけだ｣
｢お､か､ね､さん｣と先の水夫が両手で自分の肩を抱きしめ､身もだえしながら作り声で云った､｢おら､おめえが､好きだ､死ぬほど好きだ､よう｣
 助なあこは硬ばった顔でそっぽを向き､手の甲で眼を拭いた｡彼は死んでしまいたいと思った｡もしできることなら､その場で二人を半殺しのめにあわせてやりたかった｡しかし彼は痩せているし､背丈も五尺とちょっとしかない｡他の二人はどちらも彼より肉付きがよく､はるかに力も強かった｡それは沖で貝を積むときや､工場へ戻って積みおろしをするときなどでよくわかっていた｡
 彼は死んでしまいたいと思った｡
 助なあこは固い決心をし､お兼のほうへは眼も向けず､貝を剥いている彼女の前を通るときには､まっすぐに向うを見たままいそぎ足で､殆んど走るように通りぬけた｡彼はやがて機関士になるつもりで､仕事が終ったあとは､ｴｼｼﾞﾝに関する本にしがみついて､熱心に独学を続けていた｡それらの本の大部分は荒木船長に借りたものであるが､中の幾冊かは､――ﾃﾞｨｰｾﾞﾙ･ｴﾝｼﾞﾝに関する本は､自分で東京の神田へいって買ったものであった｡
 彼は夜の十二時まえに寝たことはなかった｡他の水夫やｴﾝｼﾞさんは､毎晩のように飲みにでかけ､帰ってくると｢一厘ばな｣か賽ころ博奕で夜更しをした｡ごったくやの女たちを伴れこんで､わるふざけをしたり､博奕や女のことでとっ組みあいの喧嘩をしたりした｡そういう騒ぎの中で､助なあこは小屋の隅のほうに机を移し､両手で耳を塞いで本を読んだり､ﾉｰﾄを取ったりするのであった｡その十坪ほどの､細長い､箱のような小屋には､燭光の弱い裸の電球が､天床から一つぶらさがっているだけである｡隅のほうへ届く光は極めて微弱だったが､それでも助なあこは本にしがみつき､帳面に眼を押しつけるようにしてﾉｰﾄを取った｡
 周囲の人たちにとって､この独学はばかげたことであった｡そのくらいのｴﾝｼﾞﾅｰになるには､五六年も船に乗って､実地にｴﾝｼﾞさんのすることを見ていれば､それだけで立派にｴﾝｼﾞﾅｰになれるし､現に二つの通船会社のｴﾝｼﾞさんたちでさえ､多くはそのようにして機関士になったのである｡
 お兼のことでからかわれてから､助なあこはすっかり人嫌いになり､ますます独学に熱中した｡ねごとの話はたちまちひろまったが､そのまますぐに忘れられた｡この土地では､どこのかみさんが誰と寝た､などという話は家常茶飯のことで､たとえばおめえのおっかあが誰それと寝たぞと云われたような場合でも､その亭主はべつに驚きもしない､おっかあだってたまにゃあ味の変ったのが欲しかんべえじゃあ､とか､おらのお古でよかったら使うがいいべさ､と云うくらいのものであった｡――もちろんこれら亭主たち自身も｢変った味｣をせしめているのであるし､また､全部の人たちがそんなに脱俗しているというのでもない｡｢浦粕では娘も女房も野放しだ｣と､はっきり土地の人たちは云っているが､それでも嫉妬ぶかい人間もたまにはいて､ときに凄いような騒ぎの起こることも幾たびかあった｡
 助なあこの場合には､ねごとで恋の告白をしたというだけだったから､ほんのお笑いぐさとして忘れられてしまったが､傷ついた助なあことお兼とは､それぞれの立場で忘れることができなかったようだ｡
 初夏の或る午後､二人は根戸川の土堤で初めて話をした｡その日は工場が休みで､助なあこは午めしのあと､本を二冊持って土堤へゆき､若草の伸びた斜面に腰をおろして､本をひらいた｡読んでゆき､頁を繰るが､なんにも頭にはいらない｡活字の列はただ素通りするだけで､一行読むごとにきれいに消えてしまう｡彼は音読もしてみた､一句ずつ指で押えながらやってみたが､やっぱり同じことで､いくら繰返し読んでみても､なに一つ頭に残らないのであった｡
 そこへお兼が来た｡彼女は助なあこのあとを跟けて来たのだ､まえから彼のようすを見ていて､自分のほうからきっかけをつけなければならないと悟り､その日ようやく機会をつかんだのである｡
｢あら､助さんじゃないの｣とお兼はいかにも意外そうに呼びかけた､｢こんなところでなにしてるの､あら､勉強ね｣
 助なあこは本を閉じ､振り向きもせずに､じっと固くなっていた｡彼は全身が火のように熱く､心臓が喉までとびだして来るように感じた｡お兼は斜面へおりて来て､彼と並んで草の上に腰をおろした｡すると､あま酸っぱいような女の躰臭と､白粉の匂いとが入り混った､なまあたたかい空気が彼を包み､彼は頭がくらくらするように思った｡
｢もう春もおしまいだねえ｣お兼はその言葉の品のよさに自分でうっとりとなりながら云った､｢水の流れと人の身はって､はかないもんだわねえ｣
 陽の傾いた空にはうすい靄があって､根戸川の広い水面は波もなく､まるで眠っているように静かだった｡あたためられた土の香や､若草の匂いがあたりに漂ってい､対岸の若い芦の茂みでは､ときどきけたたましく小鳥の騒ぐ声が聞えた｡
｢けけちかしら｣とお兼が云った､｢まだけけちにしては早いかしら｣
 見ると助なあこはふるえていた｡蒼く硬ばった顔を俯向け､膝を抱えた両手の指を揉みしだき､下唇を噛みしめながら､躯ぜんたいでふるえていた｡お兼はふしぎなよろこびを感じた｡これまで一度も感じたことのない､ぞっと総毛立つような､快楽の戦慄が突きぬけるように思った｡
｢あたしあんたが好きよ｣とお兼は彼の耳に囁いた､｢あんた､芳野の海苔漉き小屋､知ってるでしょ､知ってるわね｣
 助なあこは黙って頷いた｡
｢あたしあんたに話したいことがあるの｣とお兼は続けた､｢今夜ね､七時ごろあそこへ来てちょうだい､来てくれる､ねえ｣
 お兼はそっと助なあこの手に触れた｡彼はぴくっとなり､躯をいっそう固くし､そしてお兼の手に伝わるほど激しくふるえた｡お兼はまた､あのふしぎなよろこびの感覚におそわれ､助なあこの手首をぎゅっと握ってから､それを放した｡
｢もうみんなが沖から帰ってくるじぶんだわ｣とお兼は云って溜息をついた､｢みつかると口がうるさいからあたし帰るわ､世の中ってままならないもんね｣
 お兼はもういちど夜の約束をし､鼻唄をうたいながら去っていった｡
 助なあこは時間を計っていて､やがてそっと振り向いてみた｡あまり長いこと同じ姿勢でいたため､首の骨がきくんと鳴り､頸の筋がつった｡お兼はもうずっと遠く､白い煙に包まれている石灰工場の近くまでいっていた｡
｢あんたが好きよ｣助なあこは頸の筋を揉みながら､お兼の云った言葉をまねてみた､｢あたしあんたが好きよ｣
 彼の顔が歪み､眼から涙がこぼれ落ちた｡
 もう春も終りだ､世の中はままならない､あたしあんたが好きよ､水の流れと人の身は､はかないもんね｡それらの言葉が彼の頭の中で､一つ一つはっきりと､この世のものとは思えないほど美しく聞えた｡それは殆んど純金の価値を持ち､純金の光を放つように思えた｡
｢おら一生､忘れねえ｣助なあこはそっと呟いた､｢どんなに年をとっても､死ぬまでも､きっと忘れねえ､きっとだ｣
 美しいものは毀れやすい､毀れやすいからこそ美しい､などと云うつもりはない｡ここには美しいものはないのだ､逆に､美しい感情がもてあそばれ､汚されるのであるが､助なあこの受けた感動だけは美しく､清らかに純粋であった｡
 彼はその夜､約束の時間に約束の場所へいった｡芳野は堀南の釣舟屋であるが､季節には海苔もやるので､弁天社のうしろに漉き小屋と干し場を持っていた｡そこは沖の百万坪のとば口にあり､畑と荒地に囲まれ､隣りの漉き小屋とは二百ﾒｰﾄﾙもはなれていた｡――日の永くなる季節ではあったが､もうすっかり昏れてしまい､あたたかい宵闇のどこかから､みみずの鳴く声が聞えて来た｡お兼はもうそこにいて､暗い小屋の前から彼を呼んだ｡助なあこは膝ががくがくするので､転ばないように用心しながらそっちへいった｡
｢待たせるのね､あんた｣お兼はじれったそうに云った､｢女を待たせるなんて罪よ､にくらしい｣
 お兼は衝動的に助なあこの手を握った｡彼は狼狽して､ぶきようにしりごみをし､握られた手を放そうとしながら､云った､｢なにか話すことがあるって｣
 その声は喉でかすれ､言葉ははっきりしなかった｡お兼は含み笑いをしながら､握った手をもっと引きよせた｡土堤のときよりも強く､白粉と女の匂いが彼を包み､彼は眼がくらみそうになった｡
｢そうよ､大事な話があるの｣とお兼は囁いた､｢中でゆっくり聞いてもらうわ､ね､ここへはいりましょう｣
｢おら､――｣と云って彼は足を踏ん張った｡
｢世話をやかせないで｣
｢それでも､おら｣と彼は口ごもった｡
｢いいから｣とお兼は荒い息をしながら､おどろくほどの力で彼を引きよせた､｢なにもおっかないことするわけじゃないじゃないの､たまには男らしくするもんよ｣
 助なあこの歯ががちがちと鳴った｡
 お兼は彼を小屋の中へ伴れこみ､入口の戸を閉めた｡この種の漉き小屋は､入口の三尺の引戸に南京錠が掛けてある｡しかしその多くはぐらぐらで､鍵の必要はなく､ちょっと引張れば錠前ごと抜けてしまい､出てゆくときには元のように挿し込んでおけばいいのであった｡
｢あんた､まだふるえているの｣小屋の中からお兼の声が聞えた､｢さあ､そんなにしてちゃ窮屈じゃないの､この手をこうしなってば｣
 ついで彼女の含み笑いが聞えた｡
｢助さん｣とあまえた鼻声でお兼が云った､｢あんた幾つ､――そう､十九なの､若いのね､うれしい｣
 お兼はそのとき三十五歳であった｡亭主のしっつぁんは呑んだくれの怠け者で､ときたま思いだしたように､なにかの雇われ仕事にでかけるが､｢まる一日働いたことがねえ｣といわれていた｡博奕を打つでもなく､女にちょっかいを出すわけでもない｡ただ酒を飲んで寝ころがるか､ぶらぶら歩きまわってむだ話をするだけである｡云うまでもないだろうが､家計は稼ぎ手のお兼がにぎっていて､しっつぁんは与えられる小遣いでやっているのだが､そんなものが長くある筈はなく､彼はもっぱら奢ってくれそうな相手を求めてぶらつき､またしばしばお兼の男のところへいってねだった｡
 お兼は子を産まないためか､肌の艶もよく､浮気性の女に共通の嬌めかしさ､誘惑的な声と身ぶり､言葉よりずっと明確に意志を伝える眼つき､などをもっていた｡それは洗練されたものではな...浦粕町は根戸川のもっとも下流にある漁師町で､貝と海苔と釣場とで知られていた｡町はさして大きくはないが､貝の罐詰工場と､貝殻を焼いて石灰を作る工場と､冬から春にかけて無数にできる海苔干し場と､そして､魚釣りに来る客のための釣舟屋と､ごったくやといわれる小料理屋の多いのが､他の町とは違った性格をみせていた｡
 町は孤立していた｡北は田畑､東は海､西は根戸川､そして南には｢沖の百万坪｣と呼ばれる広大な荒地がひろがり､その先もまた海になっていた｡交通は乗合ﾊﾞｽと蒸気船とあるが､多くは蒸気船を利用し､｢通船｣と呼ばれる二つの船会社が運航していて､片方の船は船躰を白く塗り､片方は青く塗ってあった｡これらの発着するところを｢蒸気河岸｣と呼び､隣りあっている両桟橋の前にそれぞれの切符売り場があった｡
 西の根戸川と東の海を通じる掘割が､この町を貫流していた｡蒸気河岸とこの堀に沿って､釣舟屋が並び､洋食屋､ごったくや､地方銀行の出張所､三等郵便局､巡査駐在所､消防署――と云っても旧式な手押しﾎﾟﾝﾌﾟのはいっている車庫だけであったが､――そして町役場などがあり､その裏には貧しい漁夫や､貝を採るための長い柄の付いた竹籠を作る者や､その日によって雇われ先の変る､つまり舟を漕ぐことも知らず､力仕事のほかには能のない人たちの長屋､土地の言葉で云うと｢ぶっくれ小屋｣なるものが､ごちゃごちゃと詰めあっていた｡
 町の中心部は｢堀南｣と呼ばれ､｢四丁目｣といわれる洋食屋や､｢浦粕亭｣という寄席や､諸雑貨洋品店､理髪店､銭湯､｢山口屋｣という本当の意味の料理屋――これはもっぱら町の旦那方用であるが､そのほか他の田舎町によくみられる旅籠宿や小商いの店などが軒を列ねていた｡その南側の裏に､やはり｢ごったくや｣の一画があり､たった一軒の芝居小屋と､ときたま仮設劇場のかかる空地がある､というぐあいであった｡
 これらのことをどんなに詳しく記したところで､浦粕町の全貌を尽すわけにはいかない｡私も決してそんなつもりはないので､ただこの小さな物語の篇中に出てくる人たちや､出来事の背景になっているものだけを､いちおう予備知識として紹介したにすぎないのである｡
 はじめに｢沖の百万坪｣と呼ばれる空地が､この町の南側にひろがっていると書いた｡私は目測する能力がないので､正確にはなんともいえないが､そこは慥かにその名にふさわしい広さをもっていた｡畑といくらかの田もあるが､大部分は芦や雑草の繁った荒地と､沼や池や湿地などで占められ､そのあいだを根戸川から引いた用水堀が､｢一ついりから｢四ついり｣まで､荒地に縦横の水路を通じていた｡――この水路や沼や池には､鮒､鯉､鮠､鯰などがよく繁殖するため､陸釣りを好む人たちの取って置きの場所のようであった｡また､沼や池や芦の茂みの中には､獺とか鼬などが棲んでいて､よく人をおどろかしたり､なにごとでもすぐに信ずるような､昔ふうの住民を｢隙さえあれば化かそうと思っている｣ということであった｡
 この町ではときたま､太陽が二つ､東と西の地平線上にあらわれることがある｡そういうときはすぐにそっぽを向かなければ危ない｡おかしなことがあるものだ､などと云って二つの太陽を見ると｢うみどんぼ野郎｣になってしまう｡そうしてそのときにはすぐ脇のほうで､獺か鼬の笑っている声が聞えるということである｡特に鼬はたちの悪いいたずら好きで､人が道を歩いていると､ひょいと向うへとびだして来て､立ちあがって､交通整理でもするように､右手をあげて右をさし示したり､左手で左のほうをさしたりする｡そうしたら必ず反対のほうにゆかなければならない｡うっかりしてそちらへゆけば､きまって池か堀か､わるくすると根戸川へ落ちこんでしまう､といわれていた｡
 百万坪から眺めると､浦粕町がどんなに小さく心ぼそげであるか､ということがよくわかる｡それは荒れた平野の一部にひらべったく密集した､一とかたまりの､廃滅しかかっている部落といった感じで､貝の罐詰工場の煙突からたち昇る煙と､石灰工場の建物ぜんたいを包んで､絶えず舞いあがっている雪白の煙のほかには､動くものも見えず物音も聞えず､そこに人が生活しているとは信じがたいように思えるくらいであった｡
 私はその町の人たちから｢蒸気河岸の先生｣と呼ばれ､あしかけ三年あまり独りで住んでいた｡
｢青べか｣を買った話
 芳爺さんに初めて会ったのは｢東｣の海水小屋であった｡冬のことで､海水小屋は取り払われ､半分朽ちた葭簾の屋根と､板を打ちつけた腰掛が一部だけ残っていた｡町を西から東へ貫流する掘割が､東の海へ出る川口のところで､土地の人たちはそのあたり一帯を漠然と｢東｣と呼んでいた｡
 私は海を眺めていた｡腰掛は釘がゆるんでいるので､足を突っ張ってうまく支えていないと､すぐさま潰れてしまいそうであった｡干潮で､遠浅の海は醜い底肌を曝し､堀の水は細く､土色に濁っていた｡急に腰掛がぐらっと揺れたので､私は吃驚して､突っ張っている足に力を入れながら振り返った｡すると一人の老人が､すぐうしろに腰を掛けて､私などは眼にもはいらないといったような顔つきで､古風な莨入を腰から抜くところであった｡私は支える足に気をくばりながら､また海のほうへ眼を戻した｡
｢ずっとめえに､ここへなにかぶっ建てようと思ったっけだが｣と老人が大きな声で云った､百ﾒｰﾄﾙも先にいる人に話しかけるような声であった､｢なんかぶっ建ってくれべえと思ったっけだがねえよ｣
 私は黙っていた｡私は老人しか見なかったが､それではもう一人伴れでもいるのか､と思ったのである｡しかし答える声はなく､老人はやかましい音をさせて煙管をはたき､次のﾀﾊﾞｺを吸いつけた｡煙管はつまっていて､喘息患者の喉のように､ぐずぐずとやにの鳴る音が聞えた｡
｢ずっとめえのこった､おつゆのおっかあがまだ綿屋へ嫁にいかねえころのこった｣と老人は大きな声で云った､そしてやや暫く黙っていてから､また煙管をはたき､三服めを吸いつけて､喚きたてた､｢なんにもおっ建たなかっただよ｣
 私はやはり黙っていた｡
 二度めには百万坪で会った｡季節は春で､強い風が吹いていた｡私は｢二ついりの堀に沿った道を､沖の弁天社のほうへ歩いていた｡なんのふぜいもない､だだっ広いだけのその荒地のほぼ中ほどに､無人の､小さな､毀れかかったような古い社が､ひねこびた六七本の松に囲まれて建っている｡いつのころかたいへん流行った弁天で､特に各地の花柳界の女性たちが参詣に列を作ったそうである｡どういう霊験があったのか土地の者は知らない､ただひところばかげて流行り､夥しい参詣者の絶えなかったことと､当時その境内が別世界のように賑わったということだけは､子供たちでさえよく知っていた｡
 潮の匂いのする強い風に吹かれながら､沖の弁天のほうへ歩いていたとき､うしろからいきなり大きな声で呼びかけられ､私はとびあがりそうに驚いて振り返った｡あの老人がすぐうしろにいた｡継ぎはぎだらけの､洗い晒しためくら縞の半纏に､綿入の股引をはき､鼠色になった手拭で頬かぶりをしている｡それはこの土地の漁師たちに共通の常着であるが､もう綿入の股引をはく季節ではなかった｡
｢おめえ舟買わねえか｣と老人は私と並んで歩きながら喚いた､｢ﾀﾊﾞｺを忘れて来ちまっただが､おめえさん持ってねえだかい｣
 私はﾀﾊﾞｺを渡し､ﾏｯﾁを渡した｡老人はﾀﾊﾞｺを一本抜いて口に咥え､風をよけながら巧みに火をつけると､ﾀﾊﾞｺとﾏｯﾁの箱をふところへしまった｡
｢いい舟があんだが｣と老人は二百ﾒｰﾄﾙも向うにあるひねこびた松ﾉ木にでも話しかけるような､大きな声でどなりたてた､｢いい舟で値段も安いもんだが､買わねえかね｣
 私が答えると､老人は初めからその答えを予期していたように､なんの反応もあらわさず､吸っていたﾀﾊﾞｺを地面でもみ消し､残りを耳に挾んでから､手洟をかんだ｡
｢おめえ｣暫く歩いたのち､老人がひとなみな声で云った､｢この浦粕へなにょうしに来ただい｣
 私は考えてから答えた｡
｢ふうん｣と老人は首を振り､ついで例の高ごえで喚いた､｢おんだらにゃあよくわかんねえだが､職はあるだかい｣
 私が答えると､老人はちょっと考えた｡
｢つまり失業者だな｣と老人は喚いた､｢嫁を貰う気はねえだかい｣
 私は黙っていた｡別れるときﾏｯﾁだけ返してもらったが､急に耳の遠くなった老人は､二度も三度も私の云うことを訊き返し､そのため私は自分がひどい吝嗇漢になったような､恥ずかしさを感じた｡
 三度めは根戸川亭で会った｡それは蒸気河岸にある洋食屋で､土間が食堂､奥に座敷があって､夜になると蒸気船(通船といわれていた)の船員や漁師たちが､しばしば盛大に酔って騒いだ｡或る日の午ごろ､私が食堂のがたがたする椅子に掛け､一本のﾋﾞｰﾙでｶﾂ･ﾗｲｽを喰べていると､老人が私の卓子へ来て差向いの椅子に掛けた｡
 いまでもそうであるが､外で食事をするときには､私はなにか読みながらでないとおちつけない癖がある｡そのときも私は青巻という本を読んでいて､老人がそこへ腰掛けたものだから､いっそう熱心に読むふりをし､そうして本から少しも眼を放さないままで､ﾄﾝｶﾂを噛んだりﾋﾞｰﾙを啜ったりしていた｡
 女が座敷のところへ来て､｢芳さんなんにするだえ｣と呼びかけた｡
｢うう｣と老人が答えた､｢おっかあがいねえからめし食うべえと思って来ただが､うう､なんにすべえか考げえてるだ｣
｢うちじゃあ考げえるほどごたいそうなものは出来ねえよ｣
 すると老人が私を見ながら､――そこへ腰掛けたときからずっと､老人が私をみつめ続けていることを私は知っていた､――で､老人は私の顔を見ながら､例のずばぬけた高ごえで喚きたてた｡
｢ﾋﾞｰﾙをｺｯﾌﾟに一杯くんねえかね｣
｢ﾋﾞｰﾙを一杯だって｣と女が云った､｢おらそんなこと聞いたこともねえ､酎のまちげえじゃねえのかえ｣
 東京へゆけばﾋﾞｰﾙの一杯売りをやっている､と老人が云った｡それはﾋﾞﾔﾎｰﾙというものだ､と女が云った｡いや､ﾄﾝｶﾂやｶﾚｰﾗｲｽが出来るから洋食屋と違いはない､と老人が云った｡一杯売りをするのは生ﾋﾞｰﾙといって､樽で来るから一杯ずつでも売れるが､壜詰はあけてしまえばあとがかんのんさまだから一杯だけ売るわけにはいかないのだ､と女が云った｡あとがかんのんさまになってもしょうばいは損して得取れということがある､と老人が喚きたてた｡
 私は縛りあげられ､罠にはまったことを知った｡まだ三分の一ほど残っているﾋﾞｰﾙ壜を､老人のほうへ置き直しながら､私は云わなければならないことを云った｡
｢そうかね｣と云うより早く老人は女に向って喚きたてた､｢ｺｯﾌﾟ｣
 それから私を見て｢ﾀﾊﾞｺの持合せはねえかね｣
 私が答えると､老人は｢なに､いま欲しかねえだよ｣と云った｡
 釣舟宿の｢千本｣の三男の長から､私は老人のことを聞いた｡その土地の出来事について､籠屋のおたまと｢千本｣の長とが､つねにぬかりなく情報を呉れるのである｡おたまも長も小学校の三年生であった｡――老人の名は芳､夫婦っきりで､三本松の裏に住み､｢大蝶｣の倉庫番をしている､ということであった｡｢大蝶｣はその町でいちばん大きく貝の罐詰工場を経営してい､漁師たちの採る貝を沖で買い取るために､大蝶丸という船を持っていた｡
 私の問いに答えて､長はつよく首を振った｡
｢ううん､そんなこたねえだよ｣と長は云った､｢工場はやかましかんべ､だからみんなえっけえ声になっちまうだ｣
 えっけえとはもちろん大きなという意味である｡長はなお｢芳爺さまはそら耳を使う｣と云ったが､それはもう私の知っていることであった｡
 それからのちもときどき道で会ったが､老人は挨拶もしないし､私を見ても棒杭か石ころでも見るような眼つきしかしなかった｡頬かぶりをとった老人の顔は､痩せていて小さく､太陽と潮風にやけた頭は禿げていて､灰色の髪の毛がほんの少し後頭部にあり､頬や顎にはまばらな無精髭が､古くなったﾌﾞﾗｼのように､一本ずつ数えられるほどまばらに､きらきらと銀色に光っていた｡眼には非人間的な鈍い冷たい光があり､殆んど唇が無いようにみえる薄い唇には､いつも人を小ばかにしたような､狡猾な微笑が刻みつけられていた｡
 尤もこれは芳爺さんに限らず､その土地の一部の人たちに共通した顔だちであった｡かれらは季節ごとに来る遊覧客､――魚釣り､汐干狩り､海水浴など､遊びに来る都会の客たちから｢うまくせしめる｣習慣がついているので､その冷たく鈍い眼や､狡猾そうな口つきの裏には､いつでも朴訥な表情をつくり､あいそ笑いをする用意ができているのであった｡――四月の末か五月のはじめころ､たぶん五月のはじめころであったろう､私は三本松のところで老人に捉まった｡
 三本松といっても､樹齢の古い松ﾉ木が一本しかない｡ずっと昔は三本あったそうであるが､私の聞いた限りでは､それを自分の眼で見たという者はなかった｡――堀の岸に横這いのかたちで枝を伸ばしている｡その松ﾉ木の脇に､水から揚げて久しいべか舟が伏せてあった｡ずいぶんまえからそこにあり､私は通りかかるたびにそれを見た｡べか舟というのは一人乗りの平底舟で､多く貝や海苔採りに使われ､笹の葉のような軽快なかたちをしてい､小さいながら中央に帆桁もあって､小さな三角帆を張ることができた｡しかし､そこに伏せてあったのは胴がふくれていてかたちが悪く､外側が青いﾍﾟﾝｷで塗ってあり､見るからに鈍重で不恰好だった｡
｢あのぶっくれ舟か｣と長が或るとき鼻柱へ皺をよらせ､さも軽蔑に耐えないというように云った､｢青べかってえだよ｣
 この誇り高い小学三年生は､見る気にもなれないという顔つきでそっぽを向いた｡
 それは慥かにぶっくれ舟であった｡伏せてある平底の板は乾いてはしゃぎ､一とところあいている穴から､去年の枯れ草がひょろひょろと伸びていた｡水から揚げられた古い舟ほど､哀れに頼りなげなものはない｡それは老衰して役に立たなくなった馬が､飼主にも忘れられ､厩の裏でひとりしょんぼり首を垂れているような感じにみえる｡――その日も私は道傍に佇んで､人間も同じようなものだ､などというのは俗すぎるな､というようなことを思いながら､暫くﾀﾊﾞｺをふかしていた｡
 そこへ老人が来て話しかけた｡私は気づかなかったが､老人は私のようすを見ていたらしい｡おそらく､私がその舟にすっかり惚れこんだものと思ったのであろう､にこやかな､とりいるような笑顔をつくり､｢この舟を買わねえかね｣とあいそのいい声で喚いた｡
 私は答えることができなかった｡
｢先生はこの土地のことを詳しく見てえって云ってたんべが｣と老人が喚いた､｢そんなら岡の上べえ歩きまわってもしょあんめえじゃ､根戸川のまわりだの百万坪のいりだの､堀もそうだし､沖へも出てみるがいいだ､それにはこの舟さえあれば用が足りるだよ｣
 まあ見てくれと云って､老人は伏せてある青べかをひき起こした｡それは極めてすばやく､声をかける隙もない動作だった｡
｢ほれ見せえま｣と老人は云った､｢まっさらとは云えねえが､造ってからまだ七年にしかなんねえ､大事にしろばまだ十五年や二十年はたっぷり使えるだ｣
 私は自分の考えを述べようとした｡
｢値段もまけるだよ｣と､老人は喚きたてた､｢蒸気河岸の先生のこったからよ､思いきって五までまけるだ､たった五だ｣
 私が答えると､老人は片手を出した｡
｢ﾀﾊﾞｺ｣と老人は云った｡
 私はﾀﾊﾞｺとﾏｯﾁを渡した｡
｢じゃあ､なんだ｣と老人はﾀﾊﾞｺを一本抜いて火をつけ､ﾀﾊﾞｺの箱はふところへ入れ､ﾏｯﾁだけを返しながら喚いた､｢先生のこったから思いきって四にすべえ､四だ｣
 私が答えると､老人はﾀﾊﾞｺを地面でもみ消し､残りを耳にはさみながら喚きたてた｡私は長の顔や､軽蔑しきった口ぶりを思いだしたが､同時に､自分が老人に縛りあげられ､ぬけ出すことのできない罠にかかったことを悟った｡｢見せえま｣と老人は喚き続けた､｢揚げっ放しにしといたからちっとばかはしゃいでるだが､まだこんなにしっかりしてるだ｣
 老人は舟べりや舳先を､大事そうに撫でたり叩いたりした｡私はそれを眺めながら､老人が舟をひき起こすときのすばやい動作には二つの意図があった､ということに気づいた｡一つは私を捉えること､他の一つは去年の枯れ草が覗いていた舟底の穴を私から隠そうとしたのだ､ということである｡――もう一つ､これを書いては人が信じなくなるだろうと思って､書かないことにするつもりであるが､老人が舳先を掴んでゆすぶったとき､舳先の尖ったところが折れてしまった｡すると老人は自分の手にある折れた舳先の､折れたところへ唾をつけて､元の部分と合わせ､そこを片手で押えたまま､いっそう高ごえになって喚きたてるのであった｡事実はこのとおりだったのだが､これを文字にすると､おそらく人は筆者が調子づいてふざけていると思うにちがいない｡｢事実を書く｣ということがいかに困難なしごとであるかは､こんな些細な点でも思い知らされるのである｡
｢よし､そんなら三と五十にすべえ｣と老人は云った､｢これ以上は鐚一文負からねえだ､三と五十､これで話はきまっただ｣
 私はちょっと質問した｡
｢そんなこたあ屁でもねえさ｣と老人は云った､｢いかずちの船大工に頼めばすぐ繕ってくれるだ､いいとも､おらが持ってって頼んでやるだよ｣
｢それから｣と老人はいそいで付け加えた､｢こういう売り買いには､買い手のほうでなにか物を付けるのがしきたりになってるだ､豚肉の百匁でもいいし､夏なら西瓜の三つくれえかな､うう､おめえよく舶来のﾀﾊﾞｺを吸ってるようだが｣
 私は豚肉を届けると答えた｡
 こうして私は｢青べか｣の持ち主になった｡どんなに小さく､そしてぶっくれ舟であるにもせよ､一ぱいの舟の所有者になったのだが､私はうれしくもなかったし､誇りがましい気持にもなれなかった｡長をはじめとする少年たちの軽侮の眼や､嘲笑の声を考えるだけで､むしろ急に肩身のせまくなったような鬱陶しい､沈んだ気分にとらわれたのであった｡
｢いいさ､あんな舟｣と私は帰る道で自分に云った､｢乗らなければいいんだ｣
 私は明くる日､老人のところへ舟の代金と､豚肉を百匁だけ届け､なお青べかについて､二三のことを頼んだ｡老人はこころよく受け合い､そのとおりにすると約束した｡
蜜柑の木
 助なあこ(あにいというほどの意味)はお兼に恋をした｡助なあこは大蝶丸の水夫であり､お兼は｢大蝶｣の罐詰工場へ貝を剥きにかよう雇い女で､亭主があった｡
 この土地で恋といえば､沖の百万坪にある海苔漉き小屋へいって寝ることであった｡そんなてまをかける暇がなければ､裏の空地の枯れ芦の中でもいいし､夏なら根戸川の堤でも､妙見堂の境内でも､消防のﾎﾟﾝﾌﾟ小屋でも用は足りた｡実際のところ､海苔漉き小屋まで寝にゆくのは､よほど二人がのぼせあがっているか､ゆきすぎた声を抑えることのできない女との場合､――土地の人たちのあいだで､そういう癖のある五人の女性の名が公然と話題になっていたが､――などで､かれらの意見によれば､｢そんなにてま暇をかけるほど珍しいことでもあんめえじゃあ｣というのが常識であった｡
 助なあこはそうではなかった｡彼は中学生が女学生を恋するように､純粋に､初心に恋していた｡大蝶丸で沖へ貝を積みにいっているあいだ､彼の胸はつねにお兼を想うことで痛み､その眼にはお兼の姿､――工場の古びた建物の前で､大勢の女や老婆たちと並んで､巧みに貝を剥いている姿が､絶えずあらわれたり消えたりするのであった｡
 大蝶丸の水夫は三人で､船長の荒木さんはべつに家庭を持っていたが､ｴﾝｼﾞさんの正山さんと水夫たちは､工場の中にある小屋に住んでいた｡助なあこは自分の恋を秘し隠しにし､誰にも気どられないように､最高の抑制を保ち続けていたが､或る夜半､ねごとにお兼の名を呼んだのを､隣りに寝ていた二人の水夫に聞かれて､せっかくの努力がむだになってしまった｡
｢ゆんべが初めてじゃねえぞ｣と水夫の一人が云った､｢おんだらあ何遍も聞いているだ､なあ｣
｢おうよ｣と他の水夫が云った､｢名めえをはっきり云ったなあ､ゆんべが初めてだっけ｡ずっとめえから何遍も好きだあ好きだってねごとう云ってたっけだ｣
｢お､か､ね､さん｣と先の水夫が両手で自分の肩を抱きしめ､身もだえしながら作り声で云った､｢おら､おめえが､好きだ､死ぬほど好きだ､よう｣
 助なあこは硬ばった顔でそっぽを向き､手の甲で眼を拭いた｡彼は死んでしまいたいと思った｡もしできることなら､その場で二人を半殺しのめにあわせてやりたかった｡しかし彼は痩せているし､背丈も五尺とちょっとしかない｡他の二人はどちらも彼より肉付きがよく､はるかに力も強かった｡それは沖で貝を積むときや､工場へ戻って積みおろしをするときなどでよくわかっていた｡
 彼は死んでしまいたいと思った｡
 助なあこは固い決心をし､お兼のほうへは眼も向けず､貝を剥いている彼女の前を通るときには､まっすぐに向うを見たままいそぎ足で､殆んど走るように通りぬけた｡彼はやがて機関士になるつもりで､仕事が終ったあとは､ｴｼｼﾞﾝに関する本にしがみついて､熱心に独学を続けていた｡それらの本の大部分は荒木船長に借りたものであるが､中の幾冊かは､――ﾃﾞｨｰｾﾞﾙ･ｴﾝｼﾞﾝに関する本は､自分で東京の神田へいって買ったものであった｡
 彼は夜の十二時まえに寝たことはなかった｡他の水夫やｴﾝｼﾞさんは､毎晩のように飲みにでかけ､帰ってくると｢一厘ばな｣か賽ころ博奕で夜更しをした｡ごったくやの女たちを伴れこんで､わるふざけをしたり､博奕や女のことでとっ組みあいの喧嘩をしたりした｡そういう騒ぎの中で､助なあこは小屋の隅のほうに机を移し､両手で耳を塞いで本を読んだり､ﾉｰﾄを取ったりするのであった｡その十坪ほどの､細長い､箱のような小屋には､燭光の弱い裸の電球が､天床から一つぶらさがっているだけである｡隅のほうへ届く光は極めて微弱だったが､それでも助なあこは本にしがみつき､帳面に眼を押しつけるようにしてﾉｰﾄを取った｡
 周囲の人たちにとって､この独学はばかげたことであった｡そのくらいのｴﾝｼﾞﾅｰになるには､五六年も船に乗って､実地にｴﾝｼﾞさんのすることを見ていれば､それだけで立派にｴﾝｼﾞﾅｰになれるし､現に二つの通船会社のｴﾝｼﾞさんたちでさえ､多くはそのようにして機関士になったのである｡
 お兼のことでからかわれてから､助なあこはすっかり人嫌いになり､ますます独学に熱中した｡ねごとの話はたちまちひろまったが､そのまますぐに忘れられた｡この土地では､どこのかみさんが誰と寝た､などという話は家常茶飯のことで､たとえばおめえのおっかあが誰それと寝たぞと云われたような場合でも､その亭主はべつに驚きもしない､おっかあだってたまにゃあ味の変ったのが欲しかんべえじゃあ､とか､おらのお古でよかったら使うがいいべさ､と云うくらいのものであった｡――もちろんこれら亭主たち自身も｢変った味｣をせしめているのであるし､また､全部の人たちがそんなに脱俗しているというのでもない｡｢浦粕では娘も女房も野放しだ｣と､はっきり土地の人たちは云っているが､それでも嫉妬ぶかい人間もたまにはいて､ときに凄いような騒ぎの起こることも幾たびかあった｡
 助なあこの場合には､ねごとで恋の告白をしたというだけだったから､ほんのお笑いぐさとして忘れられてしまったが､傷ついた助なあことお兼とは､それぞれの立場で忘れることができなかったようだ｡
 初夏の或る午後､二人は根戸川の土堤で初めて話をした｡その日は工場が休みで､助なあこは午めしのあと､本を二冊持って土堤へゆき､若草の伸びた斜面に腰をおろして､本をひらいた｡読んでゆき､頁を繰るが､なんにも頭にはいらない｡活字の列はただ素通りするだけで､一行読むごとにきれいに消えてしまう｡彼は音読もしてみた､一句ずつ指で押えながらやってみたが､やっぱり同じことで､いくら繰返し読んでみても､なに一つ頭に残らないのであった｡
 そこへお兼が来た｡彼女は助なあこのあとを跟けて来たのだ､まえから彼のようすを見ていて､自分のほうからきっかけをつけなければならないと悟り､その日ようやく機会をつかんだのである｡
｢あら､助さんじゃないの｣とお兼はいかにも意外そうに呼びかけた､｢こんなところでなにしてるの､あら､勉強ね｣
 助なあこは本を閉じ､振り向きもせずに､じっと固くなっていた｡彼は全身が火のように熱く､心臓が喉までとびだして来るように感じた｡お兼は斜面へおりて来て､彼と並んで草の上に腰をおろした｡すると､あま酸っぱいような女の躰臭と､白粉の匂いとが入り混った､なまあたたかい空気が彼を包み､彼は頭がくらくらするように思った｡
｢もう春もおしまいだねえ｣お兼はその言葉の品のよさに自分でうっとりとなりながら云った､｢水の流れと人の身はって､はかないもんだわねえ｣
 陽の傾いた空にはうすい靄があって､根戸川の広い水面は波もなく､まるで眠っているように静かだった｡あたためられた土の香や､若草の匂いがあたりに漂ってい､対岸の若い芦の茂みでは､ときどきけたたましく小鳥の騒ぐ声が聞えた｡
｢けけちかしら｣とお兼が云った､｢まだけけちにしては早いかしら｣
 見ると助なあこはふるえていた｡蒼く硬ばった顔を俯向け､膝を抱えた両手の指を揉みしだき､下唇を噛みしめながら､躯ぜんたいでふるえていた｡お兼はふしぎなよろこびを感じた｡これまで一度も感じたことのない､ぞっと総毛立つような､快楽の戦慄が突きぬけるように思った｡
｢あたしあんたが好きよ｣とお兼は彼の耳に囁いた､｢あんた､芳野の海苔漉き小屋､知ってるでしょ､知ってるわね｣
 助なあこは黙って頷いた｡
｢あたしあんたに話したいことがあるの｣とお兼は続けた､｢今夜ね､七時ごろあそこへ来てちょうだい､来てくれる､ねえ｣
 お兼はそっと助なあこの手に触れた｡彼はぴくっとなり､躯をいっそう固くし､そしてお兼の手に伝わるほど激しくふるえた｡お兼はまた､あのふしぎなよろこびの感覚におそわれ､助なあこの手首をぎゅっと握ってから､それを放した｡
｢もうみんなが沖から帰ってくるじぶんだわ｣とお兼は云って溜息をついた､｢みつかると口がうるさいからあたし帰るわ､世の中ってままならないもんね｣
 お兼はもういちど夜の約束をし､鼻唄をうたいながら去っていった｡
 助なあこは時間を計っていて､やがてそっと振り向いてみた｡あまり長いこと同じ姿勢でいたため､首の骨がきくんと鳴り､頸の筋がつった｡お兼はもうずっと遠く､白い煙に包まれている石灰工場の近くまでいっていた｡
｢あんたが好きよ｣助なあこは頸の筋を揉みながら､お兼の云った言葉をまねてみた､｢あたしあんたが好きよ｣
 彼の顔が歪み､眼から涙がこぼれ落ちた｡
 もう春も終りだ､世の中はままならない､あたしあんたが好きよ､水の流れと人の身は､はかないもんね｡それらの言葉が彼の頭の中で､一つ一つはっきりと､この世のものとは思えないほど美しく聞えた｡それは殆んど純金の価値を持ち､純金の光を放つように思えた｡
｢おら一生､忘れねえ｣助なあこはそっと呟いた､｢どんなに年をとっても､死ぬまでも､きっと忘れねえ､きっとだ｣
 美しいものは毀れやすい､毀れやすいからこそ美しい､などと云うつもりはない｡ここには美しいものはないのだ､逆に､美しい感情がもてあそばれ､汚されるのであるが､助なあこの受けた感動だけは美しく､清らかに純粋であった｡
 彼はその夜､約束の時間に約束の場所へいった｡芳野は堀南の釣舟屋であるが､季節には海苔もやるので､弁天社のうしろに漉き小屋と干し場を持っていた｡そこは沖の百万坪のとば口にあり､畑と荒地に囲まれ､隣りの漉き小屋とは二百ﾒｰﾄﾙもはなれていた｡――日の永くなる季節ではあったが､もうすっかり昏れてしまい､あたたかい宵闇のどこかから､みみずの鳴く声が聞えて来た｡お兼はもうそこにいて､暗い小屋の前から彼を呼んだ｡助なあこは膝ががくがくするので､転ばないように用心しながらそっちへいった｡
｢待たせるのね､あんた｣お兼はじれったそうに云った､｢女を待たせるなんて罪よ､にくらしい｣
 お兼は衝動的に助なあこの手を握った｡彼は狼狽して､ぶきようにしりごみをし､握られた手を放そうとしながら､云った､｢なにか話すことがあるって｣
 その声は喉でかすれ､言葉ははっきりしなかった｡お兼は含み笑いをしながら､握った手をもっと引きよせた｡土堤のときよりも強く､白粉と女の匂いが彼を包み､彼は眼がくらみそうになった｡
｢そうよ､大事な話があるの｣とお兼は囁いた､｢中でゆっくり聞いてもらうわ､ね､ここへはいりましょう｣
｢おら､――｣と云って彼は足を踏ん張った｡
｢世話をやかせないで｣
｢それでも､おら｣と彼は口ごもった｡
｢いいから｣とお兼は荒い息をしながら､おどろくほどの力で彼を引きよせた､｢なにもおっかないことするわけじゃないじゃないの､たまには男らしくするもんよ｣
 助なあこの歯ががちがちと鳴った｡
 お兼は彼を小屋の中へ伴れこみ､入口の戸を閉めた｡この種の漉き小屋は､入口の三尺の引戸に南京錠が掛けてある｡しかしその多くはぐらぐらで､鍵の必要はなく､ちょっと引張れば錠前ごと抜けてしまい､出てゆくときには元のように挿し込んでおけばいいのであった｡
｢あんた､まだふるえているの｣小屋の中からお兼の声が聞えた､｢さあ､そんなにしてちゃ窮屈じゃないの､この手をこうしなってば｣
 ついで彼女の含み笑いが聞えた｡
｢助さん｣とあまえた鼻声でお兼が云った､｢あんた幾つ､――そう､十九なの､若いのね､うれしい｣
 お兼はそのとき三十五歳であった｡亭主のしっつぁんは呑んだくれの怠け者で､ときたま思いだしたように､なにかの雇われ仕事にでかけるが､｢まる一日働いたことがねえ｣といわれていた｡博奕を打つでもなく､女にちょっかいを出すわけでもない｡ただ酒を飲んで寝ころがるか､ぶらぶら歩きまわってむだ話をするだけである｡云うまでもないだろうが､家計は稼ぎ手のお兼がにぎっていて､しっつぁんは与えられる小遣いでやっているのだが､そんなものが長くある筈はなく､彼はもっぱら奢ってくれそうな相手を求めてぶらつき､またしばしばお兼の男のところへいってねだった｡
 お兼は子を産まないためか､肌の艶もよく､浮気性の女に共通の嬌めかしさ､誘惑的な声と身ぶり､言葉よりずっと明確に意志を伝える眼つき､などをもっていた｡それは洗練されたものではな..."""
    },
 


     {
    "@search.action": "upload",
    "BookId": "57192",
    "BookName": "白昼夢",
    "BookAuthor": "江戸川乱歩",
    "BookGenre": "掌編小説",
    "Release_Date": "1925年7月",
    "Rating": "3.20",
    "Text": """あれは､白昼の悪夢であったか､それとも現実の出来事であったか｡
 晩春の生暖い風が､ｵﾄﾞﾛｵﾄﾞﾛと､火照った頬に感ぜられる､蒸し暑い日の午後であった｡
 用事があって通ったのか､散歩のみちすがらであったのか､それさえぼんやりとして思い出せぬけれど､私は､ある場末の､見る限り何処までも何処までも､真直に続いている､広い埃っぽい大通りを歩いていた｡
 洗いざらした単衣物の様に白茶けた商家が､黙って軒を並べていた｡三尺のｼｮｰｳｲﾝﾄﾞｳに､埃でだんだら染めにした小学生の運動ｼｬﾂが下っていたり､碁盤の様に仕切った薄っぺらな木箱の中に､赤や黄や白や茶色などの､砂の様な種物を入れたのが､店一杯に並んでいたり､狭い薄暗い家中が､天井からどこから､自転車のﾌﾚｰﾑやﾀｲﾔで充満していたり､そして､それらの殺風景な家々の間に挟まって､細い格子戸の奥にすすけた御神燈の下った二階家が､そんなに両方から押しつけちゃ厭だわという恰好をして､ﾎﾞﾛﾝﾎﾞﾛﾝと猥褻な三味線の音を洩していたりした｡
｢ｱｯﾌﾟｸ､ﾁｷﾘｷ､ｱｯﾊﾟｯﾊﾟｱ……ｱｯﾊﾟｯﾊﾟｱ……｣
 お下げを埃でお化粧した女の子達が､道の真中に輪を作って歌っていた｡ｱｯﾊﾟｯﾊﾟｱｱｱｱ……という涙ぐましい旋律が､霞んだ春の空へのんびりと蒸発して行った｡
 男の子等は繩飛びをして遊んでいた｡長い繩の弦が､ねばり強く地を叩いては､空に上った｡田舎縞の前をはだけた一人の子が､ﾋﾟｮｲﾋﾟｮｲと飛んでいた｡その光景は､高速度撮影機を使った活動写真の様に､如何にも悠長に見えた｡
 時々､重い荷馬車がｺﾞﾛｺﾞﾛと道路や､家々を震動させて私を追い越した｡
 ふと私は､行手に当って何かが起っているのを知った｡十四五人の大人や子供が､道ばたに不規則な半円を描いて立止っていた｡
 それらの人々の顔には､皆一種の笑いが浮んでいた｡喜劇を見ている人の笑いが浮んでいた｡ある者は大口を開いてｹﾞﾗｹﾞﾗ笑っていた｡
 好奇心が､私をそこへ近付かせた｡
 近付に従って､大勢の笑顔と際立った対照を示している一つの真面目くさった顔を発見した｡その青ざめた顔は､口をとがらせて､何事か熱心に弁じ立てていた｡香具師の口上にしては余りに熱心過ぎた｡宗教家の辻説法にしては見物の態度が不謹慎だった｡一体､これは何事が始まっているのだ｡
 私は知らず知らず半円の群集に混って､聴聞者の一人となっていた｡
 演説者は､青っぽいくすんだ色のｾﾙに､黄色の角帯をｷﾁﾝと締めた､風采のよい､見た所相当教養もありそうな四十男であった｡鬘の様に綺麗に光らせた頭髪の下に､中高の薤形の青ざめた顔､細い眼､立派な口髭で隈どった真赤な脣､その脣が不作法につばきを飛ばしてﾊﾞｸﾊﾞｸ動いているのだ｡汗をかいた高い鼻､そして､着物の裾からは､砂埃にまみれた跣足の足が覗いていた｡
｢……俺はどんなに俺の女房を愛していたか｣
 演説は今や高調に達しているらしく見えた｡男は無量の感慨を罩めてこういったまま､暫く見物達の顔から顔を見廻していたが､やがて､自問に答える様に続けた｡
｢殺す程愛していたのだ!｣
｢……悲しい哉､あの女は浮気者だった｣
 ﾄﾞｯと見物の間に笑い声が起ったので､其次の｢いつ余所の男とくｯつくかも知れなかった｣という言葉は危く聞き洩す所だった｡
｢いや､もうとっくにくｯついていたかも知れないのだ｣
 そこで又､前にもました高笑いが起った｡
｢俺は心配で心配で｣彼はそういって歌舞伎役者の様に首を振って｢商売も手につかなんだ｡俺は毎晩寝床の中で女房に頼んだ｡手を合せて頼んだ｣笑声｢どうか誓って呉れ｡俺より外の男には心を移さないと誓って呉れ……併し､あの女はどうしても私の頼みを聞いては呉れない｡まるで商売人の様な巧みな嬌態で､手練手管で､その場その場をごまかすばかりです｡だが､それが､その手練手管が､どんなに私を惹きつけたか……｣
 誰かが｢ようよう､御馳走さまｯ｣と叫んだ｡そして､笑声｡
｢みなさん｣男はそんな半畳などを無視して続けた｡｢あなた方が､若し私の境遇にあったら一体どうしますか｡これが殺さないでいられましょうか!｣
｢……あの女は耳隠しがよく似合いました｡自分で上手に結うのです……鏡台の前に坐っていました｡結い上げた所です｡綺麗にお化粧した顔が私の方をふり向いて､赤い脣でﾆｯｺﾘ笑いました｣
 男はここで一つ肩を揺り上げて見えを切った｡濃い眉が両方から迫って凄い表情に変った｡赤い脣が気味悪くﾋﾝ曲った｡
｢……俺は今だと思った｡この好もしい姿を永久に俺のものにして了うのは今だと思った｣
｢用意していた千枚通しを､あの女の匂やかな襟足へ､力まかせにたたき込んだ｡笑顔の消えぬうちに､大きい糸切歯が脣から覗いたまんま……死んで了った｣
 賑かな広告の楽隊が通り過ぎた｡大喇叭が頓狂な音を出した｡｢ここはお国を何百里､離れて遠き満洲の｣子供等が節に合せて歌いながら､ｿﾞﾛｿﾞﾛとついて行った｡
｢諸君､あれは俺のことを触廻っているのだ｡真柄太郎は人殺しだ､人殺しだ､そういって触廻っているのだ｣
 又笑い声が起った｡楽隊の太鼓の音丈けが､男の演説の伴奏ででもある様に､いつまでもいつまでも聞えていた｡
｢……俺は女房の死骸を五つに切り離した｡いいかね､胴が一つ､手が二本､足が二本､これでつまり五つだ｡……惜しかったけれど仕方がない｡……よく肥ったまっ白な足だ｣
｢……あなた方はあの水の音を聞かなかったですか｣男は俄に声を低めて云った｡首を前につき出し､目をｷｮﾛｷｮﾛさせながら､さも一大事を打開けるのだといわぬばかりに､｢三七二十一日の間､私の家の水道はｻﾞｰｻﾞｰと開けっぱなしにしてあったのですよ｡五つに切った女房の死体をね､四斗樽の中へ入れて､冷していたのですよ｡これがね､みなさん｣ここで彼の声は聞えない位に低められた｡
｢秘訣なんだよ｡秘訣なんだよ｡死骸を腐らせない｡……屍蝋というものになるんだ｣
｢屍蝋｣……ある医書の｢屍蝋｣の項が､私の目の前に､その著者の黴くさい絵姿と共に浮んで来た｡一体全体､この男は何を云わんとしているのだ｡何とも知れぬ恐怖が､私の心臓を風船玉の様に軽くした｡
｢……女房の脂ぎった白い胴体や手足が､可愛い蝋細工になって了った｣
｢ﾊﾊﾊﾊﾊ､お極りを云ってらあ｡お前それを､昨日から何度おさらいするんだい｣誰かが不作法に怒鳴った｡
｢ｵｲ､諸君｣男の調子がいきなり大声に変った｡｢俺がこれ程云うのが分らんのか｡君達は､俺の女房は家出をした家出をしたと信じ切っているだろう｡ところがな､ｵｲ､よく聞け､あの女はこの俺が殺したんだよ｡どうだ､びっくりしたか｡ﾜﾊﾊﾊﾊﾊﾊ｣
 ……断切った様に笑声がやんだかと思うと､一瞬間に元の生真面目な顔が戻って来た｡男は又囁き声で始めた｡
｢それでもう､女はほんとうに私のものになり切って了ったのです｡ちっとも心配はいらないのです｡ｷｯｽのしたい時にｷｯｽが出来ます｡抱き締めたい時には抱きしめることも出来ます｡私はもう､これで本望ですよ｣
｢……だがね､用心しないと危い｡私は人殺しなんだからね｡いつ巡査に見つかるかしれない｡そこで､俺はうまいことを考えてあったのだよ｡隠し場所をね｡……巡査だろうが刑事だろうが､こいつにはお気がつくまい｡ﾎﾗ､君､見てごらん｡その死骸はちゃんと俺の店先に飾ってあるのだよ｣
 男の目が私を見た｡私はﾊｯとして後を振り向いた｡今の今まで気のつかなかったすぐ鼻の先に､白いｽﾞｯｸの日覆……｢ﾄﾞﾗｯｸﾞ｣……｢請合薬｣……見覚えのある丸ｺﾞｼｯｸの書体､そして､その奥のｶﾞﾗｽ張りの中の人体模型､その男は､何々ﾄﾞﾗｯｸﾞという商号を持った､薬屋の主人であった｡
｢ね､いるでしょう｡もっとよく私の可愛い女を見てやって下さい｣
 何がそうさせたのか｡私はいつの間にか日覆の中へ這入っていた｡
 私の目の前のｶﾞﾗｽ箱の中に女の顔があった｡彼女は糸切歯をむき出してﾆｯｺﾘ笑っていた｡いまわしい蝋細工の腫物の奥に､真実の人間の皮膚が黒ずんで見えた｡作り物でない証拠には､一面にうぶ毛が生えていた｡
 ｽｰｯと心臓が喉の所へ飛び上った｡私は倒れ相になる身体を､危くささえて日覆からのがれ出した｡そして､男に見つからない様に注意しながら､群集の側を離れた｡
 ……ふり返って見ると､群集のうしろに一人の警官が立っていた｡彼も亦､他の人達と同じ様にﾆｺﾆｺ笑いながら､男の演説を聞いていた｡
｢何を笑っているのです｡君は職務の手前それでいいのですか｡あの男のいっていることが分りませんか｡嘘だと思うならあの日覆の中へ這入って御覧なさい｡東京の町の真中で､人間の死骸がさらしものになっているじゃありませんか｣
 無神経な警官の肩を叩いて､こう告げてやろうかと思った｡けれど私にはそれを実行する丈けの気力がなかった｡私は眩暈を感じながらﾋｮﾛﾋｮﾛと歩き出した｡
 行手には､どこまでもどこまでも果しのない白い大道が続いていた｡陽炎が､立並ぶ電柱を海草の様に揺っていた｡"""
    },



     

     
{
    "@search.action": "upload",
    "BookId": "42620",
    "BookName": "堕落論",
    "BookAuthor": "坂口安吾",
    "BookGenre": "掌編小説",
    "Release_Date": "1946年4月",
    "Rating": "3.91",
    "Text": """半年のうちに世相は変った｡醜の御楯といでたつ我は｡大君のへにこそ死なめかへりみはせじ｡若者達は花と散ったが､同じ彼等が生き残って闇屋となる｡ももとせの命ねがはじいつの日か御楯とゆかん君とちぎりて｡けなげな心情で男を送った女達も半年の月日のうちに夫君の位牌にぬかずくことも事務的になるばかりであろうし､やがて新たな面影を胸に宿すのも遠い日のことではない｡人間が変ったのではない｡人間は元来そういうものであり､変ったのは世相の上皮だけのことだ｡
 昔､四十七士の助命を排して処刑を断行した理由の一つは､彼等が生きながらえて生き恥をさらし折角の名を汚す者が現れてはいけないという老婆心であったそうな｡現代の法律にこんな人情は存在しない｡けれども人の心情には多分にこの傾向が残っており､美しいものを美しいままで終らせたいということは一般的な心情の一つのようだ｡十数年前だかに童貞処女のまま愛の一生を終らせようと大磯のどこかで心中した学生と娘があったが世人の同情は大きかったし､私自身も､数年前に私と極めて親しかった姪の一人が二十一の年に自殺したとき､美しいうちに死んでくれて良かったような気がした｡一見清楚な娘であったが､壊れそうな危なさがあり真逆様に地獄へ堕ちる不安を感じさせるところがあって､その一生を正視するに堪えないような気がしていたからであった｡
 この戦争中､文士は未亡人の恋愛を書くことを禁じられていた｡戦争未亡人を挑発堕落させてはいけないという軍人政治家の魂胆で彼女達に使徒の余生を送らせようと欲していたのであろう｡軍人達の悪徳に対する理解力は敏感であって､彼等は女心の変り易さを知らなかったわけではなく､知りすぎていたので､こういう禁止項目を案出に及んだまでであった｡
 いったいが日本の武人は古来婦女子の心情を知らないと言われているが､之は皮相の見解で､彼等の案出した武士道という武骨千万な法則は人間の弱点に対する防壁がその最大の意味であった｡
 武士は仇討のために草の根を分け乞食となっても足跡を追いまくらねばならないというのであるが､真に復讐の情熱をもって仇敵の足跡を追いつめた忠臣孝子があったであろうか｡彼等の知っていたのは仇討の法則と法則に規定された名誉だけで､元来日本人は最も憎悪心の少い又永続しない国民であり､昨日の敵は今日の友という楽天性が実際の偽らぬ心情であろう｡昨日の敵と妥協否肝胆相照すのは日常茶飯事であり､仇敵なるが故に一そう肝胆相照らし､忽ち二君に仕えたがるし､昨日の敵にも仕えたがる｡生きて捕虜の恥を受けるべからず､というが､こういう規定がないと日本人を戦闘にかりたてるのは不可能なので､我々は規約に従順であるが､我々の偽らぬ心情は規約と逆なものである｡日本戦史は武士道の戦史よりも権謀術数の戦史であり､歴史の証明にまつよりも自我の本心を見つめることによって歴史のｶﾗｸﾘを知り得るであろう｡今日の軍人政治家が未亡人の恋愛に就いて執筆を禁じた如く､古の武人は武士道によって自らの又部下達の弱点を抑える必要があった｡
 小林秀雄は政治家のﾀｲﾌﾟを､独創をもたずただ管理し支配する人種と称しているが､必ずしもそうではないようだ｡政治家の大多数は常にそうであるけれども､少数の天才は管理や支配の方法に独創をもち､それが凡庸な政治家の規範となって個々の時代､個々の政治を貫く一つの歴史の形で巨大な生き者の意志を示している｡政治の場合に於て､歴史は個をつなぎ合せたものでなく､個を没入せしめた別個の巨大な生物となって誕生し､歴史の姿に於て政治も亦巨大な独創を行っているのである｡この戦争をやった者は誰であるか､東条であり軍部であるか｡そうでもあるが､然し又､日本を貫く巨大な生物､歴史のぬきさしならぬ意志であったに相違ない｡日本人は歴史の前ではただ運命に従順な子供であったにすぎない｡政治家によし独創はなくとも､政治は歴史の姿に於て独創をもち､意慾をもち､やむべからざる歩調をもって大海の波の如くに歩いて行く｡何人が武士道を案出したか｡之も亦歴史の独創､又は嗅覚であったであろう｡歴史は常に人間を嗅ぎだしている｡そして武士道は人性や本能に対する禁止条項である為に非人間的反人性的なものであるが､その人性や本能に対する洞察の結果である点に於ては全く人間的なものである｡
 私は天皇制に就ても､極めて日本的な(従って或いは独創的な)政治的作品を見るのである｡天皇制は天皇によって生みだされたものではない｡天皇は時に自ら陰謀を起したこともあるけれども､概して何もしておらず､その陰謀は常に成功のためしがなく､島流しとなったり､山奥へ逃げたり､そして結局常に政治的理由によってその存立を認められてきた｡社会的に忘れた時にすら政治的に担ぎだされてくるのであって､その存立の政治的理由はいわば政治家達の嗅覚によるもので､彼等は日本人の性癖を洞察し､その性癖の中に天皇制を発見していた｡それは天皇家に限るものではない｡代り得るものならば､孔子家でも釈迦家でもﾚｰﾆﾝ家でも構わなかった｡ただ代り得なかっただけである｡
 すくなくとも日本の政治家達(貴族や武士)は自己の永遠の隆盛(それは永遠ではなかったが､彼等は永遠を夢みたであろう)を約束する手段として絶対君主の必要を嗅ぎつけていた｡平安時代の藤原氏は天皇の擁立を自分勝手にやりながら､自分が天皇の下位であるのを疑りもしなかったし､迷惑にも思っていなかった｡天皇の存在によって御家騒動の処理をやり､弟は兄をやりこめ､兄は父をやっつける｡彼等は本能的な実質主義者であり､自分の一生が愉しければ良かったし､そのくせ朝儀を盛大にして天皇を拝賀する奇妙な形式が大好きで､満足していた｡天皇を拝むことが､自分自身の威厳を示し､又､自ら威厳を感じる手段でもあったのである｡
 我々にとっては実際馬鹿げたことだ｡我々は靖国神社の下を電車が曲るたびに頭を下げさせられる馬鹿らしさには閉口したが､或種の人々にとっては､そうすることによってしか自分を感じることが出来ないので､我々は靖国神社に就てはその馬鹿らしさを笑うけれども､外の事柄に就て､同じような馬鹿げたことを自分自身でやっている｡そして自分の馬鹿らしさには気づかないだけのことだ｡宮本武蔵は一乗寺下り松の果し場へ急ぐ途中､八幡様の前を通りかかって思わず拝みかけて思いとどまったというが､吾神仏をたのまずという彼の教訓は､この自らの性癖に発し､又向けられた悔恨深い言葉であり､我々は自発的にはずいぶん馬鹿げたものを拝み､ただそれを意識しないというだけのことだ｡道学先生は教壇で先ず書物をおしいただくが､彼はそのことに自分の威厳と自分自身の存在すらも感じているのであろう｡そして我々も何かにつけて似たことをやっている｡
 日本人の如く権謀術数を事とする国民には権謀術数のためにも大義名分のためにも天皇が必要で､個々の政治家は必ずしもその必要を感じていなくとも､歴史的な嗅覚に於て彼等はその必要を感じるよりも自らの居る現実を疑ることがなかったのだ｡秀吉は聚楽に行幸を仰いで自ら盛儀に泣いていたが､自分の威厳をそれによって感じると同時に､宇宙の神をそこに見ていた｡これは秀吉の場合であって､他の政治家の場合ではないが､権謀術数がたとえば悪魔の手段にしても､悪魔が幼児の如くに神を拝むことも必ずしも不思議ではない｡どのような矛盾も有り得るのである｡
 要するに天皇制というものも武士道と同種のもので､女心は変り易いから｢節婦は二夫に見えず｣という､禁止自体は非人間的､反人性的であるけれども､洞察の真理に於て人間的であることと同様に､天皇制自体は真理ではなく､又自然でもないが､そこに至る歴史的な発見や洞察に於て軽々しく否定しがたい深刻な意味を含んでおり､ただ表面的な真理や自然法則だけでは割り切れない｡
 まったく美しいものを美しいままで終らせたいなどと希うことは小さな人情で､私の姪の場合にしたところで､自殺などせず生きぬきそして地獄に堕ちて暗黒の曠野をさまようことを希うべきであるかも知れぬ｡現に私自身が自分に課した文学の道とはかかる曠野の流浪であるが､それにも拘らず美しいものを美しいままで終らせたいという小さな希いを消し去るわけにも行かぬ｡未完の美は美ではない｡その当然堕ちるべき地獄での遍歴に淪落自体が美でありうる時に始めて美とよびうるのかも知れないが､二十の処女をわざわざ六十の老醜の姿の上で常に見つめなければならぬのか｡これは私には分らない｡私は二十の美女を好む｡
 死んでしまえば身も蓋もないというが､果してどういうものであろうか｡敗戦して､結局気の毒なのは戦歿した英霊達だ､という考え方も私は素直に肯定することができない｡けれども､六十すぎた将軍達が尚生に恋々として法廷にひかれることを思うと､何が人生の魅力であるか､私には皆目分らず､然し恐らく私自身も､もしも私が六十の将軍であったなら矢張り生に恋々として法廷にひかれるであろうと想像せざるを得ないので､私は生という奇怪な力にただ茫然たるばかりである｡私は二十の美女を好むが､老将軍も亦二十の美女を好んでいるのか｡そして戦歿の英霊が気の毒なのも二十の美女を好む意味に於てであるか｡そのように姿の明確なものなら､私は安心することもできるし､そこから一途に二十の美女を追っかける信念すらも持ちうるのだが､生きることは､もっとわけの分らぬものだ｡
 私は血を見ることが非常に嫌いで､いつか私の眼前で自動車が衝突したとき､私はｸﾙﾘと振向いて逃げだしていた｡けれども､私は偉大な破壊が好きであった｡私は爆弾や焼夷弾に戦きながら､狂暴な破壊に劇しく亢奮していたが､それにも拘らず､このときほど人間を愛しなつかしんでいた時はないような思いがする｡
 私は疎開をすすめ又すすんで田舎の住宅を提供しようと申出てくれた数人の親切をしりぞけて東京にふみとどまっていた｡大井広介の焼跡の防空壕を､最後の拠点にするつもりで､そして九州へ疎開する大井広介と別れたときは東京からあらゆる友達を失った時でもあったが､やがて米軍が上陸し四辺に重砲弾の炸裂するさなかにその防空壕に息をひそめている私自身を想像して､私はその運命を甘受し待ち構える気持になっていたのである｡私は死ぬかも知れぬと思っていたが､より多く生きることを確信していたに相違ない｡然し廃墟に生き残り､何か抱負を持っていたかと云えば､私はただ生き残ること以外の何の目算もなかったのだ｡予想し得ぬ新世界への不思議な再生｡その好奇心は私の一生の最も新鮮なものであり､その奇怪な鮮度に対する代償としても東京にとどまることを賭ける必要があるという奇妙な呪文に憑かれていたというだけであった｡そのくせ私は臆病で､昭和二十年の四月四日という日､私は始めて四周に二時間にわたる爆撃を経験したのだが､頭上の照明弾で昼のように明るくなった､そのとき丁度上京していた次兄が防空壕の中から焼夷弾かと訊いた､いや照明弾が落ちてくるのだと答えようとした私は一応腹に力を入れた上でないと声が全然でないという状態を知った｡又､当時日本映画社の嘱託だった私は銀座が爆撃された直後､編隊の来襲を銀座の日映の屋上で迎えたが､五階の建物の上に塔があり､この上に三台のｶﾒﾗが据えてある｡空襲警報になると路上､窓､屋上､銀座からあらゆる人の姿が消え､屋上の高射砲陣地すらも掩壕に隠れて人影はなく､ただ天地に露出する人の姿は日映屋上の十名程の一団のみであった｡先ず石川島に焼夷弾の雨がふり､次の編隊が真上へくる｡私は足の力が抜け去ることを意識した｡煙草をくわえてｶﾒﾗを編隊に向けている憎々しいほど落着いたｶﾒﾗﾏﾝの姿に驚嘆したのであった｡
 けれども私は偉大な破壊を愛していた｡運命に従順な人間の姿は奇妙に美しいものである｡麹町のあらゆる大邸宅が嘘のように消え失せて余燼をたてており､上品な父と娘がたった一つの赤皮のﾄﾗﾝｸをはさんで濠端の緑草の上に坐っている｡片側に余燼をあげる茫々たる廃墟がなければ､平和なﾋﾟｸﾆｯｸと全く変るところがない｡ここも消え失せて茫々ただ余燼をたてている道玄坂では､坂の中途にどうやら爆撃のものではなく自動車にひき殺されたと思われる死体が倒れており､一枚のﾄﾀﾝがかぶせてある｡かたわらに銃剣の兵隊が立っていた｡行く者､帰る者､罹災者達のｴﾝｴﾝたる流れがまことにただ無心の流れの如くに死体をすりぬけて行き交い､路上の鮮血にも気づく者すら居らず､たまさか気づく者があっても､捨てられた紙屑を見るほどの関心しか示さない｡米人達は終戦直後の日本人は虚脱し放心していると言ったが､爆撃直後の罹災者達の行進は虚脱や放心と種類の違った驚くべき充満と重量をもつ無心であり､素直な運命の子供であった｡笑っているのは常に十五六､十六七の娘達であった｡彼女達の笑顔は爽やかだった｡焼跡をほじくりかえして焼けたﾊﾞｹﾂへ掘りだした瀬戸物を入れていたり､わずかばかりの荷物の張番をして路上に日向ぼっこをしていたり､この年頃の娘達は未来の夢でいっぱいで現実などは苦にならないのであろうか､それとも高い虚栄心のためであろうか｡私は焼野原に娘達の笑顔を探すのがたのしみであった｡
 あの偉大な破壊の下では､運命はあったが､堕落はなかった｡無心であったが､充満していた｡猛火をくぐって逃げのびてきた人達は､燃えかけている家のそばに群がって寒さの煖をとっており､同じ火に必死に消火につとめている人々から一尺離れているだけで全然別の世界にいるのであった｡偉大な破壊､その驚くべき愛情｡偉大な運命､その驚くべき愛情｡それに比べれば､敗戦の表情はただの堕落にすぎない｡
 だが､堕落ということの驚くべき平凡さや平凡な当然さに比べると､あのすさまじい偉大な破壊の愛情や運命に従順な人間達の美しさも､泡沫のような虚しい幻影にすぎないという気持がする｡
 徳川幕府の思想は四十七士を殺すことによって永遠の義士たらしめようとしたのだが､四十七名の堕落のみは防ぎ得たにしたところで､人間自体が常に義士から凡俗へ又地獄へ転落しつづけていることを防ぎうるよしもない｡節婦は二夫に見えず､忠臣は二君に仕えず､と規約を制定してみても人間の転落は防ぎ得ず､よしんば処女を刺し殺してその純潔を保たしめることに成功しても､堕落の平凡な跫音､ただ打ちよせる波のようなその当然な跫音に気づくとき､人為の卑小さ､人為によって保ち得た処女の純潔の卑小さなどは泡沫の如き虚しい幻像にすぎないことを見出さずにいられない｡
 特攻隊の勇士はただ幻影であるにすぎず､人間の歴史は闇屋となるところから始まるのではないのか｡未亡人が使徒たることも幻影にすぎず､新たな面影を宿すところから人間の歴史が始まるのではないのか｡そして或は天皇もただ幻影であるにすぎず､ただの人間になるところから真実の天皇の歴史が始まるのかも知れない｡
 歴史という生き物の巨大さと同様に人間自体も驚くほど巨大だ｡生きるという事は実に唯一の不思議である｡六十七十の将軍達が切腹もせず轡を並べて法廷にひかれるなどとは終戦によって発見された壮観な人間図であり､日本は負け､そして武士道は亡びたが､堕落という真実の母胎によって始めて人間が誕生したのだ｡生きよ堕ちよ､その正当な手順の外に､真に人間を救い得る便利な近道が有りうるだろうか｡私はﾊﾗｷﾘを好まない｡昔､松永弾正という老獪陰鬱な陰謀家は信長に追いつめられて仕方なく城を枕に討死したが､死ぬ直前に毎日の習慣通り延命の灸をすえ､それから鉄砲を顔に押し当て顔を打ち砕いて死んだ｡そのときは七十をすぎていたが､人前で平気で女と戯れる悪どい男であった｡この男の死に方には同感するが､私はﾊﾗｷﾘは好きではない｡
 私は戦きながら､然し､惚れ惚れとその美しさに見とれていたのだ｡私は考える必要がなかった｡そこには美しいものがあるばかりで､人間がなかったからだ｡実際､泥棒すらもいなかった｡近頃の東京は暗いというが､戦争中は真の闇で､そのくせどんな深夜でもｵｲﾊｷﾞなどの心配はなく､暗闇の深夜を歩き､戸締なしで眠っていたのだ｡戦争中の日本は嘘のような理想郷で､ただ虚しい美しさが咲きあふれていた｡それは人間の真実の美しさではない｡そしてもし我々が考えることを忘れるなら､これほど気楽なそして壮観な見世物はないだろう｡たとえ爆弾の絶えざる恐怖があるにしても､考えることがない限り､人は常に気楽であり､ただ惚れ惚れと見とれておれば良かったのだ｡私は一人の馬鹿であった｡最も無邪気に戦争と遊び戯れていた｡
 終戦後､我々はあらゆる自由を許されたが､人はあらゆる自由を許されたとき､自らの不可解な限定とその不自由さに気づくであろう｡人間は永遠に自由では有り得ない｡なぜなら人間は生きており､又死なねばならず､そして人間は考えるからだ｡政治上の改革は一日にして行われるが､人間の変化はそうは行かない｡遠くｷﾞﾘｼｬに発見され確立の一歩を踏みだした人性が､今日､どれほどの変化を示しているであろうか｡
 人間｡戦争がどんなすさまじい破壊と運命をもって向うにしても人間自体をどう為しうるものでもない｡戦争は終った｡特攻隊の勇士はすでに闇屋となり､未亡人はすでに新たな面影によって胸をふくらませているではないか｡人間は変りはしない｡ただ人間へ戻ってきたのだ｡人間は堕落する｡義士も聖女も堕落する｡それを防ぐことはできないし､防ぐことによって人を救うことはできない｡人間は生き､人間は堕ちる｡そのこと以外の中に人間を救う便利な近道はない｡
 戦争に負けたから堕ちるのではないのだ｡人間だから堕ちるのであり､生きているから堕ちるだけだ｡だが人間は永遠に堕ちぬくことはできないだろう｡なぜなら人間の心は苦難に対して鋼鉄の如くでは有り得ない｡人間は可憐であり脆弱であり､それ故愚かなものであるが､堕ちぬくためには弱すぎる｡人間は結局処女を刺殺せずにはいられず､武士道をあみださずにはいられず､天皇を担ぎださずにはいられなくなるであろう｡だが他人の処女でなしに自分自身の処女を刺殺し､自分自身の武士道､自分自身の天皇をあみだすためには､人は正しく堕ちる道を堕ちきることが必要なのだ｡そして人の如くに日本も亦堕ちることが必要であろう｡堕ちる道を堕ちきることによって､自分自身を発見し､救わなければならない｡政治による救いなどは上皮だけの愚にもつかない物である｡"""
    },








    {
    "@search.action": "upload",
    "BookId": "52409",
    "BookName": "三国志",
    "BookAuthor": "吉川英治",
    "BookGenre": "長編小説",
    "Release_Date": "1939年1月",
    "Text": """三国志は､いうまでもなく､今から約千八百年前の古典であるが､三国志の中に活躍している登場人物は､現在でも中国大陸の至る所にそのまま居るような気がする｡――中国大陸へ行って､そこの雑多な庶民や要人などに接し､特に親しんでみると､三国志の中に出て来る人物の誰かしらときっと似ている｡或いは､共通したものを感じる場合がしばしばある｡
 だから､現代の中国大陸には､三国志時代の治乱興亡がそのままあるし､作中の人物も､文化や姿こそ変っているが､なお､今日にも生きているといっても過言でない｡
        
 三国志には､詩がある｡
 単に尨大な治乱興亡を記述した戦記軍談の類でない所に､東洋人の血を大きく搏つ一種の諧調と音楽と色彩とがある｡
 三国志から詩を除いてしまったら､世界的といわれる大構想の価値もよほど無味乾燥なものになろう｡
 故に､三国志は､強いて簡略にしたり抄訳したものでは､大事な詩味も逸してしまうし､もっと重要な人の胸底を搏つものを失くしてしまうおそれがある｡
 で私は､簡訳や抄略をあえてせずに､長篇執筆に適当な新聞小説にこれを試みた｡そして劉玄徳とか､曹操とか関羽､張飛そのほか､主要人物などには､自分の解釈や創意をも加えて書いた｡随所､原本にない辞句､会話なども､わたくしの点描である｡
        
 いうまでもなく三国志は､中国の歴史に取材しているが､正史ではない｡けれど史中の人物を巧妙自在に拉して活躍させ､後漢の第十二代霊帝の代(わが朝の成務天皇の御世､西暦百六十八年頃)から､武帝が呉を亡ぼす太康元年までのおよそ百十二年間の長期にわたる治乱が書いてある｡構想の雄大と､舞台の地域の広さは､世界の古典小説中でも比類ないものといわれている｡登場人物なども､審らかに数えたなら何千何万人にものぼるであろう｡しかも､これに加うるに中国一流の華麗豪壮な調と､哀婉切々の情､悲歌慷慨の辞句と､誇張幽幻な趣と､拍案三嘆の熱とを以て縷述されてあるので､読む者をして百年の地上に明滅する種々雑多な人間の浮沈と文化の興亡とを､一巻に偲ばせて､転深思の感慨に耽らしめる魅力がある｡
        
 見方によれば三国志は､一つの民俗小説ともいえる｡三国志の中に見られる人間の愛欲､道徳､宗教､その生活､また､主題たる戦争行為だとか群雄割拠の状などは､さながら彩られた彼の民俗絵巻でもあり､その生々動流する相は､天地間を舞台として､壮大なる音楽に伴って演技された人類の大演劇とも観られるのである｡
        
 現在の地名と､原本の誌す地名とは､当然時代による異いがあるので､分っている地方は下に註を加えておいた｡分らない旧名もかなりある｡また､登場人物の爵位官職など､ほぼ文字で推察のつきそうなのはそのまま用いた｡あまりに現代語化しすぎると､その文字の持っている特有な色彩や感覚を失ってしまうからである｡
        
 原本には｢通俗三国志｣｢三国志演義｣その他数種あるが､私はそのいずれの直訳にもよらないで､随時､長所を択って､わたくし流に書いた｡これを書きながら思い出されるのは､少年の頃､久保天随氏の演義三国志を熱読して､三更四更まで燈下にしがみついていては､父に寝ろ寝ろといって叱られたことである｡本来､三国志の真味を酌むにはこの原書を読むに如くはないのであるが､今日の読者にその難渋は耐え得ぬことだし､また､一般の求める目的も意義も､大いに異うはずなので､あえて書肆の希望にまかせて再訂上梓することにした｡"""
    },



{
"@search.action": "upload",
"BookId": "1567",
"BookName": "走れﾒﾛｽ",
"BookAuthor": "太宰治",
"BookGenre": "短編小説",
"Release_Date": "1940年6月",
"Rating": "3.89",
"Text": """ﾒﾛｽは激怒した｡
必ず､かの邪智暴虐の王を除かなければならぬと決意した｡ﾒﾛｽには政治がわからぬ｡ﾒﾛｽは､村の牧人である｡笛を吹き､羊と遊んで暮して来た｡けれども邪悪に対しては､人一倍に敏感であった｡きょう未明ﾒﾛｽは村を出発し､野を越え山越え､十里はなれた此のｼﾗｸｽの市にやって来た｡ﾒﾛｽには父も､母も無い｡女房も無い｡十六の､内気な妹と二人暮しだ｡この妹は､村の或る律気な一牧人を､近々､花婿として迎える事になっていた｡結婚式も間近かなのである｡ﾒﾛｽは､それゆえ､花嫁の衣裳やら祝宴の御馳走やらを買いに､はるばる市にやって来たのだ｡先ず､その品々を買い集め､それから都の大路をぶらぶら歩いた｡ﾒﾛｽには竹馬の友があった｡ｾﾘﾇﾝﾃｨｳｽである｡今は此のｼﾗｸｽの市で､石工をしている｡その友を､これから訪ねてみるつもりなのだ｡久しく逢わなかったのだから､訪ねて行くのが楽しみである｡歩いているうちにﾒﾛｽは､まちの様子を怪しく思った｡ひっそりしている｡もう既に日も落ちて､まちの暗いのは当りまえだが､けれども､なんだか､夜のせいばかりでは無く､市全体が､やけに寂しい｡のんきなﾒﾛｽも､だんだん不安になって来た｡路で逢った若い衆をつかまえて､何かあったのか､二年まえに此の市に来たときは､夜でも皆が歌をうたって､まちは賑やかであった筈だが､と質問した｡若い衆は､首を振って答えなかった｡しばらく歩いて老爺に逢い､こんどはもっと､語勢を強くして質問した｡老爺は答えなかった｡ﾒﾛｽは両手で老爺のからだをゆすぶって質問を重ねた｡老爺は､あたりをはばかる低声で､わずか答えた｡
｢王様は､人を殺します｡｣
｢なぜ殺すのだ｡｣
｢悪心を抱いている､というのですが､誰もそんな､悪心を持っては居りませぬ｡｣
｢たくさんの人を殺したのか｡｣
｢はい､はじめは王様の妹婿さまを｡それから､御自身のお世嗣を｡それから､妹さまを｡それから､妹さまの御子さまを｡それから､皇后さまを｡それから､賢臣のｱﾚｷｽ様を｡｣
｢おどろいた｡国王は乱心か｡｣
｢いいえ､乱心ではございませぬ｡人を､信ずる事が出来ぬ､というのです｡このごろは､臣下の心をも､お疑いになり､少しく派手な暮しをしている者には､人質ひとりずつ差し出すことを命じて居ります｡御命令を拒めば十字架にかけられて､殺されます｡きょうは､六人殺されました｡｣
聞いて､ﾒﾛｽは激怒した｡｢呆れた王だ｡生かして置けぬ｡｣
ﾒﾛｽは､単純な男であった｡買い物を､背負ったままで､のそのそ王城にはいって行った｡たちまち彼は､巡邏の警吏に捕縛された｡調べられて､ﾒﾛｽの懐中からは短剣が出て来たので､騒ぎが大きくなってしまった｡ﾒﾛｽは､王の前に引き出された｡
｢この短刀で何をするつもりであったか｡言え!｣暴君ﾃﾞｨｵﾆｽは静かに､けれども威厳を以て問いつめた｡その王の顔は蒼白で､眉間の皺は､刻み込まれたように深かった｡
｢市を暴君の手から救うのだ｡｣とﾒﾛｽは悪びれずに答えた｡
｢おまえがか?｣王は､憫笑した｡｢仕方の無いやつじゃ｡おまえには､わしの孤独がわからぬ｡｣
｢言うな!｣とﾒﾛｽは､いきり立って反駁した｡｢人の心を疑うのは､最も恥ずべき悪徳だ｡王は､民の忠誠をさえ疑って居られる｡｣
｢疑うのが､正当の心構えなのだと､わしに教えてくれたのは､おまえたちだ｡人の心は､あてにならない｡人間は､もともと私慾のかたまりさ｡信じては､ならぬ｡｣暴君は落着いて呟き､ほっと溜息をついた｡｢わしだって､平和を望んでいるのだが｡｣
｢なんの為の平和だ｡自分の地位を守る為か｡｣こんどはﾒﾛｽが嘲笑した｡｢罪の無い人を殺して､何が平和だ｡｣
｢だまれ､下賤の者｡｣王は､さっと顔を挙げて報いた｡｢口では､どんな清らかな事でも言える｡わしには､人の腹綿の奥底が見え透いてならぬ｡おまえだって､いまに､磔になってから､泣いて詫びたって聞かぬぞ｡｣
｢ああ､王は悧巧だ｡自惚れているがよい｡私は､ちゃんと死ぬる覚悟で居るのに｡命乞いなど決してしない｡ただ､――｣と言いかけて､ﾒﾛｽは足もとに視線を落し瞬時ためらい､｢ただ､私に情をかけたいつもりなら､処刑までに三日間の日限を与えて下さい｡たった一人の妹に､亭主を持たせてやりたいのです｡三日のうちに､私は村で結婚式を挙げさせ､必ず､ここへ帰って来ます｡｣
｢ばかな｡｣と暴君は､嗄れた声で低く笑った｡｢とんでもない嘘を言うわい｡逃がした小鳥が帰って来るというのか｡｣
｢そうです｡帰って来るのです｡｣ﾒﾛｽは必死で言い張った｡｢私は約束を守ります｡私を､三日間だけ許して下さい｡妹が､私の帰りを待っているのだ｡そんなに私を信じられないならば､よろしい､この市にｾﾘﾇﾝﾃｨｳｽという石工がいます｡私の無二の友人だ｡あれを､人質としてここに置いて行こう｡私が逃げてしまって､三日目の日暮まで､ここに帰って来なかったら､あの友人を絞め殺して下さい｡たのむ､そうして下さい｡｣
それを聞いて王は､残虐な気持で､そっと北叟笑んだ｡生意気なことを言うわい｡どうせ帰って来ないにきまっている｡この嘘つきに騙された振りして､放してやるのも面白い｡そうして身代りの男を､三日目に殺してやるのも気味がいい｡人は､これだから信じられぬと､わしは悲しい顔して､その身代りの男を磔刑に処してやるのだ｡世の中の､正直者とかいう奴輩にうんと見せつけてやりたいものさ｡
｢願いを､聞いた｡その身代りを呼ぶがよい｡三日目には日没までに帰って来い｡おくれたら､その身代りを､きっと殺すぞ｡ちょっとおくれて来るがいい｡おまえの罪は､永遠にゆるしてやろうぞ｡｣
｢なに､何をおっしゃる｡｣
｢はは｡いのちが大事だったら､おくれて来い｡おまえの心は､わかっているぞ｡｣
ﾒﾛｽは口惜しく､地団駄踏んだ｡ものも言いたくなくなった｡
竹馬の友､ｾﾘﾇﾝﾃｨｳｽは､深夜､王城に召された｡暴君ﾃﾞｨｵﾆｽの面前で､佳き友と佳き友は､二年ぶりで相逢うた｡ﾒﾛｽは､友に一切の事情を語った｡ｾﾘﾇﾝﾃｨｳｽは無言で首肯き､ﾒﾛｽをひしと抱きしめた｡友と友の間は､それでよかった｡ｾﾘﾇﾝﾃｨｳｽは､縄打たれた｡ﾒﾛｽは､すぐに出発した｡初夏､満天の星である｡
ﾒﾛｽはその夜､一睡もせず十里の路を急ぎに急いで､村へ到着したのは､翌る日の午前､陽は既に高く昇って､村人たちは野に出て仕事をはじめていた｡ﾒﾛｽの十六の妹も､きょうは兄の代りに羊群の番をしていた｡よろめいて歩いて来る兄の､疲労困憊の姿を見つけて驚いた｡そうして､うるさく兄に質問を浴びせた｡
｢なんでも無い｡｣ﾒﾛｽは無理に笑おうと努めた｡｢市に用事を残して来た｡またすぐ市に行かなければならぬ｡あす､おまえの結婚式を挙げる｡早いほうがよかろう｡｣
妹は頬をあからめた｡
｢うれしいか｡綺麗な衣裳も買って来た｡さあ､これから行って､村の人たちに知らせて来い｡結婚式は､あすだと｡｣
ﾒﾛｽは､また､よろよろと歩き出し､家へ帰って神々の祭壇を飾り､祝宴の席を調え､間もなく床に倒れ伏し､呼吸もせぬくらいの深い眠りに落ちてしまった｡
眼が覚めたのは夜だった｡ﾒﾛｽは起きてすぐ､花婿の家を訪れた｡そうして､少し事情があるから､結婚式を明日にしてくれ､と頼んだ｡婿の牧人は驚き､それはいけない､こちらには未だ何の仕度も出来ていない､葡萄の季節まで待ってくれ､と答えた｡ﾒﾛｽは､待つことは出来ぬ､どうか明日にしてくれ給え､と更に押してたのんだ｡婿の牧人も頑強であった｡なかなか承諾してくれない｡夜明けまで議論をつづけて､やっと､どうにか婿をなだめ､すかして､説き伏せた｡結婚式は､真昼に行われた｡新郎新婦の､神々への宣誓が済んだころ､黒雲が空を覆い､ぽつりぽつり雨が降り出し､やがて車軸を流すような大雨となった｡祝宴に列席していた村人たちは､何か不吉なものを感じたが､それでも､めいめい気持を引きたて､狭い家の中で､むんむん蒸し暑いのも怺え､陽気に歌をうたい､手を拍った｡ﾒﾛｽも､満面に喜色を湛え､しばらくは､王とのあの約束をさえ忘れていた｡祝宴は､夜に入っていよいよ乱れ華やかになり､人々は､外の豪雨を全く気にしなくなった｡ﾒﾛｽは､一生このままここにいたい､と思った｡この佳い人たちと生涯暮して行きたいと願ったが､いまは､自分のからだで､自分のものでは無い｡ままならぬ事である｡ﾒﾛｽは､わが身に鞭打ち､ついに出発を決意した｡あすの日没までには､まだ十分の時が在る｡ちょっと一眠りして､それからすぐに出発しよう､と考えた｡その頃には､雨も小降りになっていよう｡少しでも永くこの家に愚図愚図とどまっていたかった｡ﾒﾛｽほどの男にも､やはり未練の情というものは在る｡今宵呆然､歓喜に酔っているらしい花嫁に近寄り､
｢おめでとう｡私は疲れてしまったから､ちょっとご免こうむって眠りたい｡眼が覚めたら､すぐに市に出かける｡大切な用事があるのだ｡私がいなくても､もうおまえには優しい亭主があるのだから､決して寂しい事は無い｡おまえの兄の､一ばんきらいなものは､人を疑う事と､それから､嘘をつく事だ｡おまえも､それは､知っているね｡亭主との間に､どんな秘密でも作ってはならぬ｡おまえに言いたいのは､それだけだ｡おまえの兄は､たぶん偉い男なのだから､おまえもその誇りを持っていろ｡｣
花嫁は､夢見心地で首肯いた｡ﾒﾛｽは､それから花婿の肩をたたいて､
｢仕度の無いのはお互さまさ｡私の家にも､宝といっては､妹と羊だけだ｡他には､何も無い｡全部あげよう｡もう一つ､ﾒﾛｽの弟になったことを誇ってくれ｡｣
花婿は揉み手して､てれていた｡ﾒﾛｽは笑って村人たちにも会釈して､宴席から立ち去り､羊小屋にもぐり込んで､死んだように深く眠った｡
眼が覚めたのは翌る日の薄明の頃である｡ﾒﾛｽは跳ね起き､南無三､寝過したか､いや､まだまだ大丈夫､これからすぐに出発すれば､約束の刻限までには十分間に合う｡きょうは是非とも､あの王に､人の信実の存するところを見せてやろう｡そうして笑って磔の台に上ってやる｡ﾒﾛｽは､悠々と身仕度をはじめた｡雨も､いくぶん小降りになっている様子である｡身仕度は出来た｡さて､ﾒﾛｽは､ぶるんと両腕を大きく振って､雨中､矢の如く走り出た｡
私は､今宵､殺される｡殺される為に走るのだ｡身代りの友を救う為に走るのだ｡王の奸佞邪智を打ち破る為に走るのだ｡走らなければならぬ｡そうして､私は殺される｡若い時から名誉を守れ｡さらば､ふるさと｡若いﾒﾛｽは､つらかった｡幾度か､立ちどまりそうになった｡えい､えいと大声挙げて自身を叱りながら走った｡村を出て､野を横切り､森をくぐり抜け､隣村に着いた頃には､雨も止み､日は高く昇って､そろそろ暑くなって来た｡ﾒﾛｽは額の汗をこぶしで払い､ここまで来れば大丈夫､もはや故郷への未練は無い｡妹たちは､きっと佳い夫婦になるだろう｡私には､いま､なんの気がかりも無い筈だ｡まっすぐに王城に行き着けば､それでよいのだ｡そんなに急ぐ必要も無い｡ゆっくり歩こう､と持ちまえの呑気さを取り返し､好きな小歌をいい声で歌い出した｡ぶらぶら歩いて二里行き三里行き､そろそろ全里程の半ばに到達した頃､降って湧いた災難､ﾒﾛｽの足は､はたと､とまった｡見よ､前方の川を｡きのうの豪雨で山の水源地は氾濫し､濁流滔々と下流に集り､猛勢一挙に橋を破壊し､どうどうと響きをあげる激流が､木葉微塵に橋桁を跳ね飛ばしていた｡彼は茫然と､立ちすくんだ｡あちこちと眺めまわし､また､声を限りに呼びたててみたが､繋舟は残らず浪に浚われて影なく､渡守りの姿も見えない｡流れはいよいよ､ふくれ上り､海のようになっている｡ﾒﾛｽは川岸にうずくまり､男泣きに泣きながらｾﾞｳｽに手を挙げて哀願した｡｢ああ､鎮めたまえ､荒れ狂う流れを! 時は刻々に過ぎて行きます｡太陽も既に真昼時です｡あれが沈んでしまわぬうちに､王城に行き着くことが出来なかったら､あの佳い友達が､私のために死ぬのです｡｣
濁流は､ﾒﾛｽの叫びをせせら笑う如く､ますます激しく躍り狂う｡浪は浪を呑み､捲き､煽り立て､そうして時は､刻一刻と消えて行く｡今はﾒﾛｽも覚悟した｡泳ぎ切るより他に無い｡ああ､神々も照覧あれ! 濁流にも負けぬ愛と誠の偉大な力を､いまこそ発揮して見せる｡ﾒﾛｽは､ざんぶと流れに飛び込み､百匹の大蛇のようにのた打ち荒れ狂う浪を相手に､必死の闘争を開始した｡満身の力を腕にこめて､押し寄せ渦巻き引きずる流れを､なんのこれしきと掻きわけ掻きわけ､めくらめっぽう獅子奮迅の人の子の姿には､神も哀れと思ったか､ついに憐愍を垂れてくれた｡押し流されつつも､見事､対岸の樹木の幹に､すがりつく事が出来たのである｡ありがたい｡ﾒﾛｽは馬のように大きな胴震いを一つして､すぐにまた先きを急いだ｡一刻といえども､むだには出来ない｡陽は既に西に傾きかけている｡ぜいぜい荒い呼吸をしながら峠をのぼり､のぼり切って､ほっとした時､突然､目の前に一隊の山賊が躍り出た｡
｢待て｡｣
｢何をするのだ｡私は陽の沈まぬうちに王城へ行かなければならぬ｡放せ｡｣
｢どっこい放さぬ｡持ちもの全部を置いて行け｡｣
｢私にはいのちの他には何も無い｡その､たった一つの命も､これから王にくれてやるのだ｡｣
｢その､いのちが欲しいのだ｡｣
｢さては､王の命令で､ここで私を待ち伏せしていたのだな｡｣
山賊たちは､ものも言わず一斉に棍棒を振り挙げた｡ﾒﾛｽはひょいと､からだを折り曲げ､飛鳥の如く身近かの一人に襲いかかり､その棍棒を奪い取って､
｢気の毒だが正義のためだ!｣と猛然一撃､たちまち､三人を殴り倒し､残る者のひるむ隙に､さっさと走って峠を下った｡一気に峠を駈け降りたが､流石に疲労し､折から午後の灼熱の太陽がまともに､かっと照って来て､ﾒﾛｽは幾度となく眩暈を感じ､これではならぬ､と気を取り直しては､よろよろ二､三歩あるいて､ついに､がくりと膝を折った｡立ち上る事が出来ぬのだ｡天を仰いで､くやし泣きに泣き出した｡ああ､あ､濁流を泳ぎ切り､山賊を三人も撃ち倒し韋駄天､ここまで突破して来たﾒﾛｽよ｡真の勇者､ﾒﾛｽよ｡今､ここで､疲れ切って動けなくなるとは情無い｡愛する友は､おまえを信じたばかりに､やがて殺されなければならぬ｡おまえは､稀代の不信の人間､まさしく王の思う壺だぞ､と自分を叱ってみるのだが､全身萎えて､もはや芋虫ほどにも前進かなわぬ｡路傍の草原にごろりと寝ころがった｡身体疲労すれば､精神も共にやられる｡もう､どうでもいいという､勇者に不似合いな不貞腐れた根性が､心の隅に巣喰った｡私は､これほど努力したのだ｡約束を破る心は､みじんも無かった｡神も照覧､私は精一ぱいに努めて来たのだ｡動けなくなるまで走って来たのだ｡私は不信の徒では無い｡ああ､できる事なら私の胸を截ち割って､真紅の心臓をお目に掛けたい｡愛と信実の血液だけで動いているこの心臓を見せてやりたい｡けれども私は､この大事な時に､精も根も尽きたのだ｡私は､よくよく不幸な男だ｡私は､きっと笑われる｡私の一家も笑われる｡私は友を欺いた｡中途で倒れるのは､はじめから何もしないのと同じ事だ｡ああ､もう､どうでもいい｡これが､私の定った運命なのかも知れない｡ｾﾘﾇﾝﾃｨｳｽよ､ゆるしてくれ｡君は､いつでも私を信じた｡私も君を､欺かなかった｡私たちは､本当に佳い友と友であったのだ｡いちどだって､暗い疑惑の雲を､お互い胸に宿したことは無かった｡いまだって､君は私を無心に待っているだろう｡ああ､待っているだろう｡ありがとう､ｾﾘﾇﾝﾃｨｳｽ｡よくも私を信じてくれた｡それを思えば､たまらない｡友と友の間の信実は､この世で一ばん誇るべき宝なのだからな｡ｾﾘﾇﾝﾃｨｳｽ､私は走ったのだ｡君を欺くつもりは､みじんも無かった｡信じてくれ! 私は急ぎに急いでここまで来たのだ｡濁流を突破した｡山賊の囲みからも､するりと抜けて一気に峠を駈け降りて来たのだ｡私だから､出来たのだよ｡ああ､この上､私に望み給うな｡放って置いてくれ｡どうでも､いいのだ｡私は負けたのだ｡だらしが無い｡笑ってくれ｡王は私に､ちょっとおくれて来い､と耳打ちした｡おくれたら､身代りを殺して､私を助けてくれると約束した｡私は王の卑劣を憎んだ｡けれども､今になってみると､私は王の言うままになっている｡私は､おくれて行くだろう｡王は､ひとり合点して私を笑い､そうして事も無く私を放免するだろう｡そうなったら､私は､死ぬよりつらい｡私は､永遠に裏切者だ｡地上で最も､不名誉の人種だ｡ｾﾘﾇﾝﾃｨｳｽよ､私も死ぬぞ｡君と一緒に死なせてくれ｡君だけは私を信じてくれるにちがい無い｡いや､それも私の､ひとりよがりか? ああ､もういっそ､悪徳者として生き伸びてやろうか｡村には私の家が在る｡羊も居る｡妹夫婦は､まさか私を村から追い出すような事はしないだろう｡正義だの､信実だの､愛だの､考えてみれば､くだらない｡人を殺して自分が生きる｡それが人間世界の定法ではなかったか｡ああ､何もかも､ばかばかしい｡私は､醜い裏切り者だ｡どうとも､勝手にするがよい｡やんぬる哉｡――四肢を投げ出して､うとうと､まどろんでしまった｡
ふと耳に､潺々､水の流れる音が聞えた｡そっと頭をもたげ､息を呑んで耳をすました｡すぐ足もとで､水が流れているらしい｡よろよろ起き上って､見ると､岩の裂目から滾々と､何か小さく囁きながら清水が湧き出ているのである｡その泉に吸い込まれるようにﾒﾛｽは身をかがめた｡水を両手で掬って､一くち飲んだ｡ほうと長い溜息が出て､夢から覚めたような気がした｡歩ける｡行こう｡肉体の疲労恢復と共に､わずかながら希望が生れた｡義務遂行の希望である｡わが身を殺して､名誉を守る希望である｡斜陽は赤い光を､樹々の葉に投じ､葉も枝も燃えるばかりに輝いている｡日没までには､まだ間がある｡私を､待っている人があるのだ｡少しも疑わず､静かに期待してくれている人があるのだ｡私は､信じられている｡私の命なぞは､問題ではない｡死んでお詫び､などと気のいい事は言って居られぬ｡私は､信頼に報いなければならぬ｡いまはただその一事だ｡走れ! ﾒﾛｽ｡
私は信頼されている｡私は信頼されている｡先刻の､あの悪魔の囁きは､あれは夢だ｡悪い夢だ｡忘れてしまえ｡五臓が疲れているときは､ふいとあんな悪い夢を見るものだ｡ﾒﾛｽ､おまえの恥ではない｡やはり､おまえは真の勇者だ｡再び立って走れるようになったではないか｡ありがたい! 私は､正義の士として死ぬ事が出来るぞ｡ああ､陽が沈む｡ずんずん沈む｡待ってくれ､ｾﾞｳｽよ｡私は生れた時から正直な男であった｡正直な男のままにして死なせて下さい｡
路行く人を押しのけ､跳ねとばし､ﾒﾛｽは黒い風のように走った｡野原で酒宴の､その宴席のまっただ中を駈け抜け､酒宴の人たちを仰天させ､犬を蹴とばし､小川を飛び越え､少しずつ沈んでゆく太陽の､十倍も早く走った｡一団の旅人と颯っとすれちがった瞬間､不吉な会話を小耳にはさんだ｡｢いまごろは､あの男も､磔にかかっているよ｡｣ああ､その男､その男のために私は､いまこんなに走っているのだ｡その男を死なせてはならない｡急げ､ﾒﾛｽ｡おくれてはならぬ｡愛と誠の力を､いまこそ知らせてやるがよい｡風態なんかは､どうでもいい｡ﾒﾛｽは､いまは､ほとんど全裸体であった｡呼吸も出来ず､二度､三度､口から血が噴き出た｡見える｡はるか向うに小さく､ｼﾗｸｽの市の塔楼が見える｡塔楼は､夕陽を受けてきらきら光っている｡
｢ああ､ﾒﾛｽ様｡｣うめくような声が､風と共に聞えた｡
｢誰だ｡｣ﾒﾛｽは走りながら尋ねた｡
｢ﾌｨﾛｽﾄﾗﾄｽでございます｡貴方のお友達ｾﾘﾇﾝﾃｨｳｽ様の弟子でございます｡｣その若い石工も､ﾒﾛｽの後について走りながら叫んだ｡｢もう､駄目でございます｡むだでございます｡走るのは､やめて下さい｡もう､あの方をお助けになることは出来ません｡｣
｢いや､まだ陽は沈まぬ｡｣
｢ちょうど今､あの方が死刑になるところです｡ああ､あなたは遅かった｡おうらみ申します｡ほんの少し､もうちょっとでも､早かったなら!｣
｢いや､まだ陽は沈まぬ｡｣ﾒﾛｽは胸の張り裂ける思いで､赤く大きい夕陽ばかりを見つめていた｡走るより他は無い｡
｢やめて下さい｡走るのは､やめて下さい｡いまはご自分のお命が大事です｡あの方は､あなたを信じて居りました｡刑場に引き出されても､平気でいました｡王様が､さんざんあの方をからかっても､ﾒﾛｽは来ます､とだけ答え､強い信念を持ちつづけている様子でございました｡｣
｢それだから､走るのだ｡信じられているから走るのだ｡間に合う､間に合わぬは問題でないのだ｡人の命も問題でないのだ｡私は､なんだか､もっと恐ろしく大きいものの為に走っているのだ｡ついて来い! ﾌｨﾛｽﾄﾗﾄｽ｡｣
｢ああ､あなたは気が狂ったか｡それでは､うんと走るがいい｡ひょっとしたら､間に合わぬものでもない｡走るがいい｡｣
言うにや及ぶ｡まだ陽は沈まぬ｡最後の死力を尽して､ﾒﾛｽは走った｡ﾒﾛｽの頭は､からっぽだ｡何一つ考えていない｡ただ､わけのわからぬ大きな力にひきずられて走った｡陽は､ゆらゆら地平線に没し､まさに最後の一片の残光も､消えようとした時､ﾒﾛｽは疾風の如く刑場に突入した｡間に合った｡
｢待て｡その人を殺してはならぬ｡ﾒﾛｽが帰って来た｡約束のとおり､いま､帰って来た｡｣と大声で刑場の群衆にむかって叫んだつもりであったが､喉がつぶれて嗄れた声が幽かに出たばかり､群衆は､ひとりとして彼の到着に気がつかない｡すでに磔の柱が高々と立てられ､縄を打たれたｾﾘﾇﾝﾃｨｳｽは､徐々に釣り上げられてゆく｡ﾒﾛｽはそれを目撃して最後の勇､先刻､濁流を泳いだように群衆を掻きわけ､掻きわけ､
｢私だ､刑吏! 殺されるのは､私だ｡ﾒﾛｽだ｡彼を人質にした私は､ここにいる!｣と､かすれた声で精一ぱいに叫びながら､ついに磔台に昇り､釣り上げられてゆく友の両足に､齧りついた｡群衆は､どよめいた｡あっぱれ｡ゆるせ､と口々にわめいた｡ｾﾘﾇﾝﾃｨｳｽの縄は､ほどかれたのである｡
｢ｾﾘﾇﾝﾃｨｳｽ｡｣ﾒﾛｽは眼に涙を浮べて言った｡｢私を殴れ｡ちから一ぱいに頬を殴れ｡私は､途中で一度､悪い夢を見た｡君が若し私を殴ってくれなかったら､私は君と抱擁する資格さえ無いのだ｡殴れ｡｣
ｾﾘﾇﾝﾃｨｳｽは､すべてを察した様子で首肯き､刑場一ぱいに鳴り響くほど音高くﾒﾛｽの右頬を殴った｡殴ってから優しく微笑み､
｢ﾒﾛｽ､私を殴れ｡同じくらい音高く私の頬を殴れ｡私はこの三日の間､たった一度だけ､ちらと君を疑った｡生れて､はじめて君を疑った｡君が私を殴ってくれなければ､私は君と抱擁できない｡｣
ﾒﾛｽは腕に唸りをつけてｾﾘﾇﾝﾃｨｳｽの頬を殴った｡
｢ありがとう､友よ｡｣二人同時に言い､ひしと抱き合い､それから嬉し泣きにおいおい声を放って泣いた｡
群衆の中からも､歔欷の声が聞えた｡暴君ﾃﾞｨｵﾆｽは､群衆の背後から二人の様を､まじまじと見つめていたが､やがて静かに二人に近づき､顔をあからめて､こう言った｡
｢おまえらの望みは叶ったぞ｡おまえらは､わしの心に勝ったのだ｡信実とは､決して空虚な妄想ではなかった｡どうか､わしをも仲間に入れてくれまいか｡どうか､わしの願いを聞き入れて､おまえらの仲間の一人にしてほしい｡｣
どっと群衆の間に､歓声が起った｡
｢万歳､王様万歳｡｣
ひとりの少女が､緋のﾏﾝﾄをﾒﾛｽに捧げた｡ﾒﾛｽは､まごついた｡佳き友は､気をきかせて教えてやった｡
｢ﾒﾛｽ､君は､まっぱだかじゃないか｡早くそのﾏﾝﾄを着るがいい｡この可愛い娘さんは､ﾒﾛｽの裸体を､皆に見られるのが､たまらなく口惜しいのだ｡｣
勇者は､ひどく赤面した｡"""
},






{
"@search.action": "upload",
"BookId": "46605",
"BookName": "やまなし",
"BookAuthor": "宮沢賢治",
"BookGenre": "短編小説",
"Release_Date": "1923年4月",
"Rating": "3.81",
"Text": """小さな谷川の底を写した二枚の青い幻燈です｡
一､五月
二疋の蟹の子供らが青じろい水の底で話していました｡
『ｸﾗﾑﾎﾞﾝはわらったよ｡』
『ｸﾗﾑﾎﾞﾝはかぷかぷわらったよ｡』
『ｸﾗﾑﾎﾞﾝは跳ねてわらったよ｡』
『ｸﾗﾑﾎﾞﾝはかぷかぷわらったよ｡』
上の方や横の方は､青くくらく鋼のように見えます｡そのなめらかな天井を､つぶつぶ暗い泡が流れて行きます｡
『ｸﾗﾑﾎﾞﾝはわらっていたよ｡』
『ｸﾗﾑﾎﾞﾝはかぷかぷわらったよ｡』
『それならなぜｸﾗﾑﾎﾞﾝはわらったの｡』
『知らない｡』
つぶつぶ泡が流れて行きます｡蟹の子供らもぽっぽっぽっとつづけて五六粒泡を吐きました｡それはゆれながら水銀のように光って斜めに上の方へのぼって行きました｡
つうと銀のいろの腹をひるがえして､一疋の魚が頭の上を過ぎて行きました｡
『ｸﾗﾑﾎﾞﾝは死んだよ｡』
『ｸﾗﾑﾎﾞﾝは殺されたよ｡』
『ｸﾗﾑﾎﾞﾝは死んでしまったよ………｡』
『殺されたよ｡』
『それならなぜ殺された｡』兄さんの蟹は､その右側の四本の脚の中の二本を､弟の平べったい頭にのせながら云いました｡
『わからない｡』
魚がまたﾂｳと戻って下流のほうへ行きました｡
『ｸﾗﾑﾎﾞﾝはわらったよ｡』
『わらった｡』
にわかにﾊﾟｯと明るくなり､日光の黄金は夢のように水の中に降って来ました｡
波から来る光の網が､底の白い磐の上で美しくゆらゆらのびたりちぢんだりしました｡泡や小さなごみからはまっすぐな影の棒が､斜めに水の中に並んで立ちました｡
魚がこんどはそこら中の黄金の光をまるっきりくちゃくちゃにしておまけに自分は鉄いろに変に底びかりして､又上流の方へのぼりました｡
『お魚はなぜああ行ったり来たりするの｡』
弟の蟹がまぶしそうに眼を動かしながらたずねました｡
『何か悪いことをしてるんだよとってるんだよ｡』
『とってるの｡』
『うん｡』
そのお魚がまた上流から戻って来ました｡今度はゆっくり落ちついて､ひれも尾も動かさずただ水にだけ流されながらお口を環のように円くしてやって来ました｡その影は黒くしずかに底の光の網の上をすべりました｡
『お魚は……｡』
その時です｡俄に天井に白い泡がたって､青びかりのまるでぎらぎらする鉄砲弾のようなものが､いきなり飛込んで来ました｡
兄さんの蟹ははっきりとその青いもののさきがｺﾝﾊﾟｽのように黒く尖っているのも見ました｡と思ううちに､魚の白い腹がぎらっと光って一ぺんひるがえり､上の方へのぼったようでしたが､それっきりもう青いものも魚のかたちも見えず光の黄金の網はゆらゆらゆれ､泡はつぶつぶ流れました｡
二疋はまるで声も出ず居すくまってしまいました｡
お父さんの蟹が出て来ました｡
『どうしたい｡ぶるぶるふるえているじゃないか｡』
『お父さん､いまおかしなものが来たよ｡』
『どんなもんだ｡』
『青くてね､光るんだよ｡はじがこんなに黒く尖ってるの｡それが来たらお魚が上へのぼって行ったよ｡』
『そいつの眼が赤かったかい｡』
『わからない｡』
『ふうん｡しかし､そいつは鳥だよ｡かわせみと云うんだ｡大丈夫だ､安心しろ｡おれたちはかまわないんだから｡』
『お父さん､お魚はどこへ行ったの｡』
『魚かい｡魚はこわい所へ行った』
『こわいよ､お父さん｡』
『いいいい､大丈夫だ｡心配するな｡そら､樺の花が流れて来た｡ごらん､きれいだろう｡』
泡と一緒に､白い樺の花びらが天井をたくさんすべって来ました｡
『こわいよ､お父さん｡』弟の蟹も云いました｡
光の網はゆらゆら､のびたりちぢんだり､花びらの影はしずかに砂をすべりました｡
二､十二月
蟹の子供らはもうよほど大きくなり､底の景色も夏から秋の間にすっかり変りました｡
白い柔かな円石もころがって来､小さな錐の形の水晶の粒や､金雲母のかけらもながれて来てとまりました｡
そのつめたい水の底まで､ﾗﾑﾈの瓶の月光がいっぱいに透とおり天井では波が青じろい火を､燃したり消したりしているよう､あたりはしんとして､ただいかにも遠くからというように､その波の音がひびいて来るだけです｡
蟹の子供らは､あんまり月が明るく水がきれいなので睡らないで外に出て､しばらくだまって泡をはいて天上の方を見ていました｡
『やっぱり僕の泡は大きいね｡』
『兄さん､わざと大きく吐いてるんだい｡僕だってわざとならもっと大きく吐けるよ｡』
『吐いてごらん｡おや､たったそれきりだろう｡いいかい､兄さんが吐くから見ておいで｡そら､ね､大きいだろう｡』
『大きかないや､おんなじだい｡』
『近くだから自分のが大きく見えるんだよ｡そんなら一緒に吐いてみよう｡いいかい､そら｡』
『やっぱり僕の方大きいよ｡』
『本当かい｡じゃ､も一つはくよ｡』
『だめだい､そんなにのびあがっては｡』
またお父さんの蟹が出て来ました｡
『もうねろねろ｡遅いぞ､あしたｲｻﾄﾞへ連れて行かんぞ｡』
『お父さん､僕たちの泡どっち大きいの』
『それは兄さんの方だろう』
『そうじゃないよ､僕の方大きいんだよ』弟の蟹は泣きそうになりました｡
そのとき､ﾄﾌﾞﾝ｡
黒い円い大きなものが､天井から落ちてずうっとしずんで又上へのぼって行きました｡ｷﾗｷﾗｯと黄金のぶちがひかりました｡
『かわせみだ』子供らの蟹は頸をすくめて云いました｡
お父さんの蟹は､遠めがねのような両方の眼をあらん限り延ばして､よくよく見てから云いました｡
『そうじゃない､あれはやまなしだ､流れて行くぞ､ついて行って見よう､ああいい匂いだな』
なるほど､そこらの月あかりの水の中は､やまなしのいい匂いでいっぱいでした｡
三疋はぼかぼか流れて行くやまなしのあとを追いました｡
その横あるきと､底の黒い三つの影法師が､合せて六つ踊るようにして､やまなしの円い影を追いました｡
間もなく水はｻﾗｻﾗ鳴り､天井の波はいよいよ青い焔をあげ､やまなしは横になって木の枝にひっかかってとまり､その上には月光の虹がもかもか集まりました｡
『どうだ､やっぱりやまなしだよ､よく熟している､いい匂いだろう｡』
『おいしそうだね､お父さん』
『待て待て､もう二日ばかり待つとね､こいつは下へ沈んで来る､それからひとりでにおいしいお酒ができるから､さあ､もう帰って寝よう､おいで』
親子の蟹は三疋自分等の穴に帰って行きます｡
波はいよいよ青じろい焔をゆらゆらとあげました､それは又金剛石の粉をはいているようでした｡
*
私の幻燈はこれでおしまいであります"""
},

{
"@search.action": "upload",
"BookId": "4802",
"BookName": "聖家族",
"BookAuthor": "堀辰雄",
"BookGenre": "短編小説",
"Release_Date": "1930年11月",
"Rating": "3.50",
"Text": """死があたかも一つの季節を開いたかのようだった｡
死人の家への道には､自動車の混雑が次第に増加して行った｡そしてそれは､その道幅が狭いために､各々の車は動いている間よりも､停止している間の方が長いくらいにまでなっていた｡
それは三月だった｡空気はまだ冷たかったが､もうそんなに呼吸しにくくはなかった｡いつのまにか､もの好きな群集がそれらの自動車を取り囲んで､そのなかの人達をよく見ようとしながら､硝子窓に鼻をくっつけた｡それが硝子窓を白く曇らせた｡そしてそのなかでは､その持主等が不安そうな､しかし舞踏会にでも行くときのような微笑を浮べて､彼等を見かえしていた｡
そういう硝子窓の一つのなかに､一人の貴婦人らしいのが､目を閉じたきり､頭を重たそうにｸｯｼｮﾝに凭せながら､死人のようになっているのを見ると､
｢あれは誰だろう?｣
そう人々は囁き合った｡
それは細木と云う未亡人だった｡――それまでのどれより長いように思われた自動車の停止が､その夫人をそういう仮死から蘇らせたように見えた｡するとその夫人は自分の運転手に何か言いながら､ひとりでﾄﾞｱを開けて､車から降りてしまった｡丁度そのとき前方の車が動き出したため､彼女の車はそこに自分の持主を置いたまま､再び動き出して行った｡
それと殆ど同時に人々は見たのだった｡帽子もかぶらずに毛髪をくしゃくしゃにさせた一人の青年が､群集を押し分けるようにして､そこに漂流物のように浮いたり沈んだりして見えるその夫人に近づいて行きながら､そしていかにも親しげに笑いかけながら､彼女の腕をつかまえたのを――
その二人がやっとのことで群集の外に出たとき､細木夫人は自分が一人の見知らない青年の腕にほとんど靠れかかっているのに､はじめて気づいたようだった｡彼女はその青年から腕を離すと､何か問いたげな眼ざしを彼の上に投げながら､
｢ありがとうございました｣
と言った｡青年は､相手が自分を覚えていないらしいことに気がつくと､すこし顔を赤らめながら答えた｡
｢僕､河野です｣
その名前を聞いても夫人にはどうしても思い出されないらしいその青年の顔は､しかしその上品な顔立によっていくらか夫人を安心させたらしかった｡
｢九鬼さんのお宅はもう近くでございますか｣と夫人がきいた｡
｢ええ､すぐそこです｣
そう答えながら青年は驚いたように相手をふりむいた｡突然､彼女がそこに立ち止まってしまったのだ｡
｢あの､どこかこのへんに休むところはございませんかしら｡なんだかすこし気分が悪いものですから……｣
青年はすぐその近くに一つの小さなｶｯﾌｪを見つけた｡――そのなかに彼等がはいって見ると､しかしﾃｴﾌﾞﾙは埃のにおいがし､植木鉢は木の葉がすっかり灰色になっていた｡それをいまさらのように青年は夫人のために気にするように見えたけれど､夫人の方ではそれをそれほど気にはしていないらしかった｡鉢植の木の葉の灰色なのは自分のかなしみのためのように思って居るのかも知れぬと青年は考えた｡
青年は夫人の顔色がいくらかよくなったのを見ると､すこし吃りながら言った｡
｢僕､ちょっとまだ用事がありますので……すぐまた参りますから……｣
そうして彼は立ち上った｡

そこに一人ぎりになると､細木夫人はまた目をとじて死人の真似をした｡
――まるで舞踏会かなんぞのようなあの騒ぎは何ということだろう｡私にはとてもあの人達の中へはいって行けそうもない｡私はこのまま帰ってしまった方がいい……
それにしても夫人はいまの青年の帰ってくるまで待っていようと思った｡何だかその青年に一度どこかで会ったこともあるような気がし出したから｡そう言えば何処かしら死んだ九鬼に似ているところがあると彼女は思った｡そしてその類似が彼女に一つの記憶を喚び起した｡
数年前のことだった｡軽井沢のﾏﾝﾍﾟｲ･ﾎﾃﾙで偶然､彼女は九鬼に出会ったことがあった｡その時九鬼はひとりの十五ぐらいの少年を連れていたが､彼はその少年にちがいないと思い出した｡――その快活そうな少年を見ながら､彼女がすこし意地わるそうに､｢あなたによく似ていますわ｡あなたのお子さんじゃありませんの?｣そう言うと､九鬼は何か反撥するような微笑をしたきり黙りこんでしまった｡その時くらい九鬼が自分を憎んでいるように思われたことはない……


河野扁理は事実､その夫人の思い出のなかの少年なのだ｡
扁理の方では､勿論､数年前､軽井沢で九鬼と一しょに出会ったその夫人のことを忘れている筈はない｡
その時､彼は十五であった｡
彼はまだ快活で､無邪気な少年だった｡
九鬼が夫人をよほど好きなのではないかしらと思い出したのは､ずっと後のことだ｡その当時は､ただ九鬼が夫人を心から尊敬しているらしいのだけが分った｡それがいつしか夫人を彼の犯し難い偶像にさせていた｡ﾎﾃﾙでは､夫人の部屋は二階にあって､向日葵の咲いている中庭に面していた｡そしてその部屋の中に､ほとんど一日中閉じこもっていた｡そこへ一度もはいる機会のなかった彼は､日向葵の下から､よくその部屋を見上げた｡それは非常に神聖な､美しい､そして何か非現実なもののように思われた｡
そのﾎﾃﾙの部屋は､その後､彼の夢の中にしばしば現われた｡彼は夢の中では飛ぶことができた｡そのおかげで､彼はその部屋の中を窓ｶﾞﾗｽごしに見ることができた｡それは夢毎にかならず装飾を変えていた｡或る時はｲｷﾞﾘｽ風に､或る時は巴里風に｡
彼は今年二十になった｡同じ夢を抱いて､前よりはすこし悲しそうに､すこし痩せて｡
そしてさっきも､群集の間から､自動車のなかに死んだようになっている夫人をｶﾞﾗｽごしに見たときは､彼は自分が歩きながら夢を見ているのではないかと信じたくらいだった……


告別式の混雑によってすっかり死の感情を忘れさせられながら､その式場から帰ってきた扁理は､埃だらけのｶｯﾌｪのなかに､再びその死の感情を夫人と共に発見した｡
彼にはそれらのものが近づき難いように思われた｡そこでそれらに近づくために彼は出来るだけ悲しみを装おうとした｡だが､自分で気のついているよりずっと深いものだった､彼自身の悲しみがそれを彼にうまくさせなかった｡そして愚かそうに､彼はそこに突立っていた｡
｢どうでしたか?｣夫人が彼の方に顔をあげた｡
｢え､まだ大変な混雑です｣彼はどぎまぎしながら答えた｡
｢では､私､もうあちらへお伺いしないで､このまま帰りますわ……｣
そう言いながら夫人は自分の帯の間から小さな名刺を出してそれを彼に渡した｡
｢すっかりお見それして居りましたの……こんどお閑でしたら､宅へもお遊びにいらしって下さいませ｣
扁理は､自分が夫人に思い出されたことを知り､その上そういう夫人からの申し出を聞くと､一そうどぎまぎしながら､何かしきりに自分もﾎﾟｹｯﾄの中を探し出した｡そうしてやっと一枚の名刺を取り出した｡それは九鬼の名刺だった｡
｢自分の名刺がありませんので……｣そう言って､もの怖じた子供のように微笑しながら､彼はその名刺を裏がえし､そこに
河野扁理
という字を不恰好に書いた｡
それを見ながら､さっきからこの青年と九鬼とは何処がこんなに似ているのだろうと考えていた細木夫人は､やっとその類似点を彼女独特の方法で発見した｡
――まるで九鬼を裏がえしにしたような青年だ｡
このように､彼等が偶然出会い､そして彼等自身すら思いもよらない速さで相手を互に理解し合ったのは､その見えない媒介者が或は死であったからかも知れないのだ｡


河野扁理には､細木夫人の発見したように､どこかに九鬼を裏がえしにしたという風がある｡
容貌の点から言うと彼にはあまり九鬼に似たところがない｡むしろ対蹠的と言っていい位なものだ｡だが､その対蹠がかえって或る人々には彼等の精神的類似を目立たせるのだ｡
九鬼はこの少年を非常に好きだったらしい｡それがこの少年をして彼の弱点を速かに理解させたのであろう｡九鬼は自分の気弱さを世間に見せまいとしてそれを独特な皮肉でなければ現わすまいとした人だった｡九鬼はそれになかば成功したと言っていい｡だが､彼自身の心の中に隠すことが出来れば出来るほど､その気弱さは彼にはますます堪え難いものになって行った｡扁理はそういう不幸を目の前に見ていた｡そして九鬼と同じような気弱さを持っていた扁理は､そこで彼とは反対に､そういう気弱さを出来るだけ自分の表面に持ち出そうとしていた｡彼がそれにどれだけ成功するかは､これからの問題だが｡――

九鬼の突然の死は､勿論､この青年の心をめちゃくちゃにさせた｡しかし､九鬼の不自然な死をも彼には極めて自然に思わせるような残酷な方法で｡


九鬼の死後､扁理はその遺族のものから頼まれて彼の蔵書の整理をしだした｡
毎日､黴臭い書庫の中にはいったきり､彼は根気よくその仕事をしていた｡この仕事は彼の悲しみに気に入っているようだった｡
或る日､彼は一冊の古びた洋書の間に､何か古い手紙の切れっぱしのようなものの挟まってあるのを発見した｡彼はそれを女の筆跡らしいと思った｡そしてそれを何気なく読んだ｡もう一度読みかえした｡それからそれを注意深く元の場所にはさんで､なるたけ奥の方にその本を入れて置いた｡覚えておくためにその表紙を見たら､それはﾒﾘﾒの書簡集だった｡
それからしばらく､彼は口癖のように繰り返していた｡
――どちらが相手をより多く苦しますことが出来るか､私たちは試して見ましょう……


夕方になると､扁理は自分のｱﾊﾟｱﾄﾒﾝﾄに帰える｡
彼の部屋は実によく散らかっている｡それは彼が毎日九鬼の書庫を整理するのと同じような根気よさで､散らかしたもののように見える｡――或る日､彼がその部屋へはいって行くと､新聞とか雑誌とかﾈｸﾀｲとか薔薇とかﾊﾟｲﾌﾟなどの堆積の上に､丁度水たまりの上に浮んだ石油のように､虹色になって何かが浮んでいるのを彼は発見した｡
それは､よく見ると､一つの美しい封筒だった｡裏がえすと細木と書いてあった｡そしてその筆跡は彼にすぐこの間のﾒﾘﾒ書簡集のなかに発見した古手紙のそれを思い出させた｡
彼は丁寧に封筒を切りながら､ひょいと老人のような微笑を浮べた｡何も彼も知っているんだと言った風な……
――扁理はそんな風に二通りの微笑を使い分けるのだ｡子供のような微笑と老人のような微笑と｡つまり､他人に向ってするのと自分に向ってするのとを区別していたのだ｡
そしてそういう微笑のために､彼は自分の心を複雑なのだと信じていた｡


扁理にとって､細木夫人との二度目の面会が､その前のときよりもずっと深い心の状態においてなされたのは､そういうｴﾋﾟｿｵﾄﾞのためだった｡細木夫人の部屋は､彼の夢とは異って､装飾などもすこぶる質素だった｡決してｲｷﾞﾘｽ風でも､巴里風でもなかった｡そしてそれは彼に何となく一等船室のｻﾛﾝを思わせた｡
ときどき彼が船暈を感じている人のような眼ざしを夫人の上に投げるのに注意するがいい｡
だが扁理の心理をそんなに不安にさせているのは､そういう環境のためばかりではなしに､細木夫人とともに故人の思い出を語りながら､たえず相手の気持について行こうとして､出来るだけ自分の年齢の上に背伸びをしているためでもあったのだ｡
――この人もまた九鬼を愛していたのにちがいない､九鬼がこの人を愛していたように｡と扁理は考えた｡しかしこの人の硬い心は彼の弱い心を傷つけずにそれに触れることが出来なかったのだ｡丁度ﾀﾞｲｱﾓﾝﾄﾞが硝子に触れるとそれを傷つけずにはおかないように｡そしてこの人もまた自分で相手につけた傷のために苦しんでいる……
そういう考えがたえず扁理を彼の年齢の達することのできない処に持ち上げようとしていたのだ｡
――やがて､ひとりの十七八の少女が客間のなかに入ってくるのを彼は見た｡
彼はそれが夫人の娘の絹子であることを知った｡その少女は彼女の母にまだあんまり似ていなかった｡それが彼に何となくその少女を気に入らなく思わせた｡
彼は自分のいまの気持からは十七八の少女はあんまり離れ過ぎているように思った｡彼はその少女の顔よりも彼女の母のそれの方をもっと新鮮に見出した｡
絹子の方でもまた､少女特有の敏感さによって､扁理の気持が彼女から遠くにあることを見抜いたらしかった｡彼女は黙ったまま､二人の会話にはいろうとしなかった｡
彼女の母はすぐそれに気づいた｡そして彼女の微妙な心づかいがそれをそのままにしておくことを許さなかった｡彼女は母らしい注意をしながら､その二人をもっと近づけようとした｡
彼女はそれとなく扁理に娘の話をしだした｡――或る日､絹子は学校友だちに誘われるままに初めて本郷の古本屋というものに入ってみたという｡彼女がふとそこにあったﾗﾌｧｴﾛの画集を手にとって見ると､その扉には九鬼という蔵書印がしてあった｡そして彼女はそれを非常に欲しがっていた……
突然､扁理が遮った｡
｢それは僕の売ったものかも知れません｣
夫人たちは驚いて彼を見上げた｡すると彼は例の特有の無邪気な微笑を見せながらつけ加えた｡
｢九鬼さんにずっと前に貰ったのを､あの方の亡くなられる四五日前に､どうにも仕様がなくなって売ってしまったんです｡今になってたいへん後悔しているんですけれども……｣
そういう自分の貧しさをどうしてこういう豊かな夫人たちの前で告白するような気になったのか､扁理自身にもよく分らなかった｡だが､この告白は何となく彼の気に入った｡彼は自分の思いがけない率直な言葉によって､夫人たちがひどく驚いているらしいのを､むしろ満足そうに眺めた｡
そうして扁理自身もまた､自分自身の子供らしい率直さにいつか驚き出した……


それまで彼の夢にしか過ぎなかった細木家というものが､急に一つの現実となって扁理の生活の中にはいってきた｡
扁理はそれを九鬼やなんかの思い出といっしょくたに､新聞､雑誌､ﾈｸﾀｲ､薔薇､ﾊﾟｲﾌﾟなどの混雑のなかに､無造作に放り込んでおいた｡
そういう乱雑さをすこしも彼は気にしなかった｡むしろそれに､彼自身に最もふさわしい生活様式を見出していたのだ｡

或る晩､彼の夢のなかで､九鬼が大きな画集を彼に渡した｡そのなかの一枚の画をさしつけながら､
｢この画を知っているか?｣
｢ﾗﾌｧｴﾛの聖家族でしょう｣
と彼は気まり悪そうに答えた｡それがどうやら自分の売りとばした画集らしい気がしたのだ｡
｢もう一度､よく見てみたまえ｣と九鬼が言った｡
そこで彼はもう一ぺんその画を見直した｡すると､どうもﾗﾌｧｴﾛの筆に似てはいるが､その画のなかの聖母の顔は細木夫人のようでもあるし､幼児のそれは絹子のようでもあるので､へんな気がしながら､なおよく他の天使たちを見ようとしていると､
｢わからないのかい?｣と九鬼は皮肉な笑い方をした……
扁理は目をさました｡見ると､散らかった自分の枕もとに､見おぼえのある､立派な封筒が一つ落ちているのだ｡
おや､まだ夢の続きを見ているのかしら……と思いながら､それでもいそいでその封を切って見ると､手紙の中の文句は明瞭だった｡ﾗﾌｧｴﾛの画集を買い戻しなさいと言うのだ｡そしてそれと一しょになって一枚の為替が入っていた｡
彼はﾍﾞｯﾄﾞの中で再び眼をつぶった｡自分はまだ夢の続きを見ているのだと自分自身に言ってきかせるかのように｡


その日の午後､細木家を訪れた扁理は大きなﾗﾌｧｴﾛの画集をかかえていた｡
｢まあ､わざわざ持っていらっしったんですか｡あなたのところに置いておけばおよろしかったのに｣
そう言いながらも､夫人はそれをすぐ受取った｡そうして籐椅子に腰かけながら､しずかにそれを一枚一枚めくっていった……と思うと､突然､それを荒あらしい動作で自分の顔のところに持ち上げた｡そしてその本のにおいでも嗅いでいるらしい｡
｢なんだか莨のにおいがいたしますわ｣
扁理は驚いて夫人を見上げた｡咄嗟に九鬼が非常に莨好きだったことを思い出しながら｡そうして彼は夫人の顔が気味悪いくらいに蒼ざめているのに気づいた｡
｢この人の様子にはどこかしら罪人と云った風があるな｣と扁理は考えた｡
その時､庭の中から絹子が彼に声をかけた｡
｢庭をごらんになりません?｣
彼は夫人をそのまま一人きりにさせて置く方が彼女の気に入るだろうと考えながら､ひっそりとした庭のなかへ絹子のあとについて行った｡
少女は､扁理を自分のうしろに従えながら､庭の奥の方へはいって行けば行くほど､へんに歩きにくくなり出した｡彼女はそれを自分のうしろにいる扁理のためだとは気づかなかった｡そして少女のみが思いつき得るような単純な理由を発見した｡彼女は扁理をふりかえりながら言った｡
｢このへんに野薔薇がありますから､踏むと危のうございますわ｣
野薔薇に花が咲いているには季節があまり早すぎた｡そして扁理には､どれが野薔薇だか､その葉だけでは見わけられないのだ｡彼もまた､いつのまにか不器用に歩き出していた｡


絹子は､自分では少しも気づかなかったが､扁理に初めて会った時分から､少しずつ心が動揺しだしていた｡――扁理に初めて会った時分からではすこし正確ではない｡それはむしろ九鬼の死んだ時分からと言い直すべきかも知れない｡
それまで絹子はもう十七であるのに､いまだに死んだ父の影響の下に生きることを好んでいた｡そして彼女は自分の母のﾀﾞｲｱﾓﾝﾄﾞ属の美しさを所有しようとはせずに､それを眺め､そしてそれを愛する側にばかりなっていた｡
ところが､九鬼の死によって自分の母があんまり悲しそうにしているのを､最初はただ思いがけなく思っていたに過ぎなかったが､いつかその母の女らしい感情が彼女の中にまだ眠っていた或る層を目ざめさせた｡その時から彼女は一つの秘密を持つようになった｡しかし､それが何であるかを知ろうとはせずに｡――そして､それからというもの､彼女は知らず識らず自分の母の眼を通して物事を見るような傾向に傾いて行きつつあった｡
そして彼女はいつしか自分の母の眼を通して扁理を見つめだした｡もっと正確に言うならば､彼の中に､母が見ているように､裏がえしにした九鬼を｡
しかし彼女自身は､そういうすべてを殆ど意識していなかったと言っていい｡


そのうち一度､扁理が彼女の母の留守に訪ねて来たことがある｡
扁理はちょっと困ったような顔をしていたが､それでも絹子にすすめられるまま､客間に腰を下してしまった｡
あいにく雨が降っていた｡それでこの前のように庭へ出ることもできないのだ｡
二人は向い合って坐っていたが､別に話すこともなかったし､それに二人はお互に､相手が退屈しているだろうと想像することによって､自分自身までも退屈しているかのように感じていた｡
そうして二人は長い間､へんに息苦しい沈黙のなかに坐っていた｡
しかし二人は室内の暗くなったことにも気のつかないくらいだった｡――そんなに暗くなっていることに初めて気がつくと､驚いて扁理は帰って行った｡
絹子はそのあとで､何だか頭痛がするような気がした｡彼女はそれを扁理との退屈な時間のせいにした｡だが､実は､それは薔薇のそばにあんまり長く居過ぎたための頭痛のようなものだったのだ｡


そういう愛の最初の徴候は､絹子と同じように､扁理にも現われだした｡
自分の乱雑な生き方のおかげで､扁理はその徴候をば単なる倦怠のそれと間違えながら､それを女達の硬い性質と自分の弱い性質との差異のせいにした｡そして｢ﾀﾞｲｱﾓﾝﾄﾞは硝子を傷ける｣という原理を思い出して､自分もまた九鬼のように傷つけられないうちに､彼女たちから早く遠ざかってしまった方がいいと考えた｡そして彼は彼独特の言い方で自分に向って言った｡――自分を彼女たちに近づけさせたところの九鬼の死そのものが､今度は逆に自分を彼女たちから遠ざけさせるのだと｡
そしてそういう驚くほど簡単な考え方で彼女たちから遠ざかりながら､扁理は再び自分の散らかった部屋のなかに閉じこもって､自分一人きりで生きようとした｡すると今度は､その閉じ切った部屋の中から､本当の倦怠が生れ出した｡しかし扁理自身はその本物も贋物もごっちゃにしながら､ただ､そういうものから自分を救い出してくれるような一つの合図しか待っていなかった｡
一つの合図｡それはｶｼﾞﾉの踊り子たちに夢中になっている彼の友人たちから来た｡
或る晩､扁理は友人たちと一しょにｺｯｸ場のような臭いのするｶｼﾞﾉの楽屋廊下に立ちながら､踊り子たちを待っていた｡
彼はすぐ一人の踊り子を知った｡
その踊り子は小さくて､そんなに美しくなかった｡そして一日十幾回の踊りにすっかり疲れていた｡だが､その自棄気味で､陽気そうなところが､扁理の心をひきつけた｡彼はその踊り子に気に入るために出来るだけ自分も陽気になろうとした｡
しかし踊り子の陽気そうなのは､彼女の悪い技巧にすぎなかった｡彼女もまた彼と同じくらいに臆病だった｡が､彼女の臆病は､人に欺かれまいとするあまりに人を欺こうとする種類のそれだった｡
彼女は扁理の心を奪おうとして､他のすべての男たちとふざけ合った｡そして彼を自分から離すまいとして､彼と約束して置きながら､わざと彼を待ち呆けさせた｡
一度､扁理が踊り子の肩に手をかけようとしたことがある｡すると踊り子はすばやくその手から自分の肩を引いてしまった｡そして彼女は､扁理が顔を赤らめているのを見ながら､彼の心を奪いつつあると信じた｡
こういう二人の気の小さな恋人同志がどうして何時までもうまくやって行けるだろうか?
或る日､彼は公園の噴水のほとりで踊り子を待っていた｡彼女はなかなかやって来ない｡それには慣れているから彼はそれをそれほど苦痛には感じない｡が､そのうちふと､踊り子とは別の少女――絹子のことを彼は考え出した｡そして若しいま自分の待っているのがその踊り子ではなくて､あの絹子だったらどんなだろうと空想した｡……が､その莫迦げた空想にすぐ自分で気がついて､彼はそれを踊り子のための現在の苦痛から回避しようとしている自分自身のせいにした｡

扁理の乱雑な生活のなかに埋もれながら､なお絶えず成長しつつあった一つの純潔な愛が､こうしてひょっくりその表面に顔を出したのだ｡だが､それは彼に気づかれずに再び引込んで行った……


絹子はといえば､扁理が自分たちから遠ざかって行くのを､最初のうちは何かほっとした気持で見送っていた｡が､それが或る限度を越え出すと､今度は逆にそれが彼女を苦しめ出した｡しかし､それが扁理に対する愛からであることを認めるには､少女の心はあまりに硬過ぎた｡
細木夫人の方は､扁理がこうして遠ざかって行くのを､むしろ､彼に訪問の機会を与えてやらない自分自身の過失のように考えていた｡しかし夫人には扁理を見ることは楽しいことよりも､むしろ苦しいことの方が多かった｡そうして月日が九鬼の死を遠ざければ遠ざけるほど､彼女に欲しいのは平静さだけであった｡だから､彼女は扁理がだんだん遠ざかって行くのを見ても､それをそのままにして置いたのだ｡
或る朝､二人は公園のなかに自動車をﾄﾞﾗｲｳﾞさせていた｡
噴水のほとりに､扁理が一人の小さい女と歩いているのを､彼女たちが見つけたのはほとんど同時だった｡その小さい女は黄と黒の縞の外套をきていて､何か快活そうに笑っていた｡それと並んで扁理は考え深そうにうつむきながら歩いていた｡
｢あら!｣と絹子が車の中でかすかに声を立てた｡
と同時に彼女は､彼女の母がもしかしたら扁理たちに気づかなかったかも知れないと思った｡そうして彼女自身もそれに気づかなかったような風をしようとした｡
｢なんだか目の中にｺﾞﾐがはいっちゃったわ……｣
夫人は夫人でまた､絹子が扁理たちを見なかったことを､ひそかに欲していた｡そうして､ほんとうに目の中にｺﾞﾐかなんか入って彼等を見なかったのかも知れないと思った｡
｢びっくりしたじゃないの……｣
そう言って､夫人は自分の心持蒼くなっている顔をごまかした｡


その沈黙はしかし､二人の間にながく尾をひいた｡
それからというもの､絹子はよく一人で町へ散歩に出かけた｡彼女は心の中のうっとうしさを運動不足のせいにしていたのだ｡そうして母からも離れて一人きりになりたい気持や､こうして歩いているうちにまたひょっとしたら扁理に会えるかも知れないという考えなどの彼女にあったことは､少しも自分で認めようとはしなかった｡
彼女は扁理とその恋人らしいものの姿を､下手な写真師のように修整していた｡その写真のなかでは､例の小さい踊り子は彼女と同じような上流社会の立派な令嬢に仕上げられていた｡
彼女はそういう扁理たちに対して何とも云えないにがさを味った｡しかし､それが扁理のための嫉妬であることには､勿論､彼女は気づかなかった｡何故なら､彼女は扁理たちのような年輩のどういう二人づれを見てもその同じようなにがさを味ったからだ｡そして彼女はそれを世間一般の恋人たちに対するにがさであると信じた｡――実は､彼女はどういう二人づれを見ても知らず識らず扁理たちを思い出していたのだが……
彼女は歩きながら､飾窓に映る自分の姿を見つめた｡そうして彼女は､いますれちがったばかりの二人づれに自分を比較した｡ときどき硝子の中の彼女は妙に顔をゆがめていた｡彼女はそれを悪い硝子のせいにした｡


或る日､そういう散歩から帰ってくると､絹子は玄関にどこか見おぼえのある男の帽子と靴とを見出した｡
そうしてそれが誰のだかはっきり思い出せないことが､彼女をちょっと不安にさせた｡
｢誰かしら｣
と思いながら､彼女が客間に近づいて行ってみると､その中から､こわれたｷﾞﾀｱのような声が聞えてきた｡
それは斯波という男の声であった｡
斯波という男は､――｢あいつはまるで壁の花みたいな奴ですよ｡そら､舞踏会で踊れないもんだから､壁にばかりくっついている奴がよくあるでしょう｡そういう奴のことを英語で Wall Flower というんだそうだけれど……斯波の人生における立場なんか全くそれですね｣――そんなことをいつか扁理が言っていたのを思い出しながら､それから､彼女はふと扁理のことを考えた｡……
彼女が客間に入って行くと､斯波は急に話すのを歇めた｡
が､すぐ､斯波は､例のこわれたｷﾞﾀｱのような声で､彼女に向って言いだした｡
｢いま､扁理の悪口を言っていたところなんですよ｡あいつはこの頃全く手がつけられなくなったんです｡くだらない踊り子かなんかに引っかかっていて……｣
｢あら､そうですの｣
絹子はそれを聞くと同時ににっこりと笑った｡いかにも朗らかそうに｡そして自分でも笑いながら､こんな風に笑ったのは実にひさしぶりであるような気がした｡
このながく眠っていた薔薇を開かせるためには､たった一つの言葉で充分だったのだ｡それは踊り子の一語だ｡扁理と一しょにいた人はそんな人だったのか､と彼女は考え出した｡私はそれを私と同じような身分の人とばかり考えていたのに｡そしてそういう人だけしか扁理の相手にはなれないと思っていたのに｡……そうだわ､きっと扁理はそんな人なんか愛していないのかも知れない｡もしかすると､あの人の愛しているのはやっぱし私なのかも知れない｡それだのに私があの人を愛していないと思っているので､私から遠ざかろうとしているのではないかしら｡そうして自分をごまかすためにきっとそんな踊り子などと一しょに暮らしているのだ｡そんな人なんかあの人には似合わないのに……
それは少女らしい驕慢な論理だった｡しかし､大抵の場合､少女は自分自身の感情はその計算の中に入れないものだ｡そして絹子の場合もそうだった｡


ときどき鳴りもしないのにﾍﾞﾙの音を聞いたような気がして自分で玄関に出て行ったり､器械がこわれていてﾍﾞﾙが鳴らないのかしらと始終思ったりしながら､絹子はたえず何かを待っていた｡
｢扁理を待っているのかしら?｣ふと彼女はそんなことを考えることもあったが､そんな考えはすぐ彼女の不浸透性の心の表面を滑って行った｡
或る晩､ﾍﾞﾙが鳴った｡――その訪問者が扁理であることを知っても､絹子は容易に自分の部屋から出て行こうとしなかった｡
やっと彼女が客間にはいって行くと､扁理は､帽子もかぶらずに歩いていたらしく､毛髪をくしゃくしゃにさせながら､青い顔をして､ちらりと彼女の方をにらんだ｡それきり彼は彼女の方をふりむきもしなかった｡
細木夫人は､そういう扁理を前にしながら､手にしている葡萄の皿から､その小さい実を丹念に口の中へ滑り込ましていた｡夫人は目の前の扁理のだらしのない様子から､ふと､九鬼の告別式の日に途中で彼に出会った時のことを思い出し､それからそれへと様々なことが考えられてならないのだが､彼女はそれから出来るだけ心をそらそうとして､一そう丹念に自分の指を動かしていた｡
突然､扁理が言った――
｢僕､しばらく旅行して来ようと思います｣
｢どちらへ?｣夫人は葡萄の皿から眼を上げた｡
｢まだはっきり決めてないんですが……｣
｢ながくですの?｣
｢ええ､一年ぐらい……｣
夫人はふと､扁理が､例の踊り子と一しょにそんなところへ行くのではないかと疑いながら､
｢淋しくはありませんか｣と訊いた｡
｢さあ……｣
扁理はいかにも気のない返事をしたきりだった｡
絹子はといえば､その間黙ったまま､彼の肖像でも描こうとするかのように､熱心に彼を見つめていた｡
そうして彼女の母が､扁理の､梳らない毛髪や不恰好に結んだﾈｸﾀｲや悪い顔色などのなかに､踊り子の感化を見出している間､絹子はその同じものの中に彼女自身のために苦しんでいる青年の痛々しさだけしか見出さなかった｡


扁理が帰った後､絹子は自分の部屋にはいるなり､思わず眼をつぶった｡さっきあんまり扁理の赤い縞のあるﾈｸﾀｲを見つめ過ぎたので､眼が痛むのだ｡するとその閉じた眼の中には､いつまでも赤い縞のようなものがﾁﾗﾁﾗしていた……


扁理は出発した｡
都会が遠ざかり､そしてそれが小さくなるのを見れば見るほど､彼には出発前に見てきた一つの顔だけが次第に大きくなって行くように思われた｡
一つの少女の顔｡ﾗﾌｧｴﾛの描いた天使のように聖らかな顔｡実物よりも十倍位の大きさの一つの神秘的な顔｡そしていま､それだけがあらゆるものから孤立し､膨大し､そしてその他のすべてのものを彼の目から覆い隠そうとしている……
｢おれのほんとうに愛しているのはこの人かしら?｣
扁理は目をつぶった｡
｢……だが､もうどうでもいいんだ……｣
そんなにまで彼は疲れ､傷つき､絶望していた｡
扁理｡――この乱雑の犠牲者には今まで自分の本当の心が少しも見分けられなかったのだ｡そして何の考えもなしに自分のほんとうに愛しているものから遠ざかるために､別の女と生きようとし､しかもその女のために､もうどうしていいか分らないくらい､疲れさせられてしまっているのだ｡
そうして彼はいま何処へ到着しようとしているのか?
何処へ?……
彼は突然､汽車が一つの停車場に停まると同時に､慌ててそこへ飛び降りてしまった｡
それは何かの薬品の名を思い出させるような名前の､小さな海辺の町であった｡
そしてこの一個のﾄﾗﾝｸすら持たぬ悲しげな旅行者は､停車場を出ると､すぐその見知らない町の中へ何の目的もなしに足を運んで行った｡
彼はしかし歩いてゆくうちに､ふと変な気がしだした｡……通行人の顔､風が気味わるく持ち上げている何かのﾋﾞﾗ､何とも言えず不快な感じのする壁の上の落書､電線にひっかかっている紙屑のようなもの､――そういうものが彼になにかしら不吉な思い出を強請するのだ｡扁理は或る小さなﾎﾃﾙにはいり､それから見知らない一つの部屋にはいった｡あらゆるﾎﾃﾙの部屋に似ている一つの部屋｡しかし､それすら彼に何かを思い出させようとし､彼を苦しめ出すのだ｡彼は疲れていて非常に眠かった｡そして彼はそのすべてを自分の疲れと眠たさのせいにしょうとした｡彼はすこし眠った｡……目をさますと､もう暗くなっていた｡窓から入ってくる､湿っぽい風が扁理に､自分が見知らない町に来ていることを知らせた｡彼は起き上り､それから再びﾎﾃﾙを出た｡
そうしてまた､さっき一度歩いたことのある道を歩きながら､あの時から少しも失われていない自分のなかの不可解な感じを､犬のように追いかけて行った｡
突然､或る考えが扁理にすべてを理解させ出したように見える｡さっきから自分をこうして苦しめているもの､それは死の暗号ではないのか｡通行人の顔､ﾋﾞﾗ､落書､紙屑のようなもの､それらは死が彼のために記して行った暗号ではないのか｡どこへ行ってもこの町にこびりついている死の印｡――それは彼には同時に九鬼の影であった｡そうして彼にはどうしてだか､九鬼が数年前に一度この町へやってきて､今の自分と同じように誰にも知られずに歩きながら､やはり今の自分と同じような苦痛を感じていたような気がされてならないのだ……
そうして扁理はようやく理解し出した､死んだ九鬼が自分の裏側にたえず生きていて､いまだに自分を力強く支配していることを､そしてそれに気づかなかったことが自分の生の乱雑さの原因であったことを｡
そうしてこんな風に､すべてのものから遠ざかりながら､そしてただ一つの死を自分の生の裏側にいきいきと､非常に近くしかも非常に遠く感じながら､この見知らない町の中を何の目的もなしに歩いていることが､扁理にはいつか何とも言えず快い休息のように思われ出した｡
――そのうちに扁理は､強い香りのする､夥しい漂流物に取りかこまれながら､うす暗い海岸に愚かそうに突立っている自分自身を発見した｡そうして自分の足もとに散らばっている貝殻や海草や死んだ魚などが､彼に､彼自身の生の乱雑さを思い出させていた｡――その漂流物のなかには､一ぴきの小さな犬の死骸が混っていた｡そうしてそれが意地のわるい波にときどき白い歯で噛まれたり､裏がえしにされたりするのを､扁理はじっと見入りながら､次第にいきいきと自分の心臓の鼓動するのを感じ出していた……



扁理の出発後､絹子は病気になった｡
そうして或る日､彼女はとうとう始めて扁理への愛を自白した｡彼女は寝台の上で､ｼｲﾂのように青ざめた顔をしながら､こんなことを繰り返えし繰り返えし考えていた｡
――何故私はああだったのかしら｡何故私はあの人の前で意地のわるい顔ばかりしていたのかしら｡それがきっとあの人を苦しめていたのだわ｡そうしてこんな風に私たちから遠ざからせてしまったのにちがいない｡それに､あの人は始終自分の貧乏なことを気にしていたようだけれど……(そんな考えがさっと少女の頬を赤らめた)……それで､あの人は私のお母さんに誘惑者のように思われたくなかったのかも知れない｡あの人が私のお母さんを怖れていたことはそれは本当だわ｡こんな風にあの人を遠ざからせてしまったのはお母さんだって悪いんだ｡私のせいばかりではない｡ひょっとしたら何もかもお母さんのせいかも知れない……
そんな風にこんぐらかった独語が､娘の顔の上にいつのまにか､十七の少女に似つかわしくないような､にがにがしげな表情を雕りつけていた｡それは実に彼女自身への意地であったのだけれども､彼女には､それを彼女の母への意地であるかのように誤って信じさせながら……


｢はいってもよくって?｣
そのとき部屋の外で母の声がした｡
｢いいわ｣
絹子は､彼女の母がはいって来るのを見ると､いきなり自分の狂暴な顔を壁の方にねじむけた｡細木夫人はそれを彼女が涙をかくすためにしたのだとしか思わなかった｡
｢河野さんから絵はがきが来たのよ｣と夫人はおどおどしながら言った｡
その言葉が絹子の顔を夫人の方にねじむけさせた｡今度は夫人がそれから自分の顔をそむかせる番だった｡
――この頃､細木夫人はすっかり若さを失っていた｡そして彼女には､自分の娘が何んだか自分から遠くに離れてしまったように思われてならないのだった｡彼女はときどき自分の娘を､まるで見知らない少女のようにさえ思うことがあった｡そして今も､そうだった……
絹子は､海の絵はがきの裏に､鉛筆で書かれた扁理の神経質な字を読んだ｡彼は､その海岸が気に入ったからしばらく滞在するつもりだ､と書いて寄こしたきりだった｡
絹子はその絵はがきから､彼女の狂暴な顔をいきなり夫人の方にむけながら､
｢河野さんは死ぬんじゃなくって?｣と出しぬけに質問した｡
細木夫人はその瞬間､自分の方を睨んでいる､一人の見知らぬ少女の､そんなにも恐い眼つきに驚いたようだった｡が､その少女のそんな眼つきは突然､夫人に､彼女がその少女と同じくらいの年齢であった時分､彼女の愛していた人に見せつけずにはいられなかった自分の恐い眼つきを思い出させた｡そうして夫人は､その見知らない少女がその頃の自分にひどく肖ていることに､そして､その少女が実は自分の娘であることに､なんだか始めて気づいたかのように見えた｡夫人は溜息をしずかに洩らした｡――娘は誰かを愛している｡自分が､昔､あの人を愛していたように愛している｡そしてそれはきっと扁理にちがいない……
細木夫人は､しかし次の瞬間､自分のなかに長いこと眠っていた女らしい感情が､再び目ざめだしたように感じた｡九鬼の死後､彼女の苦しんでいた様子が､絹子の中にそれまで眠っていた女らしい感情を喚び起したのとまったく同じの心理作用が､今度は､その反作用ででもあるかのように起ったのだ｡そしてそれは､夫人もまた絹子と同じように扁理を愛しているかのように､彼女に信じさせたくらいの新鮮さで｡――
二人はそのまましばらく黙っていた｡そしてその沈黙が､絹子の今しがた言った恐しい言葉を､そっくりそのまま肯定しているかのように思われそうになった時､細木夫人はようやく自分の母としての義務を取り戻した｡
そうして夫人はいかにも自信ありげな微笑を浮べながら､答えたのである｡
｢……そんなことはないことよ……それはあの方には九鬼さんが憑いていなさるかも知れないわ｡けれども､そのために反ってあの方は救われるのじゃなくって?｣
河野扁理にはじめて会った時から､夫人に､彼の生のなかには九鬼の死が緯のように織りまざっていることを､そしてそれが彼をして死に見入ることによって生がようやく分るような不幸な青年にさせていることを見抜かせたところの､一種の鋭い直覚が､いま再び彼女のなかに蘇って来ながら､そういう扁理の不幸を絹子に理解させるためには､いま言ったようなごく簡単な逆説だけで充分であることを彼女に知らせたのだ｡
｢そうかしら……｣
絹子はそう答えながら､始めはまだ何処かしら苦痛をおびた表情で､彼女の母の顔を見あげていたけれども､そのうちにじっとその母の古びた神々しい顔に見入りだしたその少女の眼ざしは､だんだんと古画のなかで聖母を見あげている幼児のそれに似てゆくように思われた｡"""
},

{
"@search.action": "upload",
"BookId": "753",
"BookName": "文鳥",
"BookAuthor": "夏目漱石",
"BookGenre": "短編小説",
"Release_Date": "1908年6月",
"Rating": "4.38",
"Text": """十月早稲田に移る｡伽藍のような書斎にただ一人､片づけた顔を頬杖で支えていると､三重吉が来て､鳥を御飼いなさいと云う｡飼ってもいいと答えた｡しかし念のためだから､何を飼うのかねと聞いたら､文鳥ですと云う返事であった｡
文鳥は三重吉の小説に出て来るくらいだから奇麗な鳥に違なかろうと思って､じゃ買ってくれたまえと頼んだ｡ところが三重吉は是非御飼いなさいと､同じような事を繰り返している｡うむ買うよ買うよとやはり頬杖を突いたままで､むにゃむにゃ云ってるうちに三重吉は黙ってしまった｡おおかた頬杖に愛想を尽かしたんだろうと､この時始めて気がついた｡
すると三分ばかりして､今度は籠を御買いなさいと云いだした｡これも宜しいと答えると､是非御買いなさいと念を押す代りに､鳥籠の講釈を始めた｡その講釈はだいぶ込み入ったものであったが､気の毒な事に､みんな忘れてしまった｡ただ好いのは二十円ぐらいすると云う段になって､急にそんな高価のでなくっても善かろうと云っておいた｡三重吉はにやにやしている｡
それから全体どこで買うのかと聞いて見ると､なにどこの鳥屋にでもありますと､実に平凡な答をした｡籠はと聞き返すと､籠ですか､籠はその何ですよ､なにどこにかあるでしょう､とまるで雲を攫むような寛大な事を云う｡でも君あてがなくっちゃいけなかろうと､あたかもいけないような顔をして見せたら､三重吉は頬ぺたへ手をあてて､何でも駒込に籠の名人があるそうですが､年寄だそうですから､もう死んだかも知れませんと､非常に心細くなってしまった｡
何しろ言いだしたものに責任を負わせるのは当然の事だから､さっそく万事を三重吉に依頼する事にした｡すると､すぐ金を出せと云う｡金はたしかに出した｡三重吉はどこで買ったか､七子の三つ折の紙入を懐中していて､人の金でも自分の金でも悉皆この紙入の中に入れる癖がある｡自分は三重吉が五円札をたしかにこの紙入の底へ押し込んだのを目撃した｡
かようにして金はたしかに三重吉の手に落ちた｡しかし鳥と籠とは容易にやって来ない｡
そのうち秋が小春になった｡三重吉はたびたび来る｡よく女の話などをして帰って行く｡文鳥と籠の講釈は全く出ない｡硝子戸を透して五尺の縁側には日が好く当る｡どうせ文鳥を飼うなら､こんな暖かい季節に､この縁側へ鳥籠を据えてやったら､文鳥も定めし鳴き善かろうと思うくらいであった｡
三重吉の小説によると､文鳥は千代千代と鳴くそうである｡その鳴き声がだいぶん気に入ったと見えて､三重吉は千代千代を何度となく使っている｡あるいは千代と云う女に惚れていた事があるのかも知れない｡しかし当人はいっこうそんな事を云わない｡自分も聞いてみない｡ただ縁側に日が善く当る｡そうして文鳥が鳴かない｡
そのうち霜が降り出した｡自分は毎日伽藍のような書斎に､寒い顔を片づけてみたり､取乱してみたり､頬杖を突いたりやめたりして暮していた｡戸は二重に締め切った｡火鉢に炭ばかり継いでいる｡文鳥はついに忘れた｡
ところへ三重吉が門口から威勢よく這入って来た｡時は宵の口であった｡寒いから火鉢の上へ胸から上を翳して､浮かぬ顔をわざとほてらしていたのが､急に陽気になった｡三重吉は豊隆を従えている｡豊隆はいい迷惑である｡二人が籠を一つずつ持っている｡その上に三重吉が大きな箱を兄き分に抱えている｡五円札が文鳥と籠と箱になったのはこの初冬の晩であった｡
三重吉は大得意である｡まあ御覧なさいと云う｡豊隆その洋灯をもっとこっちへ出せなどと云う｡そのくせ寒いので鼻の頭が少し紫色になっている｡
なるほど立派な籠ができた｡台が漆で塗ってある｡竹は細く削った上に､色が染けてある｡それで三円だと云う｡安いなあ豊隆と云っている｡豊隆はうん安いと云っている｡自分は安いか高いか判然と判らないが､まあ安いなあと云っている｡好いのになると二十円もするそうですと云う｡二十円はこれで二返目である｡二十円に比べて安いのは無論である｡
この漆はね､先生､日向へ出して曝しておくうちに黒味が取れてだんだん朱の色が出て来ますから､――そうしてこの竹は一返善く煮たんだから大丈夫ですよなどと､しきりに説明をしてくれる｡何が大丈夫なのかねと聞き返すと､まあ鳥を御覧なさい､奇麗でしょうと云っている｡
なるほど奇麗だ｡次の間へ籠を据えて四尺ばかりこっちから見ると少しも動かない｡薄暗い中に真白に見える｡籠の中にうずくまっていなければ鳥とは思えないほど白い｡何だか寒そうだ｡
寒いだろうねと聞いてみると､そのために箱を作ったんだと云う｡夜になればこの箱に入れてやるんだと云う｡籠が二つあるのはどうするんだと聞くと､この粗末な方へ入れて時々行水を使わせるのだと云う｡これは少し手数が掛るなと思っていると､それから糞をして籠を汚しますから､時々掃除をしておやりなさいとつけ加えた｡三重吉は文鳥のためにはなかなか強硬である｡
それをはいはい引受けると､今度は三重吉が袂から粟を一袋出した｡これを毎朝食わせなくっちゃいけません｡もし餌をかえてやらなければ､餌壺を出して殻だけ吹いておやんなさい｡そうしないと文鳥が実のある粟を一々拾い出さなくっちゃなりませんから｡水も毎朝かえておやんなさい｡先生は寝坊だからちょうど好いでしょうと大変文鳥に親切を極めている｡そこで自分もよろしいと万事受合った｡ところへ豊隆が袂から餌壺と水入を出して行儀よく自分の前に並べた｡こういっさい万事を調えておいて､実行を逼られると､義理にも文鳥の世話をしなければならなくなる｡内心ではよほど覚束なかったが､まずやってみようとまでは決心した｡もしできなければ家のものが､どうかするだろうと思った｡
やがて三重吉は鳥籠を叮嚀に箱の中へ入れて､縁側へ持ち出して､ここへ置きますからと云って帰った｡自分は伽藍のような書斎の真中に床を展べて冷かに寝た｡夢に文鳥を背負い込んだ心持は､少し寒かったが眠ってみれば不断の夜のごとく穏かである｡
翌朝眼が覚めると硝子戸に日が射している｡たちまち文鳥に餌をやらなければならないなと思った｡けれども起きるのが退儀であった｡今にやろう､今にやろうと考えているうちに､とうとう八時過になった｡仕方がないから顔を洗うついでをもって､冷たい縁を素足で踏みながら､箱の葢を取って鳥籠を明海へ出した｡文鳥は眼をぱちつかせている｡もっと早く起きたかったろうと思ったら気の毒になった｡
文鳥の眼は真黒である｡瞼の周囲に細い淡紅色の絹糸を縫いつけたような筋が入っている｡眼をぱちつかせるたびに絹糸が急に寄って一本になる｡と思うとまた丸くなる｡籠を箱から出すや否や､文鳥は白い首をちょっと傾けながらこの黒い眼を移して始めて自分の顔を見た｡そうしてちちと鳴いた｡
自分は静かに鳥籠を箱の上に据えた｡文鳥はぱっと留り木を離れた｡そうしてまた留り木に乗った｡留り木は二本ある｡黒味がかった青軸をほどよき距離に橋と渡して横に並べた｡その一本を軽く踏まえた足を見るといかにも華奢にできている｡細長い薄紅の端に真珠を削ったような爪が着いて､手頃な留り木を甘く抱え込んでいる｡すると､ひらりと眼先が動いた｡文鳥はすでに留り木の上で方向を換えていた｡しきりに首を左右に傾ける｡傾けかけた首をふと持ち直して､心持前へ伸したかと思ったら､白い羽根がまたちらりと動いた｡文鳥の足は向うの留り木の真中あたりに具合よく落ちた｡ちちと鳴く｡そうして遠くから自分の顔を覗き込んだ｡
自分は顔を洗いに風呂場へ行った｡帰りに台所へ廻って､戸棚を明けて､昨夕三重吉の買って来てくれた粟の袋を出して､餌壺の中へ餌を入れて､もう一つには水を一杯入れて､また書斎の縁側へ出た｡
三重吉は用意周到な男で､昨夕叮嚀に餌をやる時の心得を説明して行った｡その説によると､むやみに籠の戸を明けると文鳥が逃げ出してしまう｡だから右の手で籠の戸を明けながら､左の手をその下へあてがって､外から出口を塞ぐようにしなくっては危険だ｡餌壺を出す時も同じ心得でやらなければならない｡とその手つきまでして見せたが､こう両方の手を使って､餌壺をどうして籠の中へ入れる事ができるのか､つい聞いておかなかった｡
自分はやむをえず餌壺を持ったまま手の甲で籠の戸をそろりと上へ押し上げた｡同時に左の手で開いた口をすぐ塞いだ｡鳥はちょっと振り返った｡そうして､ちちと鳴いた｡自分は出口を塞いだ左の手の処置に窮した｡人の隙を窺って逃げるような鳥とも見えないので､何となく気の毒になった｡三重吉は悪い事を教えた｡
大きな手をそろそろ籠の中へ入れた｡すると文鳥は急に羽搏を始めた｡細く削った竹の目から暖かいむく毛が､白く飛ぶほどに翼を鳴らした｡自分は急に自分の大きな手が厭になった｡粟の壺と水の壺を留り木の間にようやく置くや否や､手を引き込ました｡籠の戸ははたりと自然に落ちた｡文鳥は留り木の上に戻った｡白い首を半ば横に向けて､籠の外にいる自分を見上げた｡それから曲げた首を真直にして足の下にある粟と水を眺めた｡自分は食事をしに茶の間へ行った｡
その頃は日課として小説を書いている時分であった｡飯と飯の間はたいてい机に向って筆を握っていた｡静かな時は自分で紙の上を走るﾍﾟﾝの音を聞く事ができた｡伽藍のような書斎へは誰も這入って来ない習慣であった｡筆の音に淋しさと云う意味を感じた朝も昼も晩もあった｡しかし時々はこの筆の音がぴたりとやむ､またやめねばならぬ､折もだいぶあった｡その時は指の股に筆を挟んだまま手の平へ顎を載せて硝子越に吹き荒れた庭を眺めるのが癖であった｡それが済むと載せた顎を一応撮んで見る｡それでも筆と紙がいっしょにならない時は､撮んだ顎を二本の指で伸して見る｡すると縁側で文鳥がたちまち千代千代と二声鳴いた｡
筆を擱いて､そっと出て見ると､文鳥は自分の方を向いたまま､留り木の上から､のめりそうに白い胸を突き出して､高く千代と云った｡三重吉が聞いたらさぞ喜ぶだろうと思うほどな美い声で千代と云った｡三重吉は今に馴れると千代と鳴きますよ､きっと鳴きますよ､と受合って帰って行った｡
自分はまた籠の傍へしゃがんだ｡文鳥は膨らんだ首を二三度竪横に向け直した｡やがて一団の白い体がぽいと留り木の上を抜け出した｡と思うと奇麗な足の爪が半分ほど餌壺の縁から後へ出た｡小指を掛けてもすぐ引っ繰り返りそうな餌壺は釣鐘のように静かである｡さすがに文鳥は軽いものだ｡何だか淡雪の精のような気がした｡
文鳥はつと嘴を餌壺の真中に落した｡そうして二三度左右に振った｡奇麗に平して入れてあった粟がはらはらと籠の底に零れた｡文鳥は嘴を上げた｡咽喉の所で微な音がする｡また嘴を粟の真中に落す｡また微な音がする｡その音が面白い｡静かに聴いていると､丸くて細やかで､しかも非常に速かである｡菫ほどな小さい人が､黄金の槌で瑪瑙の碁石でもつづけ様に敲いているような気がする｡
嘴の色を見ると紫を薄く混ぜた紅のようである｡その紅がしだいに流れて､粟をつつく口尖の辺は白い｡象牙を半透明にした白さである｡この嘴が粟の中へ這入る時は非常に早い｡左右に振り蒔く粟の珠も非常に軽そうだ｡文鳥は身を逆さまにしないばかりに尖った嘴を黄色い粒の中に刺し込んでは､膨くらんだ首を惜気もなく右左へ振る｡籠の底に飛び散る粟の数は幾粒だか分らない｡それでも餌壺だけは寂然として静かである｡重いものである｡餌壺の直径は一寸五分ほどだと思う｡
自分はそっと書斎へ帰って淋しくﾍﾟﾝを紙の上に走らしていた｡縁側では文鳥がちちと鳴く｡折々は千代千代とも鳴く｡外では木枯が吹いていた｡
夕方には文鳥が水を飲むところを見た｡細い足を壺の縁へ懸けて､小い嘴に受けた一雫を大事そうに､仰向いて呑み下している｡この分では一杯の水が十日ぐらい続くだろうと思ってまた書斎へ帰った｡晩には箱へしまってやった｡寝る時硝子戸から外を覗いたら､月が出て､霜が降っていた｡文鳥は箱の中でことりともしなかった｡
明る日もまた気の毒な事に遅く起きて､箱から籠を出してやったのは､やっぱり八時過ぎであった｡箱の中ではとうから目が覚めていたんだろう｡それでも文鳥はいっこう不平らしい顔もしなかった｡籠が明るい所へ出るや否や､いきなり眼をしばたたいて､心持首をすくめて､自分の顔を見た｡
昔し美しい女を知っていた｡この女が机に凭れて何か考えているところを､後から､そっと行って､紫の帯上げの房になった先を､長く垂らして､頸筋の細いあたりを､上から撫で廻したら､女はものう気に後を向いた｡その時女の眉は心持八の字に寄っていた｡それで眼尻と口元には笑が萌していた｡同時に恰好の好い頸を肩まですくめていた｡文鳥が自分を見た時､自分はふとこの女の事を思い出した｡この女は今嫁に行った｡自分が紫の帯上でいたずらをしたのは縁談のきまった二三日後である｡
餌壺にはまだ粟が八分通り這入っている｡しかし殻もだいぶ混っていた｡水入には粟の殻が一面に浮いて､苛く濁っていた｡易えてやらなければならない｡また大きな手を籠の中へ入れた｡非常に要心して入れたにもかかわらず､文鳥は白い翼を乱して騒いだ｡小い羽根が一本抜けても､自分は文鳥にすまないと思った｡殻は奇麗に吹いた｡吹かれた殻は木枯がどこかへ持って行った｡水も易えてやった｡水道の水だから大変冷たい｡
その日は一日淋しいﾍﾟﾝの音を聞いて暮した｡その間には折々千代千代と云う声も聞えた｡文鳥も淋しいから鳴くのではなかろうかと考えた｡しかし縁側へ出て見ると､二本の留り木の間を､あちらへ飛んだり､こちらへ飛んだり､絶間なく行きつ戻りつしている｡少しも不平らしい様子はなかった｡
夜は箱へ入れた｡明る朝目が覚めると､外は白い霜だ｡文鳥も眼が覚めているだろうが､なかなか起きる気にならない｡枕元にある新聞を手に取るさえ難儀だ｡それでも煙草は一本ふかした｡この一本をふかしてしまったら､起きて籠から出してやろうと思いながら､口から出る煙の行方を見つめていた｡するとこの煙の中に､首をすくめた､眼を細くした､しかも心持眉を寄せた昔の女の顔がちょっと見えた｡自分は床の上に起き直った｡寝巻の上へ羽織を引掛けて､すぐ縁側へ出た｡そうして箱の葢をはずして､文鳥を出した｡文鳥は箱から出ながら千代千代と二声鳴いた｡
三重吉の説によると､馴れるにしたがって､文鳥が人の顔を見て鳴くようになるんだそうだ｡現に三重吉の飼っていた文鳥は､三重吉が傍にいさえすれば､しきりに千代千代と鳴きつづけたそうだ｡のみならず三重吉の指の先から餌を食べると云う｡自分もいつか指の先で餌をやって見たいと思った｡
次の朝はまた怠けた｡昔の女の顔もつい思い出さなかった｡顔を洗って､食事を済まして､始めて､気がついたように縁側へ出て見ると､いつの間にか籠が箱の上に乗っている｡文鳥はもう留り木の上を面白そうにあちら､こちらと飛び移っている｡そうして時々は首を伸して籠の外を下の方から覗いている｡その様子がなかなか無邪気である｡昔紫の帯上でいたずらをした女は襟の長い､背のすらりとした､ちょっと首を曲げて人を見る癖があった｡
粟はまだある｡水もまだある｡文鳥は満足している｡自分は粟も水も易えずに書斎へ引込んだ｡
昼過ぎまた縁側へ出た｡食後の運動かたがた､五六間の廻り縁を､あるきながら書見するつもりであった｡ところが出て見ると粟がもう七分がた尽きている｡水も全く濁ってしまった｡書物を縁側へ抛り出しておいて､急いで餌と水を易えてやった｡
次の日もまた遅く起きた｡しかも顔を洗って飯を食うまでは縁側を覗かなかった｡書斎に帰ってから､あるいは昨日のように､家人が籠を出しておきはせぬかと､ちょっと縁へ顔だけ出して見たら､はたして出してあった｡その上餌も水も新しくなっていた｡自分はやっと安心して首を書斎に入れた｡途端に文鳥は千代千代と鳴いた｡それで引込めた首をまた出して見た｡けれども文鳥は再び鳴かなかった｡けげんな顔をして硝子越に庭の霜を眺めていた｡自分はとうとう机の前に帰った｡
書斎の中では相変らずﾍﾟﾝの音がさらさらする｡書きかけた小説はだいぶんはかどった｡指の先が冷たい｡今朝埋けた佐倉炭は白くなって､薩摩五徳に懸けた鉄瓶がほとんど冷めている｡炭取は空だ｡手を敲いたがちょっと台所まで聴えない｡立って戸を明けると､文鳥は例に似ず留り木の上にじっと留っている｡よく見ると足が一本しかない｡自分は炭取を縁に置いて､上からこごんで籠の中を覗き込んだ｡いくら見ても足は一本しかない｡文鳥はこの華奢な一本の細い足に総身を託して黙然として､籠の中に片づいている｡
自分は不思議に思った｡文鳥について万事を説明した三重吉もこの事だけは抜いたと見える｡自分が炭取に炭を入れて帰った時､文鳥の足はまだ一本であった｡しばらく寒い縁側に立って眺めていたが､文鳥は動く気色もない｡音を立てないで見つめていると､文鳥は丸い眼をしだいに細くし出した｡おおかた眠たいのだろうと思って､そっと書斎へ這入ろうとして､一歩足を動かすや否や､文鳥はまた眼を開いた｡同時に真白な胸の中から細い足を一本出した｡自分は戸を閉てて火鉢へ炭をついだ｡
小説はしだいに忙しくなる｡朝は依然として寝坊をする｡一度家のものが文鳥の世話をしてくれてから､何だか自分の責任が軽くなったような心持がする｡家のものが忘れる時は､自分が餌をやる水をやる｡籠の出し入れをする｡しない時は､家のものを呼んでさせる事もある｡自分はただ文鳥の声を聞くだけが役目のようになった｡
それでも縁側へ出る時は､必ず籠の前へ立留って文鳥の様子を見た｡たいていは狭い籠を苦にもしないで､二本の留り木を満足そうに往復していた｡天気の好い時は薄い日を硝子越に浴びて､しきりに鳴き立てていた｡しかし三重吉の云ったように､自分の顔を見てことさらに鳴く気色はさらになかった｡
自分の指からじかに餌を食うなどと云う事は無論なかった｡折々機嫌のいい時は麺麭の粉などを人指指の先へつけて竹の間からちょっと出して見る事があるが文鳥はけっして近づかない｡少し無遠慮に突き込んで見ると､文鳥は指の太いのに驚いて白い翼を乱して籠の中を騒ぎ廻るのみであった｡二三度試みた後､自分は気の毒になって､この芸だけは永久に断念してしまった｡今の世にこんな事のできるものがいるかどうだかはなはだ疑わしい｡おそらく古代の聖徒の仕事だろう｡三重吉は嘘を吐いたに違ない｡
或日の事､書斎で例のごとくﾍﾟﾝの音を立てて侘びしい事を書き連ねていると､ふと妙な音が耳に這入った｡縁側でさらさら､さらさら云う｡女が長い衣の裾を捌いているようにも受取られるが､ただの女のそれとしては､あまりに仰山である｡雛段をあるく､内裏雛の袴の襞の擦れる音とでも形容したらよかろうと思った｡自分は書きかけた小説をよそにして､ﾍﾟﾝを持ったまま縁側へ出て見た｡すると文鳥が行水を使っていた｡
水はちょうど易え立てであった｡文鳥は軽い足を水入の真中に胸毛まで浸して､時々は白い翼を左右にひろげながら､心持水入の中にしゃがむように腹を圧しつけつつ､総身の毛を一度に振っている｡そうして水入の縁にひょいと飛び上る｡しばらくしてまた飛び込む｡水入の直径は一寸五分ぐらいに過ぎない｡飛び込んだ時は尾も余り､頭も余り､背は無論余る｡水に浸かるのは足と胸だけである｡それでも文鳥は欣然として行水を使っている｡
自分は急に易籠を取って来た｡そうして文鳥をこの方へ移した｡それから如露を持って風呂場へ行って､水道の水を汲んで､籠の上からさあさあとかけてやった｡如露の水が尽きる頃には白い羽根から落ちる水が珠になって転がった｡文鳥は絶えず眼をぱちぱちさせていた｡
昔紫の帯上でいたずらをした女が､座敷で仕事をしていた時､裏二階から懐中鏡で女の顔へ春の光線を反射させて楽しんだ事がある｡女は薄紅くなった頬を上げて､繊い手を額の前に翳しながら､不思議そうに瞬をした｡この女とこの文鳥とはおそらく同じ心持だろう｡
日数が立つにしたがって文鳥は善く囀ずる｡しかしよく忘れられる｡或る時は餌壺が粟の殻だけになっていた事がある｡ある時は籠の底が糞でいっぱいになっていた事がある｡ある晩宴会があって遅く帰ったら､冬の月が硝子越に差し込んで､広い縁側がほの明るく見えるなかに､鳥籠がしんとして､箱の上に乗っていた｡その隅に文鳥の体が薄白く浮いたまま留り木の上に､有るか無きかに思われた｡自分は外套の羽根を返して､すぐ鳥籠を箱のなかへ入れてやった｡
翌日文鳥は例のごとく元気よく囀っていた｡それからは時々寒い夜も箱にしまってやるのを忘れることがあった｡ある晩いつもの通り書斎で専念にﾍﾟﾝの音を聞いていると､突然縁側の方でがたりと物の覆った音がした｡しかし自分は立たなかった｡依然として急ぐ小説を書いていた｡わざわざ立って行って､何でもないといまいましいから､気にかからないではなかったが､やはりちょっと聞耳を立てたまま知らぬ顔ですましていた｡その晩寝たのは十二時過ぎであった｡便所に行ったついで､気がかりだから､念のため一応縁側へ廻って見ると――
籠は箱の上から落ちている｡そうして横に倒れている｡水入も餌壺も引繰返っている｡粟は一面に縁側に散らばっている｡留り木は抜け出している｡文鳥はしのびやかに鳥籠の桟にかじりついていた｡自分は明日から誓ってこの縁側に猫を入れまいと決心した｡
翌日文鳥は鳴かなかった｡粟を山盛入れてやった｡水を漲るほど入れてやった｡文鳥は一本足のまま長らく留り木の上を動かなかった｡午飯を食ってから､三重吉に手紙を書こうと思って､二三行書き出すと､文鳥がちちと鳴いた｡自分は手紙の筆を留めた｡文鳥がまたちちと鳴いた｡出て見たら粟も水もだいぶん減っている｡手紙はそれぎりにして裂いて捨てた｡
翌日文鳥がまた鳴かなくなった｡留り木を下りて籠の底へ腹を圧しつけていた｡胸の所が少し膨らんで､小さい毛が漣のように乱れて見えた｡自分はこの朝､三重吉から例の件で某所まで来てくれと云う手紙を受取った｡十時までにと云う依頼であるから､文鳥をそのままにしておいて出た｡三重吉に逢って見ると例の件がいろいろ長くなって､いっしょに午飯を食う｡いっしょに晩飯を食う｡その上明日の会合まで約束して宅へ帰った｡帰ったのは夜の九時頃である｡文鳥の事はすっかり忘れていた｡疲れたから､すぐ床へ這入って寝てしまった｡
翌日眼が覚めるや否や､すぐ例の件を思いだした｡いくら当人が承知だって､そんな所へ嫁にやるのは行末よくあるまい､まだ子供だからどこへでも行けと云われる所へ行く気になるんだろう｡いったん行けばむやみに出られるものじゃない｡世の中には満足しながら不幸に陥って行く者がたくさんある｡などと考えて楊枝を使って､朝飯を済ましてまた例の件を片づけに出掛けて行った｡
帰ったのは午後三時頃である｡玄関へ外套を懸けて廊下伝いに書斎へ這入るつもりで例の縁側へ出て見ると､鳥籠が箱の上に出してあった｡けれども文鳥は籠の底に反っ繰り返っていた｡二本の足を硬く揃えて､胴と直線に伸ばしていた｡自分は籠の傍に立って､じっと文鳥を見守った｡黒い眼を眠っている｡瞼の色は薄蒼く変った｡
餌壺には粟の殻ばかり溜っている｡啄むべきは一粒もない｡水入は底の光るほど涸れている｡西へ廻った日が硝子戸を洩れて斜めに籠に落ちかかる｡台に塗った漆は､三重吉の云ったごとく､いつの間にか黒味が脱けて､朱の色が出て来た｡
自分は冬の日に色づいた朱の台を眺めた｡空になった餌壺を眺めた｡空しく橋を渡している二本の留り木を眺めた｡そうしてその下に横わる硬い文鳥を眺めた｡
自分はこごんで両手に鳥籠を抱えた｡そうして､書斎へ持って這入った｡十畳の真中へ鳥籠を卸して､その前へかしこまって､籠の戸を開いて､大きな手を入れて､文鳥を握って見た｡柔かい羽根は冷きっている｡
拳を籠から引き出して､握った手を開けると､文鳥は静に掌の上にある｡自分は手を開けたまま､しばらく死んだ鳥を見つめていた｡それから､そっと座布団の上に卸した｡そうして､烈しく手を鳴らした｡
十六になる小女が､はいと云って敷居際に手をつかえる｡自分はいきなり布団の上にある文鳥を握って､小女の前へ抛り出した｡小女は俯向いて畳を眺めたまま黙っている｡自分は､餌をやらないから､とうとう死んでしまったと云いながら､下女の顔を睥めつけた｡下女はそれでも黙っている｡
自分は机の方へ向き直った｡そうして三重吉へ端書をかいた｡｢家人が餌をやらないものだから､文鳥はとうとう死んでしまった｡たのみもせぬものを籠へ入れて､しかも餌をやる義務さえ尽くさないのは残酷の至りだ｣と云う文句であった｡
自分は､これを投函して来い､そうしてその鳥をそっちへ持って行けと下女に云った｡下女は､どこへ持って参りますかと聞き返した｡どこへでも勝手に持って行けと怒鳴りつけたら､驚いて台所の方へ持って行った｡
しばらくすると裏庭で､子供が文鳥を埋るんだ埋るんだと騒いでいる｡庭掃除に頼んだ植木屋が､御嬢さん､ここいらが好いでしょうと云っている｡自分は進まぬながら､書斎でﾍﾟﾝを動かしていた｡
翌日は何だか頭が重いので､十時頃になってようやく起きた｡顔を洗いながら裏庭を見ると､昨日植木屋の声のしたあたりに､小さい公札が､蒼い木賊の一株と並んで立っている｡高さは木賊よりもずっと低い｡庭下駄を穿いて､日影の霜を踏み砕いて､近づいて見ると､公札の表には､この土手登るべからずとあった｡筆子の手蹟である｡
午後三重吉から返事が来た｡文鳥は可愛想な事を致しましたとあるばかりで家人が悪いとも残酷だともいっこう書いてなかった｡"""
},

{
"@search.action": "upload",
"BookId": "473",
"BookName": "よだかの星",
"BookAuthor": "宮沢賢治",
"BookGenre": "短編小説",
"Release_Date": "1934年1月",
"Rating": "4.05",
"Text": """よだかは､実にみにくい鳥です｡
顔は､ところどころ､味噌をつけたようにまだらで､くちばしは､ひらたくて､耳までさけています｡
足は､まるでよぼよぼで､一間とも歩けません｡
ほかの鳥は､もう､よだかの顔を見ただけでも､いやになってしまうという工合でした｡
たとえば､ひばりも､あまり美しい鳥ではありませんが､よだかよりは､ずっと上だと思っていましたので､夕方など､よだかにあうと､さもさもいやそうに､しんねりと目をつぶりながら､首をそっ方へ向けるのでした｡もっとちいさなおしゃべりの鳥などは､いつでもよだかのまっこうから悪口をしました｡
｢ﾍﾝ｡又出て来たね｡まあ､あのざまをごらん｡ほんとうに､鳥の仲間のつらよごしだよ｡｣
｢ね､まあ､あのくちのおおきいことさ｡きっと､かえるの親類か何かなんだよ｡｣
こんな調子です｡おお､よだかでないただのたかならば､こんな生はんかのちいさい鳥は､もう名前を聞いただけでも､ぶるぶるふるえて､顔色を変えて､からだをちぢめて､木の葉のかげにでもかくれたでしょう｡ところが夜だかは､ほんとうは鷹の兄弟でも親類でもありませんでした｡かえって､よだかは､あの美しいかわせみや､鳥の中の宝石のような蜂すずめの兄さんでした｡蜂すずめは花の蜜をたべ､かわせみはお魚を食べ､夜だかは羽虫をとってたべるのでした｡それによだかには､するどい爪もするどいくちばしもありませんでしたから､どんなに弱い鳥でも､よだかをこわがる筈はなかったのです｡
それなら､たかという名のついたことは不思議なようですが､これは､一つはよだかのはねが無暗に強くて､風を切って翔けるときなどは､まるで鷹のように見えたことと､も一つはなきごえがするどくて､やはりどこか鷹に似ていた為です｡もちろん､鷹は､これをひじょうに気にかけて､いやがっていました｡それですから､よだかの顔さえ見ると､肩をいからせて､早く名前をあらためろ､名前をあらためろと､いうのでした｡
ある夕方､とうとう､鷹がよだかのうちへやって参りました｡
｢おい｡居るかい｡まだお前は名前をかえないのか｡ずいぶんお前も恥知らずだな｡お前とおれでは､よっぽど人格がちがうんだよ｡たとえばおれは､青いそらをどこまででも飛んで行く｡おまえは､曇ってうすぐらい日か､夜でなくちゃ､出て来ない｡それから､おれのくちばしやつめを見ろ｡そして､よくお前のとくらべて見るがいい｡｣
｢鷹さん｡それはあんまり無理です｡私の名前は私が勝手につけたのではありません｡神さまから下さったのです｡｣
｢いいや｡おれの名なら､神さまから貰ったのだと云ってもよかろうが､お前のは､云わば､おれと夜と､両方から借りてあるんだ｡さあ返せ｡｣
｢鷹さん｡それは無理です｡｣
｢無理じゃない｡おれがいい名を教えてやろう｡市蔵というんだ｡市蔵とな｡いい名だろう｡そこで､名前を変えるには､改名の披露というものをしないといけない｡いいか｡それはな､首へ市蔵と書いたふだをぶらさげて､私は以来市蔵と申しますと､口上を云って､みんなの所をおじぎしてまわるのだ｡｣
｢そんなことはとても出来ません｡｣
｢いいや｡出来る｡そうしろ｡もしあさっての朝までに､お前がそうしなかったら､もうすぐ､つかみ殺すぞ｡つかみ殺してしまうから､そう思え｡おれはあさっての朝早く､鳥のうちを一軒ずつまわって､お前が来たかどうかを聞いてあるく｡一軒でも来なかったという家があったら､もう貴様もその時がおしまいだぞ｡｣
｢だってそれはあんまり無理じゃありませんか｡そんなことをする位なら､私はもう死んだ方がましです｡今すぐ殺して下さい｡｣
｢まあ､よく､あとで考えてごらん｡市蔵なんてそんなにわるい名じゃないよ｡｣鷹は大きなはねを一杯にひろげて､自分の巣の方へ飛んで帰って行きました｡
よだかは､じっと目をつぶって考えました｡
(一たい僕は､なぜこうみんなにいやがられるのだろう｡僕の顔は､味噌をつけたようで､口は裂けてるからなあ｡それだって､僕は今まで､なんにも悪いことをしたことがない｡赤ん坊のめじろが巣から落ちていたときは､助けて巣へ連れて行ってやった｡そしたらめじろは､赤ん坊をまるでぬす人からでもとりかえすように僕からひきはなしたんだなあ｡それからひどく僕を笑ったっけ｡それにああ､今度は市蔵だなんて､首へふだをかけるなんて､つらいはなしだなあ｡)
あたりは､もううすくらくなっていました｡夜だかは巣から飛び出しました｡雲が意地悪く光って､低くたれています｡夜だかはまるで雲とすれすれになって､音なく空を飛びまわりました｡
それからにわかによだかは口を大きくひらいて､はねをまっすぐに張って､まるで矢のようにそらをよこぎりました｡小さな羽虫が幾匹も幾匹もその咽喉にはいりました｡
からだがつちにつくかつかないうちに､よだかはひらりとまたそらへはねあがりました｡もう雲は鼠色になり､向うの山には山焼けの火がまっ赤です｡
夜だかが思い切って飛ぶときは､そらがまるで二つに切れたように思われます｡一疋の甲虫が､夜だかの咽喉にはいって､ひどくもがきました｡よだかはすぐそれを呑みこみましたが､その時何だかせなかがぞっとしたように思いました｡
雲はもうまっくろく､東の方だけ山やけの火が赤くうつって､恐ろしいようです｡よだかはむねがつかえたように思いながら､又そらへのぼりました｡
また一疋の甲虫が､夜だかののどに､はいりました｡そしてまるでよだかの咽喉をひっかいてばたばたしました｡よだかはそれを無理にのみこんでしまいましたが､その時､急に胸がどきっとして､夜だかは大声をあげて泣き出しました｡泣きながらぐるぐるぐるぐる空をめぐったのです｡
(ああ､かぶとむしや､たくさんの羽虫が､毎晩僕に殺される｡そしてそのただ一つの僕がこんどは鷹に殺される｡それがこんなにつらいのだ｡ああ､つらい､つらい｡僕はもう虫をたべないで餓えて死のう｡いやその前にもう鷹が僕を殺すだろう｡いや､その前に､僕は遠くの遠くの空の向うに行ってしまおう｡)
山焼けの火は､だんだん水のように流れてひろがり､雲も赤く燃えているようです｡
よだかはまっすぐに､弟の川せみの所へ飛んで行きました｡きれいな川せみも､丁度起きて遠くの山火事を見ていた所でした｡そしてよだかの降りて来たのを見て云いました｡
｢兄さん｡今晩は｡何か急のご用ですか｡｣
｢いいや､僕は今度遠い所へ行くからね､その前一寸お前に遭いに来たよ｡｣
｢兄さん｡行っちゃいけませんよ｡蜂雀もあんな遠くにいるんですし､僕ひとりぼっちになってしまうじゃありませんか｡｣
｢それはね｡どうも仕方ないのだ｡もう今日は何も云わないで呉れ｡そしてお前もね､どうしてもとらなければならない時のほかはいたずらにお魚を取ったりしないようにして呉れ｡ね､さよなら｡｣
｢兄さん｡どうしたんです｡まあもう一寸お待ちなさい｡｣
｢いや､いつまで居てもおんなじだ｡はちすずめへ､あとでよろしく云ってやって呉れ｡さよなら｡もうあわないよ｡さよなら｡｣
よだかは泣きながら自分のお家へ帰って参りました｡みじかい夏の夜はもうあけかかっていました｡
羊歯の葉は､よあけの霧を吸って､青くつめたくゆれました｡よだかは高くきしきしきしと鳴きました｡そして巣の中をきちんとかたづけ､きれいにからだ中のはねや毛をそろえて､また巣から飛び出しました｡
霧がはれて､お日さまが丁度東からのぼりました｡夜だかはぐらぐらするほどまぶしいのをこらえて､矢のように､そっちへ飛んで行きました｡
｢お日さん､お日さん｡どうぞ私をあなたの所へ連れてって下さい｡灼けて死んでもかまいません｡私のようなみにくいからだでも灼けるときには小さなひかりを出すでしょう｡どうか私を連れてって下さい｡｣
行っても行っても､お日さまは近くなりませんでした｡かえってだんだん小さく遠くなりながらお日さまが云いました｡
｢お前はよだかだな｡なるほど､ずいぶんつらかろう｡今度そらを飛んで､星にそうたのんでごらん｡お前はひるの鳥ではないのだからな｡｣
夜だかはおじぎを一つしたと思いましたが､急にぐらぐらしてとうとう野原の草の上に落ちてしまいました｡そしてまるで夢を見ているようでした｡からだがずうっと赤や黄の星のあいだをのぼって行ったり､どこまでも風に飛ばされたり､又鷹が来てからだをつかんだりしたようでした｡
つめたいものがにわかに顔に落ちました｡よだかは眼をひらきました｡一本の若いすすきの葉から露がしたたったのでした｡もうすっかり夜になって､空は青ぐろく､一面の星がまたたいていました｡よだかはそらへ飛びあがりました｡今夜も山やけの火はまっかです｡よだかはその火のかすかな照りと､つめたいほしあかりの中をとびめぐりました｡それからもう一ぺん飛びめぐりました｡そして思い切って西のそらのあの美しいｵﾘｵﾝの星の方に､まっすぐに飛びながら叫びました｡
｢お星さん｡西の青じろいお星さん｡どうか私をあなたのところへ連れてって下さい｡灼けて死んでもかまいません｡｣
ｵﾘｵﾝは勇ましい歌をつづけながらよだかなどはてんで相手にしませんでした｡よだかは泣きそうになって､よろよろと落ちて､それからやっとふみとまって､もう一ぺんとびめぐりました｡それから､南の大犬座の方へまっすぐに飛びながら叫びました｡
｢お星さん｡南の青いお星さん｡どうか私をあなたの所へつれてって下さい｡やけて死んでもかまいません｡｣
大犬は青や紫や黄やうつくしくせわしくまたたきながら云いました｡
｢馬鹿を云うな｡おまえなんか一体どんなものだい｡たかが鳥じゃないか｡おまえのはねでここまで来るには､億年兆年億兆年だ｡｣そしてまた別の方を向きました｡
よだかはがっかりして､よろよろ落ちて､それから又二へん飛びめぐりました｡それから又思い切って北の大熊星の方へまっすぐに飛びながら叫びました｡
｢北の青いお星さま､あなたの所へどうか私を連れてって下さい｡｣
大熊星はしずかに云いました｡
｢余計なことを考えるものではない｡少し頭をひやして来なさい｡そう云うときは､氷山の浮いている海の中へ飛び込むか､近くに海がなかったら､氷をうかべたｺｯﾌﾟの水の中へ飛び込むのが一等だ｡｣
よだかはがっかりして､よろよろ落ちて､それから又､四へんそらをめぐりました｡そしてもう一度､東から今のぼった天の川の向う岸の鷲の星に叫びました｡
｢東の白いお星さま､どうか私をあなたの所へ連れてって下さい｡やけて死んでもかまいません｡｣
鷲は大風に云いました｡
｢いいや､とてもとても､話にも何にもならん｡星になるには､それ相応の身分でなくちゃいかん｡又よほど金もいるのだ｡｣
よだかはもうすっかり力を落してしまって､はねを閉じて､地に落ちて行きました｡そしてもう一尺で地面にその弱い足がつくというとき､よだかは俄かにのろしのようにそらへとびあがりました｡そらのなかほどへ来て､よだかはまるで鷲が熊を襲うときするように､ぶるっとからだをゆすって毛をさかだてました｡
それからｷｼｷｼｷｼｷｼｷｼｯと高く高く叫びました｡その声はまるで鷹でした｡野原や林にねむっていたほかのとりは､みんな目をさまして､ぶるぶるふるえながら､いぶかしそうにほしぞらを見あげました｡
夜だかは､どこまでも､どこまでも､まっすぐに空へのぼって行きました｡もう山焼けの火はたばこの吸殻のくらいにしか見えません｡よだかはのぼってのぼって行きました｡
寒さにいきはむねに白く凍りました｡空気がうすくなった為に､はねをそれはそれはせわしくうごかさなければなりませんでした｡
それだのに､ほしの大きさは､さっきと少しも変りません｡つくいきはふいごのようです｡寒さや霜がまるで剣のようによだかを刺しました｡よだかははねがすっかりしびれてしまいました｡そしてなみだぐんだ目をあげてもう一ぺんそらを見ました｡そうです｡これがよだかの最後でした｡もうよだかは落ちているのか､のぼっているのか､さかさになっているのか､上を向いているのかも､わかりませんでした｡ただこころもちはやすらかに､その血のついた大きなくちばしは､横にまがっては居ましたが､たしかに少しわらって居りました｡
それからしばらくたってよだかははっきりまなこをひらきました｡そして自分のからだがいま燐の火のような青い美しい光になって､しずかに燃えているのを見ました｡
すぐとなりは､ｶｼｵﾋﾟｱ座でした｡天の川の青じろいひかりが､すぐうしろになっていました｡
そしてよだかの星は燃えつづけました｡いつまでもいつまでも燃えつづけました｡
今でもまだ燃えています｡"""
}
]


In [8]:
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

Upload of new document succeeded: True


In [9]:
results =  search_client.search(query_type='simple',
    search_text="*" ,
    select='BookName,BookAuthor',
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["BookName"])
    print(f"BookAuthor: {result['BookAuthor']}")

Total Documents Matching Query: 1
1.0
蜘蛛の糸
BookAuthor: 芥川龍之介


In [10]:
results =  search_client.search(query_type='simple',
    search_text="What book did Ryunosuke Akutagawa write?" ,
    select='BookName,BookAuthor,Rating')

for result in results:
    print(result["@search.score"])
    print(result["BookName"])
    print(f"Text: {result['Text']}")

In [11]:
results =  search_client.search(query_type='semantic', semantic_configuration_name='my-semantic-config',
    search_text="What books are highly rated", 
    select='BookName,BookAuthor,Rating', query_caption='extractive')

for result in results:
    print(result["@search.reranker_score"])
    print(result["BookName"])
    print(f"Text: {result['Text']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")

In [12]:
results =  search_client.search(query_type='semantic', query_language='ja-jp', semantic_configuration_name='my-semantic-config',
 search_text="What books are highly rated?", 
 select='BookName,BookAuthor,Rating', query_caption='extractive', query_answer="extractive",)

for result in results:
    print(result["@search.reranker_score"])
    print(result["BookName"])
    print(f"Text: {result['Text']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")